Connected to thesis (Python 3.11.4)

In [1]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import KFold
from datetime import datetime
#from torch.utils.tensorboard import SummaryWriter
import torchaudio
from typing import List, Tuple
from pt_utils import *
from pt_dataset import *
from pt_models import *
from pt_utils import *
from tensorboardX import SummaryWriter
from transformers import AutoModel, AdamW, get_cosine_schedule_with_warmup

def create_run_directory():
    base_dir = "pt_runs_SOTA"
    timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
    run_dir = os.path.join(base_dir, timestamp)
    os.makedirs(run_dir, exist_ok=True)
    return run_dir

def _calculate_flattened_accuracy(average, ground_truth_labels):
    s_acc = 0
    for b in range(len(ground_truth_labels)):
        s, _ = scipy.stats.pearsonr(average[b], ground_truth_labels[b])
        s_acc += s
    return s_acc / len(ground_truth_labels)

def _choose_real_labs_only_with_filenames(labels, filenames):
    return labels[labels['filename'].isin(filenames)]

def _get_ground_truth_labels(ground_truth_names, labels):
    ground_truth_labels = []
    for batch_name in ground_truth_names:
        ground_truth_label = _choose_real_labs_only_with_filenames(labels, [batch_name])
        ground_truth_labels.append(ground_truth_label)
    return np.array(ground_truth_labels)[:, :, -1].astype(np.float32)

# Function to gradually unfreeze layers
def gradually_unfreeze(model, epoch, total_epochs, initial_unfreeze=2):
    total_layers = len(model.wav_model.encoder.layers)
    layers_to_unfreeze = initial_unfreeze + (epoch * (total_layers - initial_unfreeze) // total_epochs)
    model.unfreeze_last_n_blocks(layers_to_unfreeze)
    

def train(path_to_data, path_to_labels, window_size=16, step_size=6, data_parts=4, epochs=100, batch_size=10, early_stopping_patience=20, config = None, processor = None):
    run_dir = create_run_directory()
    log_dir = os.path.join(run_dir, "logs")
    os.makedirs(log_dir, exist_ok=True)

    # Parameters
    length_sequence = window_size 
    step_sequence = step_size

    # Load and prepare data
    train_data, train_labels, train_dict, frame_rate = load_data(path_to_data, path_to_labels, 'train')
    devel_data, devel_labels, devel_dict, frame_rate = load_data(path_to_data, path_to_labels, 'devel')
    test_data, test_labels, test_dict, frame_rate = load_data(path_to_data, path_to_labels, 'test')

    # Combine train and devel data
    all_data = np.concatenate((train_data, devel_data), axis=0)
    all_labels = pd.concat([train_labels, devel_labels])
    all_dict = np.concatenate((list(train_dict.values()), list(devel_dict.values())), axis=0)

    # Prepare data
    prepared_data, prepared_labels, prepared_labels_timesteps = prepare_data(all_data, all_labels, all_dict, frame_rate, length_sequence * 16000, step_sequence * 16000)
    prepared_test_data, prepared_test_labels, prepared_test_labels_timesteps = prepare_data(test_data, test_labels, test_dict, frame_rate, length_sequence * 16000, step_sequence * 16000)

    # Create CSV file for storing fold indices
    fold_indices_df = pd.DataFrame(columns=['Fold', 'Train_Indices', 'Val_Indices'])

    # Cross-validation
    kf = KFold(n_splits=data_parts)
    fold_metrics = []
    # To accumulate metrics across folds for each epoch
    train_acc_epoch = []
    val_acc_epoch = []
    test_acc_epoch = []

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    
    config["output_size"] = prepared_labels.shape[-1]
    writer = SummaryWriter(log_dir=os.path.join(log_dir,config["model_name"]))

    for fold, (train_index, val_index) in enumerate(kf.split(prepared_data)):
        print(f"Fold {fold + 1}/{data_parts}")
        best_model_path = f"{run_dir}/best_model_fold{fold+1}"
        # Save fold indices
        fold_indices_df = fold_indices_df._append({
            'Fold': fold + 1,
            'Train_Indices': train_index.tolist(),
            'Val_Indices': val_index.tolist()
        }, ignore_index=True)

        # Split data
        train_d, val_d = prepared_data[train_index], prepared_data[val_index]
        train_lbs, val_lbs = prepared_labels[train_index], prepared_labels[val_index]
        train_timesteps, val_timesteps = prepared_labels_timesteps[train_index], prepared_labels_timesteps[val_index]
        
        # Reshape data
        train_d, train_lbs = reshaping_data_for_model(train_d, train_lbs)
        val_d, val_lbs = reshaping_data_for_model(val_d, val_lbs)
        test_d, test_lbs = reshaping_data_for_model(prepared_test_data, prepared_test_labels)
        
        print(train_d.shape)

        # Create datasets
        #train_dataset = BreathingDataset(train_d, train_lbs, processor, window_size, step_sequence, augment=True)
        train_dataset = BreathingDataset(train_d, train_lbs, processor,window_size, step_sequence)
        val_dataset = BreathingDataset(val_d, val_lbs, processor, window_size, step_sequence)
        test_dataset = BreathingDataset(test_d, test_lbs, processor, window_size, step_sequence)

        # Create DataLoaders
        train_loader = DataLoader(train_dataset, batch_size=batch_size,num_workers=2, shuffle=True, collate_fn=train_dataset.collate_fn)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, num_workers=1, collate_fn=val_dataset.collate_fn)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, num_workers=1, collate_fn=test_dataset.collate_fn)
        print(config["output_size"])
        # Create and initialize model
        model = config["model"](config).to(device)
        
        #### training optimiser parameters fror apple
        #learning_rate = 0.005 
        #optimizer = optim.Adam(model.parameters(), lr=learning_rate)

        #### training optimiser parameters fror harma_2023 VRB model 
        learning_rate = 0.01 # From the paper
        optimizer = optim.SGD(model.parameters(), lr=learning_rate)       
        
        best_val_loss = float('inf')
        best_val_loss_flat = float('inf')
        early_stopping_counter = 0
        # To accumulate metrics across folds for each epoch
        train_acc = []
        val_acc = []
        test_acc = []
        for epoch in range(epochs):

            model.train()
            train_loss = 0.0
            progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")
            
            for batch_d, batch_lbs in progress_bar:
                optimizer.zero_grad()
                input_values = batch_d.to(device)
                batch_lbs = batch_lbs.to(device)
                outputs = model(input_values)
                loss = correlation_coefficient_loss(outputs, batch_lbs)
                loss.backward()
                optimizer.step()
                train_loss += loss.item()
                progress_bar.set_postfix({'train_loss': f'{train_loss/(progress_bar.n+1):.4f}'})


            train_loss /= len(train_loader)

            # Combined validation loop
            model.eval()
            val_loss = 0.0
            val_pred = []
            with torch.no_grad():
                for batch_d, batch_lbs in val_loader:
                    input_values = batch_d.to(device)
                    batch_lbs = batch_lbs.to(device)
                    
                    outputs = model(input_values)
                    loss = correlation_coefficient_loss(outputs, batch_lbs)
                    val_loss += loss.item()
                    val_pred.extend(outputs.cpu().numpy())

            val_loss /= len(val_loader)

            # Calculate validation metrics
            val_pred = np.array(val_pred).reshape(val_timesteps.shape)
            val_ground_truth = _get_ground_truth_labels([all_dict[i] for i in val_index], all_labels)
            val_pred_flat = unsplit_data_ogsize(val_pred, window_size, step_sequence, 25, val_ground_truth.shape[-1])
            val_prc_coef = _calculate_flattened_accuracy(val_pred_flat, val_ground_truth)
            
            # Accumulate metrics for this fold and epoch
            train_acc.append(1- train_loss)
            val_acc.append(1- val_loss)

            # Log metrics
            writer.add_scalar(f"Loss/train_fold_{fold + 1}", train_loss, epoch)
            writer.add_scalar(f"Loss/val_fold_{fold + 1}", val_loss, epoch)
            writer.add_scalar(f"Pearson/val_fold_{fold + 1}", val_prc_coef, epoch)

            print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Pearson: {val_prc_coef:.4f}")

            # Check if validation loss improved
            if val_loss < best_val_loss:
                print(f"Validation loss improved from {best_val_loss:.4f} to {val_loss:.4f}. Saving best model...")
                best_val_loss = val_loss
                best_val_loss_flat = val_prc_coef
                early_stopping_counter = 0

                # Save the best model
                torch.save(model.state_dict(), best_model_path)
            else:
                early_stopping_counter += 1
                print(f"Validation loss did not improve for {early_stopping_counter} epochs.")
                #model.load_state_dict(torch.load(best_model_path))


            # Early stopping
            if early_stopping_counter >= early_stopping_patience:
                print(f"Early stopping triggered at epoch {epoch + 1}. Loading best model.")
                # Load the best model's weights
                model.load_state_dict(torch.load(best_model_path))
                break

        test_pred = []
        test_loss = 0.0
        with torch.no_grad():
            for batch_d, batch_lbs in test_loader:
                input_values = batch_d.to(device)
                batch_lbs = batch_lbs.to(device)
                
                outputs = model(input_values)
                loss = correlation_coefficient_loss(outputs, batch_lbs)
                test_loss += loss.item()
                test_pred.extend(outputs.cpu().numpy())

        test_loss /= len(test_loader)
        test_pred = np.array(test_pred).reshape(prepared_test_labels_timesteps.shape)
        test_ground_truth = _get_ground_truth_labels(list(test_dict.values()), test_labels)
        test_pred_flat = unsplit_data_ogsize(test_pred, window_size, step_sequence, 25, test_ground_truth.shape[-1])
        test_prc_coef = _calculate_flattened_accuracy(test_pred_flat, test_ground_truth)

        print(f"Fold {fold + 1}:")
        print(f"  Validation Pearson Coefficient  acc: {1- best_val_loss}")
        print(f"  Validation Pearson Coefficient flat acc: {best_val_loss_flat}")
        print(f"  Test acc: {1- test_loss}")
        print(f"  Test Pearson Coefficient acc(flattened): {test_prc_coef}")

        fold_metrics.append({
            'Fold': fold + 1,
            'val_prc_acc': 1- best_val_loss,
            'val_prc_acc_flat': best_val_loss_flat,
            'test_acc': 1- test_loss,
            'test_prc_flat': test_prc_coef
        })


                # Log fold-specific metrics as tables
        fold_table = f"| Fold | Val Pearson Acc | Val Pearson Flat | Test Acc | Test Pearson Flat |\n" \
                     f"|------|-----------------|------------------|----------|-------------------|\n" \
                     f"| {fold + 1} | {1 - val_loss:.4f} | {val_prc_coef:.4f} | {1 - test_loss:.4f} | {test_prc_coef:.4f} |\n"
        writer.add_text(f"Fold_{fold + 1}_Metrics", fold_table)
        # Accumulate fold metrics across all folds
        train_acc_epoch.append(train_acc)
        val_acc_epoch.append(train_acc)


    
        # After all folds, compute and log the average metrics per epoch across all folds
    for epoch in range(epochs):
        avg_train_loss = np.mean([fold_losses[epoch] for fold_losses in train_acc_epoch if len(fold_losses) > epoch])
        avg_val_loss = np.mean([fold_losses[epoch] for fold_losses in val_acc_epoch if len(fold_losses) > epoch])

        # Log the averaged metrics for the epoch across all folds
        writer.add_scalar("Average_acc/train", avg_train_loss, epoch)
        writer.add_scalar("Average_acc/val", avg_val_loss, epoch)
            

    # Calculate average metrics
    avg_metrics = {key: np.mean([fold[key] for fold in fold_metrics if key != 'Fold']) for key in fold_metrics[0].keys() if key != 'Fold'}
        # Log the final average table
    avg_table = "| Fold | Val Pearson Acc | Val Pearson Flat | Test Acc | Test Pearson Flat |\n" \
                "|------|-----------------|------------------|----------|-------------------|\n" \
                f"| Average | {avg_metrics['val_prc_acc']:.4f} | {avg_metrics['val_prc_acc_flat']:.4f} | {avg_metrics['test_acc']:.4f} | {avg_metrics['test_prc_flat']:.4f} |\n"
    writer.add_text("Average_Metrics", avg_table)
    # Add average metrics to results
    avg_metrics['Fold'] = 'Average'
    fold_metrics.append(avg_metrics)

    # save averga date to CSV
    results_df = pd.DataFrame(fold_metrics)
    csv_path = os.path.join(run_dir, 'fold_results.csv')
    results_df.to_csv(csv_path, index=False)
    
    # Save fold indices CSV
    fold_indices_df.to_csv(os.path.join(run_dir, 'fold_indices.csv'), index=False)
    
    writer.close()


    print("\nTraining completed.")
    print("Average metrics across all folds:")
    for key, value in avg_metrics.items():
        print(f"  {key}: {value}")


if __name__ == "__main__":
    ## Path to data
    #path = "/home/glenn/Downloads/"
    path = "../DATA/"


    # Model parameters
    model_config = {
        "VRBModel": {
            "model" : VRBModel,
            "model_name": "facebook/hubert-large-ls960-ft",
            "hidden_units": 64,
            "n_gru": 3,
            "output_size": None  # Will be set dynamically
        },
        "Wav2Vec2ConvLSTMModel": {
            "model" : Wav2Vec2ConvLSTMModel,
            "model_name": "facebook/wav2vec2-base",
            "hidden_units": 128,
            "n_lstm": 2,
            "output_size": None  # Will be set dynamically
        }
    }
    

    
    # Train and data parameters
    epochs = 128
    batch_size = 128
    window_size = 30
    step_size = 6
    data_parts = 4 # aka folds
    early_stopping_patience = 15
    
    config = model_config["VRBModel"]
    

    ## same wav2vec2 base model and pipeline used in the paper
    processor = AutoProcessor.from_pretrained(config["model_name"])
    
    train(
        path_to_data=path+"ComParE2020_Breathing/wav/",
        path_to_labels=path+"ComParE2020_Breathing/lab/",
        window_size=window_size,
        batch_size=batch_size,
        config = config,
        step_size=step_size,
        data_parts= data_parts ,
        early_stopping_patience= early_stopping_patience,
        epochs= epochs,
        processor = processor
    )

Fold 1/4
(864, 480000)
750


Epoch 1/128: 100%|██████████| 7/7 [00:35<00:00,  5.11s/it, train_loss=1.0063]


Epoch 1/128 - Train Loss: 1.0063, Val Loss: 1.0034, Val Pearson: -0.0086
Validation loss improved from inf to 1.0034. Saving best model...


RuntimeError: [enforce fail at inline_container.cc:603] . unexpected pos 98176 vs 98064

Restarted thesis (Python 3.11.4)

In [1]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import KFold
from datetime import datetime
#from torch.utils.tensorboard import SummaryWriter
import torchaudio
from typing import List, Tuple
from pt_utils import *
from pt_dataset import *
from pt_models import *
from pt_utils import *
from tensorboardX import SummaryWriter
from transformers import get_cosine_schedule_with_warmup


def create_run_directory():
    base_dir = "pt_runs_SOTA"
    timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
    run_dir = os.path.join(base_dir, timestamp)
    os.makedirs(run_dir, exist_ok=True)
    return run_dir

def _calculate_flattened_accuracy(average, ground_truth_labels):
    s_acc = 0
    for b in range(len(ground_truth_labels)):
        s, _ = scipy.stats.pearsonr(average[b], ground_truth_labels[b])
        s_acc += s
    return s_acc / len(ground_truth_labels)

def _choose_real_labs_only_with_filenames(labels, filenames):
    return labels[labels['filename'].isin(filenames)]

def _get_ground_truth_labels(ground_truth_names, labels):
    ground_truth_labels = []
    for batch_name in ground_truth_names:
        ground_truth_label = _choose_real_labs_only_with_filenames(labels, [batch_name])
        ground_truth_labels.append(ground_truth_label)
    return np.array(ground_truth_labels)[:, :, -1].astype(np.float32)

def train(path_to_data, path_to_labels, window_size=16, step_size=6, data_parts=4, epochs=100, batch_size=10, early_stopping_patience=20, config = None, processor = None):
    run_dir = create_run_directory()
    log_dir = os.path.join(run_dir, "logs")
    os.makedirs(log_dir, exist_ok=True)

    # Parameters
    length_sequence = window_size 
    step_sequence = step_size

    # Load and prepare data
    train_data, train_labels, train_dict, frame_rate = load_data(path_to_data, path_to_labels, 'train')
    devel_data, devel_labels, devel_dict, frame_rate = load_data(path_to_data, path_to_labels, 'devel')
    test_data, test_labels, test_dict, frame_rate = load_data(path_to_data, path_to_labels, 'test')

    # Combine train and devel data
    all_data = np.concatenate((train_data, devel_data), axis=0)
    all_labels = pd.concat([train_labels, devel_labels])
    all_dict = np.concatenate((list(train_dict.values()), list(devel_dict.values())), axis=0)

    # Prepare data
    prepared_data, prepared_labels, prepared_labels_timesteps = prepare_data(all_data, all_labels, all_dict, frame_rate, length_sequence * 16000, step_sequence * 16000)
    prepared_test_data, prepared_test_labels, prepared_test_labels_timesteps = prepare_data(test_data, test_labels, test_dict, frame_rate, length_sequence * 16000, step_sequence * 16000)

    # Create CSV file for storing fold indices
    fold_indices_df = pd.DataFrame(columns=['Fold', 'Train_Indices', 'Val_Indices'])

    # Cross-validation
    kf = KFold(n_splits=data_parts)
    fold_metrics = []
    # To accumulate metrics across folds for each epoch
    train_acc_epoch = []
    val_acc_epoch = []
    test_acc_epoch = []

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    
    config["output_size"] = prepared_labels.shape[-1]
    writer = SummaryWriter(log_dir=os.path.join(log_dir,config["model_name"]))

    for fold, (train_index, val_index) in enumerate(kf.split(prepared_data)):
        print(f"Fold {fold + 1}/{data_parts}")
        best_model_path = f"{run_dir}/best_model_fold{fold+1}"
        # Save fold indices
        fold_indices_df = fold_indices_df._append({
            'Fold': fold + 1,
            'Train_Indices': train_index.tolist(),
            'Val_Indices': val_index.tolist()
        }, ignore_index=True)

        # Split data
        train_d, val_d = prepared_data[train_index], prepared_data[val_index]
        train_lbs, val_lbs = prepared_labels[train_index], prepared_labels[val_index]
        train_timesteps, val_timesteps = prepared_labels_timesteps[train_index], prepared_labels_timesteps[val_index]
        
        # Reshape data
        train_d, train_lbs = reshaping_data_for_model(train_d, train_lbs)
        val_d, val_lbs = reshaping_data_for_model(val_d, val_lbs)
        test_d, test_lbs = reshaping_data_for_model(prepared_test_data, prepared_test_labels)
        
        print(train_d.shape)

        # Create datasets
        #train_dataset = BreathingDataset(train_d, train_lbs, processor, window_size, step_sequence, augment=True)
        train_dataset = BreathingDataset(train_d, train_lbs, processor,window_size, step_sequence)
        val_dataset = BreathingDataset(val_d, val_lbs, processor, window_size, step_sequence)
        test_dataset = BreathingDataset(test_d, test_lbs, processor, window_size, step_sequence)

        # Create DataLoaders
        train_loader = DataLoader(train_dataset, batch_size=batch_size,num_workers=2, shuffle=True, collate_fn=train_dataset.collate_fn)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, num_workers=1, collate_fn=val_dataset.collate_fn)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, num_workers=1, collate_fn=test_dataset.collate_fn)
        print(config["output_size"])
        # Create and initialize model
        model = config["model"](config).to(device)
        
        #### training optimiser parameters fror apple
        #learning_rate = 0.005 
        #optimizer = optim.Adam(model.parameters(), lr=learning_rate)

        #### training optimiser parameters fror harma_2023 VRB model 
        learning_rate = 0.01 # From the paper
        optimizer = optim.SGD(model.parameters(), lr=learning_rate)       
        
        best_val_loss = float('inf')
        best_val_loss_flat = float('inf')
        early_stopping_counter = 0
        # To accumulate metrics across folds for each epoch
        train_acc = []
        val_acc = []
        test_acc = []
        for epoch in range(epochs):

            model.train()
            train_loss = 0.0
            progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")
            
            for batch_d, batch_lbs in progress_bar:
                optimizer.zero_grad()
                input_values = batch_d.to(device)
                batch_lbs = batch_lbs.to(device)
                outputs = model(input_values)
                loss = correlation_coefficient_loss(outputs, batch_lbs)
                loss.backward()
                optimizer.step()
                train_loss += loss.item()
                progress_bar.set_postfix({'train_loss': f'{train_loss/(progress_bar.n+1):.4f}'})


            train_loss /= len(train_loader)

            # Combined validation loop
            model.eval()
            val_loss = 0.0
            val_pred = []
            with torch.no_grad():
                for batch_d, batch_lbs in val_loader:
                    input_values = batch_d.to(device)
                    batch_lbs = batch_lbs.to(device)
                    
                    outputs = model(input_values)
                    loss = correlation_coefficient_loss(outputs, batch_lbs)
                    val_loss += loss.item()
                    val_pred.extend(outputs.cpu().numpy())

            val_loss /= len(val_loader)

            # Calculate validation metrics
            val_pred = np.array(val_pred).reshape(val_timesteps.shape)
            val_ground_truth = _get_ground_truth_labels([all_dict[i] for i in val_index], all_labels)
            val_pred_flat = unsplit_data_ogsize(val_pred, window_size, step_sequence, 25, val_ground_truth.shape[-1])
            val_prc_coef = _calculate_flattened_accuracy(val_pred_flat, val_ground_truth)
            
            # Accumulate metrics for this fold and epoch
            train_acc.append(1- train_loss)
            val_acc.append(1- val_loss)

            # Log metrics
            writer.add_scalar(f"Loss/train_fold_{fold + 1}", train_loss, epoch)
            writer.add_scalar(f"Loss/val_fold_{fold + 1}", val_loss, epoch)
            writer.add_scalar(f"Pearson/val_fold_{fold + 1}", val_prc_coef, epoch)

            print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Pearson: {val_prc_coef:.4f}")

            # Check if validation loss improved
            if val_loss < best_val_loss:
                print(f"Validation loss improved from {best_val_loss:.4f} to {val_loss:.4f}. Saving best model...")
                best_val_loss = val_loss
                best_val_loss_flat = val_prc_coef
                early_stopping_counter = 0

                # Save the best model
                torch.save(model.state_dict(), best_model_path)
            else:
                early_stopping_counter += 1
                print(f"Validation loss did not improve for {early_stopping_counter} epochs.")
                #model.load_state_dict(torch.load(best_model_path))


            # Early stopping
            if early_stopping_counter >= early_stopping_patience:
                print(f"Early stopping triggered at epoch {epoch + 1}. Loading best model.")
                # Load the best model's weights
                model.load_state_dict(torch.load(best_model_path))
                break

        test_pred = []
        test_loss = 0.0
        with torch.no_grad():
            for batch_d, batch_lbs in test_loader:
                input_values = batch_d.to(device)
                batch_lbs = batch_lbs.to(device)
                
                outputs = model(input_values)
                loss = correlation_coefficient_loss(outputs, batch_lbs)
                test_loss += loss.item()
                test_pred.extend(outputs.cpu().numpy())

        test_loss /= len(test_loader)
        test_pred = np.array(test_pred).reshape(prepared_test_labels_timesteps.shape)
        test_ground_truth = _get_ground_truth_labels(list(test_dict.values()), test_labels)
        test_pred_flat = unsplit_data_ogsize(test_pred, window_size, step_sequence, 25, test_ground_truth.shape[-1])
        test_prc_coef = _calculate_flattened_accuracy(test_pred_flat, test_ground_truth)

        print(f"Fold {fold + 1}:")
        print(f"  Validation Pearson Coefficient  acc: {1- best_val_loss}")
        print(f"  Validation Pearson Coefficient flat acc: {best_val_loss_flat}")
        print(f"  Test acc: {1- test_loss}")
        print(f"  Test Pearson Coefficient acc(flattened): {test_prc_coef}")

        fold_metrics.append({
            'Fold': fold + 1,
            'val_prc_acc': 1- best_val_loss,
            'val_prc_acc_flat': best_val_loss_flat,
            'test_acc': 1- test_loss,
            'test_prc_flat': test_prc_coef
        })


                # Log fold-specific metrics as tables
        fold_table = f"| Fold | Val Pearson Acc | Val Pearson Flat | Test Acc | Test Pearson Flat |\n" \
                     f"|------|-----------------|------------------|----------|-------------------|\n" \
                     f"| {fold + 1} | {1 - val_loss:.4f} | {val_prc_coef:.4f} | {1 - test_loss:.4f} | {test_prc_coef:.4f} |\n"
        writer.add_text(f"Fold_{fold + 1}_Metrics", fold_table)
        # Accumulate fold metrics across all folds
        train_acc_epoch.append(train_acc)
        val_acc_epoch.append(train_acc)


    
        # After all folds, compute and log the average metrics per epoch across all folds
    for epoch in range(epochs):
        avg_train_loss = np.mean([fold_losses[epoch] for fold_losses in train_acc_epoch if len(fold_losses) > epoch])
        avg_val_loss = np.mean([fold_losses[epoch] for fold_losses in val_acc_epoch if len(fold_losses) > epoch])

        # Log the averaged metrics for the epoch across all folds
        writer.add_scalar("Average_acc/train", avg_train_loss, epoch)
        writer.add_scalar("Average_acc/val", avg_val_loss, epoch)
            

    # Calculate average metrics
    avg_metrics = {key: np.mean([fold[key] for fold in fold_metrics if key != 'Fold']) for key in fold_metrics[0].keys() if key != 'Fold'}
        # Log the final average table
    avg_table = "| Fold | Val Pearson Acc | Val Pearson Flat | Test Acc | Test Pearson Flat |\n" \
                "|------|-----------------|------------------|----------|-------------------|\n" \
                f"| Average | {avg_metrics['val_prc_acc']:.4f} | {avg_metrics['val_prc_acc_flat']:.4f} | {avg_metrics['test_acc']:.4f} | {avg_metrics['test_prc_flat']:.4f} |\n"
    writer.add_text("Average_Metrics", avg_table)
    # Add average metrics to results
    avg_metrics['Fold'] = 'Average'
    fold_metrics.append(avg_metrics)

    # save averga date to CSV
    results_df = pd.DataFrame(fold_metrics)
    csv_path = os.path.join(run_dir, 'fold_results.csv')
    results_df.to_csv(csv_path, index=False)
    
    # Save fold indices CSV
    fold_indices_df.to_csv(os.path.join(run_dir, 'fold_indices.csv'), index=False)
    
    writer.close()


    print("\nTraining completed.")
    print("Average metrics across all folds:")
    for key, value in avg_metrics.items():
        print(f"  {key}: {value}")


if __name__ == "__main__":
    ## Path to data
    #path = "/home/glenn/Downloads/"
    path = "../DATA/"


    # Model parameters
    model_config = {
        "VRBModel": {
            "model" : VRBModel,
            "model_name": "facebook/hubert-large-ls960-ft",
            "hidden_units": 64,
            "n_gru": 3,
            "output_size": None  # Will be set dynamically
        },
        "Wav2Vec2ConvLSTMModel": {
            "model" : Wav2Vec2ConvLSTMModel,
            "model_name": "facebook/wav2vec2-base",
            "hidden_units": 128,
            "n_lstm": 2,
            "output_size": None  # Will be set dynamically
        }
    }
    

    
    # Train and data parameters
    epochs = 128
    batch_size = 128
    window_size = 30
    step_size = 6
    data_parts = 4 # aka folds
    early_stopping_patience = 3
    
    config = model_config["VRBModel"]
    

    ## same wav2vec2 base model and pipeline used in the paper
    processor = AutoProcessor.from_pretrained(config["model_name"])
    
    train(
        path_to_data=path+"ComParE2020_Breathing/wav/",
        path_to_labels=path+"ComParE2020_Breathing/lab/",
        window_size=window_size,
        batch_size=batch_size,
        config = config,
        step_size=step_size,
        data_parts= data_parts ,
        early_stopping_patience= early_stopping_patience,
        epochs= epochs,
        processor = processor
    )

Fold 1/4
(864, 480000)
750


Epoch 1/128: 100%|██████████| 7/7 [00:37<00:00,  5.30s/it, train_loss=1.0028]


Epoch 1/128 - Train Loss: 1.0028, Val Loss: 1.0063, Val Pearson: -0.0148
Validation loss improved from inf to 1.0063. Saving best model...


Epoch 2/128: 100%|██████████| 7/7 [00:34<00:00,  4.87s/it, train_loss=0.9925]


Epoch 2/128 - Train Loss: 0.9925, Val Loss: 1.0088, Val Pearson: -0.0156
Validation loss did not improve for 1 epochs.


...

Epoch 4/128: 100%|██████████| 7/7 [00:36<00:00,  5.16s/it, train_loss=0.9850]


Epoch 4/128 - Train Loss: 0.9850, Val Loss: 1.0110, Val Pearson: -0.0160
Validation loss did not improve for 3 epochs.
Early stopping triggered at epoch 4. Loading best model.


<ipython-input-1-4bad5d9317cb>:208: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(best_model_path))


Fold 1:
  Validation Pearson Coefficient  acc: -0.0062912702560424805
  Validation Pearson Coefficient flat acc: -0.014798802708906985
  Test acc: -0.009982633590698287
  Test Pearson Coefficient acc(flattened): -0.01610076019429338
Fold 2/4
(900, 480000)
750


Epoch 1/128:  25%|██▌       | 2/8 [00:12<00:35,  5.99s/it, train_loss=1.0000]

Restarted thesis (Python 3.11.4)

In [1]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import KFold
from datetime import datetime
#from torch.utils.tensorboard import SummaryWriter
import torchaudio
from typing import List, Tuple
from pt_utils import *
from pt_dataset import *
from pt_models import *
from pt_utils import *
from tensorboardX import SummaryWriter
from transformers import get_cosine_schedule_with_warmup


def create_run_directory():
    base_dir = "pt_runs_SOTA"
    timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
    run_dir = os.path.join(base_dir, timestamp)
    os.makedirs(run_dir, exist_ok=True)
    return run_dir

def _calculate_flattened_accuracy(average, ground_truth_labels):
    s_acc = 0
    for b in range(len(ground_truth_labels)):
        s, _ = scipy.stats.pearsonr(average[b], ground_truth_labels[b])
        s_acc += s
    return s_acc / len(ground_truth_labels)

def _choose_real_labs_only_with_filenames(labels, filenames):
    return labels[labels['filename'].isin(filenames)]

def _get_ground_truth_labels(ground_truth_names, labels):
    ground_truth_labels = []
    for batch_name in ground_truth_names:
        ground_truth_label = _choose_real_labs_only_with_filenames(labels, [batch_name])
        ground_truth_labels.append(ground_truth_label)
    return np.array(ground_truth_labels)[:, :, -1].astype(np.float32)

def train(path_to_data, path_to_labels, window_size=16, step_size=6, data_parts=4, epochs=100, batch_size=10, early_stopping_patience=20, config = None, processor = None):
    run_dir = create_run_directory()
    log_dir = os.path.join(run_dir, "logs")
    os.makedirs(log_dir, exist_ok=True)

    # Parameters
    length_sequence = window_size 
    step_sequence = step_size

    # Load and prepare data
    train_data, train_labels, train_dict, frame_rate = load_data(path_to_data, path_to_labels, 'train')
    devel_data, devel_labels, devel_dict, frame_rate = load_data(path_to_data, path_to_labels, 'devel')
    test_data, test_labels, test_dict, frame_rate = load_data(path_to_data, path_to_labels, 'test')

    # Combine train and devel data
    all_data = np.concatenate((train_data, devel_data), axis=0)
    all_labels = pd.concat([train_labels, devel_labels])
    all_dict = np.concatenate((list(train_dict.values()), list(devel_dict.values())), axis=0)

    # Prepare data
    prepared_data, prepared_labels, prepared_labels_timesteps = prepare_data(all_data, all_labels, all_dict, frame_rate, length_sequence * 16000, step_sequence * 16000)
    prepared_test_data, prepared_test_labels, prepared_test_labels_timesteps = prepare_data(test_data, test_labels, test_dict, frame_rate, length_sequence * 16000, step_sequence * 16000)

    # Create CSV file for storing fold indices
    fold_indices_df = pd.DataFrame(columns=['Fold', 'Train_Indices', 'Val_Indices'])

    # Cross-validation
    kf = KFold(n_splits=data_parts)
    fold_metrics = []
    # To accumulate metrics across folds for each epoch
    train_acc_epoch = []
    val_acc_epoch = []
    test_acc_epoch = []

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    
    config["output_size"] = prepared_labels.shape[-1]
    writer = SummaryWriter(log_dir=os.path.join(log_dir,config["model_name"]))

    for fold, (train_index, val_index) in enumerate(kf.split(prepared_data)):
        print(f"Fold {fold + 1}/{data_parts}")
        best_model_path = f"{run_dir}/best_model_fold{fold+1}"
        # Save fold indices
        fold_indices_df = fold_indices_df._append({
            'Fold': fold + 1,
            'Train_Indices': train_index.tolist(),
            'Val_Indices': val_index.tolist()
        }, ignore_index=True)

        # Split data
        train_d, val_d = prepared_data[train_index], prepared_data[val_index]
        train_lbs, val_lbs = prepared_labels[train_index], prepared_labels[val_index]
        train_timesteps, val_timesteps = prepared_labels_timesteps[train_index], prepared_labels_timesteps[val_index]
        
        # Reshape data
        train_d, train_lbs = reshaping_data_for_model(train_d, train_lbs)
        val_d, val_lbs = reshaping_data_for_model(val_d, val_lbs)
        test_d, test_lbs = reshaping_data_for_model(prepared_test_data, prepared_test_labels)
        
        print(train_d.shape)

        # Create datasets
        #train_dataset = BreathingDataset(train_d, train_lbs, processor, window_size, step_sequence, augment=True)
        train_dataset = BreathingDataset(train_d, train_lbs, processor,window_size, step_sequence)
        val_dataset = BreathingDataset(val_d, val_lbs, processor, window_size, step_sequence)
        test_dataset = BreathingDataset(test_d, test_lbs, processor, window_size, step_sequence)

        # Create DataLoaders
        train_loader = DataLoader(train_dataset, batch_size=batch_size,num_workers=2, shuffle=True, collate_fn=train_dataset.collate_fn)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, num_workers=1, collate_fn=val_dataset.collate_fn)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, num_workers=1, collate_fn=test_dataset.collate_fn)
        print(config["output_size"])
        # Create and initialize model
        model = config["model"](config).to(device)
        
        #### training optimiser parameters fror apple
        #learning_rate = 0.005 
        #optimizer = optim.Adam(model.parameters(), lr=learning_rate)

        #### training optimiser parameters fror harma_2023 VRB model 
        learning_rate = 0.01 # From the paper
        optimizer = optim.SGD(model.parameters(), lr=learning_rate)       
        
        best_val_loss = float('inf')
        best_val_loss_flat = float('inf')
        early_stopping_counter = 0
        # To accumulate metrics across folds for each epoch
        train_acc = []
        val_acc = []
        test_acc = []
        for epoch in range(epochs):

            model.train()
            train_loss = 0.0
            progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")
            
            for batch_d, batch_lbs in progress_bar:
                optimizer.zero_grad()
                input_values = batch_d.to(device)
                batch_lbs = batch_lbs.to(device)
                outputs = model(input_values)
                loss = correlation_coefficient_loss(outputs, batch_lbs)
                loss.backward()
                optimizer.step()
                train_loss += loss.item()
                progress_bar.set_postfix({'train_loss': f'{train_loss/(progress_bar.n+1):.4f}'})


            train_loss /= len(train_loader)

            # Combined validation loop
            model.eval()
            val_loss = 0.0
            val_pred = []
            with torch.no_grad():
                for batch_d, batch_lbs in val_loader:
                    input_values = batch_d.to(device)
                    batch_lbs = batch_lbs.to(device)
                    
                    outputs = model(input_values)
                    loss = correlation_coefficient_loss(outputs, batch_lbs)
                    val_loss += loss.item()
                    val_pred.extend(outputs.cpu().numpy())

            val_loss /= len(val_loader)

            # Calculate validation metrics
            val_pred = np.array(val_pred).reshape(val_timesteps.shape)
            val_ground_truth = _get_ground_truth_labels([all_dict[i] for i in val_index], all_labels)
            val_pred_flat = unsplit_data_ogsize(val_pred, window_size, step_sequence, 25, val_ground_truth.shape[-1])
            val_prc_coef = _calculate_flattened_accuracy(val_pred_flat, val_ground_truth)
            
            # Accumulate metrics for this fold and epoch
            train_acc.append(1- train_loss)
            val_acc.append(1- val_loss)

            # Log metrics
            writer.add_scalar(f"Loss/train_fold_{fold + 1}", train_loss, epoch)
            writer.add_scalar(f"Loss/val_fold_{fold + 1}", val_loss, epoch)
            writer.add_scalar(f"Pearson/val_fold_{fold + 1}", val_prc_coef, epoch)

            print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Pearson: {val_prc_coef:.4f}")

            # Check if validation loss improved
            if val_loss < best_val_loss:
                print(f"Validation loss improved from {best_val_loss:.4f} to {val_loss:.4f}. Saving best model...")
                best_val_loss = val_loss
                best_val_loss_flat = val_prc_coef
                early_stopping_counter = 0

                # Save the best model
                torch.save(model.state_dict(), best_model_path)
            else:
                early_stopping_counter += 1
                print(f"Validation loss did not improve for {early_stopping_counter} epochs.")
                #model.load_state_dict(torch.load(best_model_path))


            # Early stopping
            if early_stopping_counter >= early_stopping_patience:
                print(f"Early stopping triggered at epoch {epoch + 1}. Loading best model.")
                # Load the best model's weights
                model.load_state_dict(torch.load(best_model_path))
                break

        test_pred = []
        test_loss = 0.0
        with torch.no_grad():
            for batch_d, batch_lbs in test_loader:
                input_values = batch_d.to(device)
                batch_lbs = batch_lbs.to(device)
                
                outputs = model(input_values)
                loss = correlation_coefficient_loss(outputs, batch_lbs)
                test_loss += loss.item()
                test_pred.extend(outputs.cpu().numpy())

        test_loss /= len(test_loader)
        test_pred = np.array(test_pred).reshape(prepared_test_labels_timesteps.shape)
        test_ground_truth = _get_ground_truth_labels(list(test_dict.values()), test_labels)
        test_pred_flat = unsplit_data_ogsize(test_pred, window_size, step_sequence, 25, test_ground_truth.shape[-1])
        test_prc_coef = _calculate_flattened_accuracy(test_pred_flat, test_ground_truth)

        print(f"Fold {fold + 1}:")
        print(f"  Validation Pearson Coefficient  acc: {1- best_val_loss}")
        print(f"  Validation Pearson Coefficient flat acc: {best_val_loss_flat}")
        print(f"  Test acc: {1- test_loss}")
        print(f"  Test Pearson Coefficient acc(flattened): {test_prc_coef}")

        fold_metrics.append({
            'Fold': fold + 1,
            'val_prc_acc': 1- best_val_loss,
            'val_prc_acc_flat': best_val_loss_flat,
            'test_acc': 1- test_loss,
            'test_prc_flat': test_prc_coef
        })


                # Log fold-specific metrics as tables
        fold_table = f"| Fold | Val Pearson Acc | Val Pearson Flat | Test Acc | Test Pearson Flat |\n" \
                     f"|------|-----------------|------------------|----------|-------------------|\n" \
                     f"| {fold + 1} | {1 - val_loss:.4f} | {val_prc_coef:.4f} | {1 - test_loss:.4f} | {test_prc_coef:.4f} |\n"
        writer.add_text(f"Fold_{fold + 1}_Metrics", fold_table)
        # Accumulate fold metrics across all folds
        train_acc_epoch.append(train_acc)
        val_acc_epoch.append(train_acc)


    
        # After all folds, compute and log the average metrics per epoch across all folds
    for epoch in range(epochs):
        avg_train_loss = np.mean([fold_losses[epoch] for fold_losses in train_acc_epoch if len(fold_losses) > epoch])
        avg_val_loss = np.mean([fold_losses[epoch] for fold_losses in val_acc_epoch if len(fold_losses) > epoch])

        # Log the averaged metrics for the epoch across all folds
        writer.add_scalar("Average_acc/train", avg_train_loss, epoch)
        writer.add_scalar("Average_acc/val", avg_val_loss, epoch)
            

    # Calculate average metrics
    avg_metrics = {key: np.mean([fold[key] for fold in fold_metrics if key != 'Fold']) for key in fold_metrics[0].keys() if key != 'Fold'}
        # Log the final average table
    avg_table = "| Fold | Val Pearson Acc | Val Pearson Flat | Test Acc | Test Pearson Flat |\n" \
                "|------|-----------------|------------------|----------|-------------------|\n" \
                f"| Average | {avg_metrics['val_prc_acc']:.4f} | {avg_metrics['val_prc_acc_flat']:.4f} | {avg_metrics['test_acc']:.4f} | {avg_metrics['test_prc_flat']:.4f} |\n"
    writer.add_text("Average_Metrics", avg_table)
    # Add average metrics to results
    avg_metrics['Fold'] = 'Average'
    fold_metrics.append(avg_metrics)

    # save averga date to CSV
    results_df = pd.DataFrame(fold_metrics)
    csv_path = os.path.join(run_dir, 'fold_results.csv')
    results_df.to_csv(csv_path, index=False)
    
    # Save fold indices CSV
    fold_indices_df.to_csv(os.path.join(run_dir, 'fold_indices.csv'), index=False)
    
    writer.close()


    print("\nTraining completed.")
    print("Average metrics across all folds:")
    for key, value in avg_metrics.items():
        print(f"  {key}: {value}")


if __name__ == "__main__":
    ## Path to data
    #path = "/home/glenn/Downloads/"
    path = "../DATA/"


    # Model parameters
    model_config = {
        "VRBModel": {
            "model" : VRBModel,
            "model_name": "facebook/hubert-large-ls960-ft",
            "hidden_units": 64,
            "n_gru": 3,
            "output_size": None  # Will be set dynamically
        },
        "Wav2Vec2ConvLSTMModel": {
            "model" : Wav2Vec2ConvLSTMModel,
            "model_name": "facebook/wav2vec2-base",
            "hidden_units": 128,
            "n_lstm": 2,
            "output_size": None  # Will be set dynamically
        }
    }
    

    
    # Train and data parameters
    epochs = 128
    batch_size = 128
    window_size = 30
    step_size = 6
    data_parts = 4 # aka folds
    early_stopping_patience = 10
    
    config = model_config["VRBModel"]
    

    ## same wav2vec2 base model and pipeline used in the paper
    processor = AutoProcessor.from_pretrained(config["model_name"])
    
    train(
        path_to_data=path+"ComParE2020_Breathing/wav/",
        path_to_labels=path+"ComParE2020_Breathing/lab/",
        window_size=window_size,
        batch_size=batch_size,
        config = config,
        step_size=step_size,
        data_parts= data_parts ,
        early_stopping_patience= early_stopping_patience,
        epochs= epochs,
        processor = processor
    )

Fold 1/4
(864, 480000)
750


Epoch 1/128: 100%|██████████| 7/7 [00:33<00:00,  4.85s/it, train_loss=1.0058]


Epoch 1/128 - Train Loss: 1.0058, Val Loss: 1.0051, Val Pearson: -0.0082
Validation loss improved from inf to 1.0051. Saving best model...


Epoch 2/128: 100%|██████████| 7/7 [00:30<00:00,  4.31s/it, train_loss=0.9950]


Epoch 2/128 - Train Loss: 0.9950, Val Loss: 1.0087, Val Pearson: -0.0122
Validation loss did not improve for 1 epochs.


...

Epoch 3/128 - Train Loss: 0.9901, Val Loss: 1.0102, Val Pearson: -0.0132
Validation loss did not improve for 2 epochs.


Epoch 4/128: 100%|██████████| 7/7 [00:31<00:00,  4.46s/it, train_loss=0.9857]


Epoch 4/128 - Train Loss: 0.9857, Val Loss: 1.0108, Val Pearson: -0.0135
Validation loss did not improve for 3 epochs.


Epoch 5/128: 100%|██████████| 7/7 [00:28<00:00,  4.10s/it, train_loss=0.9881]


ValueError: array must not contain infs or NaNs

Restarted thesis (Python 3.11.4)

In [1]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import KFold
from datetime import datetime
#from torch.utils.tensorboard import SummaryWriter
import torchaudio
from typing import List, Tuple
from pt_utils import *
from pt_dataset import *
from pt_models import *
from pt_utils import *
from tensorboardX import SummaryWriter
from transformers import get_cosine_schedule_with_warmup


def create_run_directory():
    base_dir = "pt_runs_SOTA"
    timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
    run_dir = os.path.join(base_dir, timestamp)
    os.makedirs(run_dir, exist_ok=True)
    return run_dir

def _calculate_flattened_accuracy(average, ground_truth_labels):
    s_acc = 0
    for b in range(len(ground_truth_labels)):
        s, _ = scipy.stats.pearsonr(average[b], ground_truth_labels[b])
        s_acc += s
    return s_acc / len(ground_truth_labels)

def _choose_real_labs_only_with_filenames(labels, filenames):
    return labels[labels['filename'].isin(filenames)]

def _get_ground_truth_labels(ground_truth_names, labels):
    ground_truth_labels = []
    for batch_name in ground_truth_names:
        ground_truth_label = _choose_real_labs_only_with_filenames(labels, [batch_name])
        ground_truth_labels.append(ground_truth_label)
    return np.array(ground_truth_labels)[:, :, -1].astype(np.float32)

def train(path_to_data, path_to_labels, window_size=16, step_size=6, data_parts=4, epochs=100, batch_size=10, early_stopping_patience=20, config = None, processor = None):
    run_dir = create_run_directory()
    log_dir = os.path.join(run_dir, "logs")
    os.makedirs(log_dir, exist_ok=True)

    # Parameters
    length_sequence = window_size 
    step_sequence = step_size

    # Load and prepare data
    train_data, train_labels, train_dict, frame_rate = load_data(path_to_data, path_to_labels, 'train')
    devel_data, devel_labels, devel_dict, frame_rate = load_data(path_to_data, path_to_labels, 'devel')
    test_data, test_labels, test_dict, frame_rate = load_data(path_to_data, path_to_labels, 'test')

    # Combine train and devel data
    all_data = np.concatenate((train_data, devel_data), axis=0)
    all_labels = pd.concat([train_labels, devel_labels])
    all_dict = np.concatenate((list(train_dict.values()), list(devel_dict.values())), axis=0)

    # Prepare data
    prepared_data, prepared_labels, prepared_labels_timesteps = prepare_data(all_data, all_labels, all_dict, frame_rate, length_sequence * 16000, step_sequence * 16000)
    prepared_test_data, prepared_test_labels, prepared_test_labels_timesteps = prepare_data(test_data, test_labels, test_dict, frame_rate, length_sequence * 16000, step_sequence * 16000)

    # Create CSV file for storing fold indices
    fold_indices_df = pd.DataFrame(columns=['Fold', 'Train_Indices', 'Val_Indices'])

    # Cross-validation
    kf = KFold(n_splits=data_parts)
    fold_metrics = []
    # To accumulate metrics across folds for each epoch
    train_acc_epoch = []
    val_acc_epoch = []
    test_acc_epoch = []

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    
    config["output_size"] = prepared_labels.shape[-1]
    writer = SummaryWriter(log_dir=os.path.join(log_dir,config["model_name"]))

    for fold, (train_index, val_index) in enumerate(kf.split(prepared_data)):
        print(f"Fold {fold + 1}/{data_parts}")
        best_model_path = f"{run_dir}/best_model_fold{fold+1}"
        # Save fold indices
        fold_indices_df = fold_indices_df._append({
            'Fold': fold + 1,
            'Train_Indices': train_index.tolist(),
            'Val_Indices': val_index.tolist()
        }, ignore_index=True)

        # Split data
        train_d, val_d = prepared_data[train_index], prepared_data[val_index]
        train_lbs, val_lbs = prepared_labels[train_index], prepared_labels[val_index]
        train_timesteps, val_timesteps = prepared_labels_timesteps[train_index], prepared_labels_timesteps[val_index]
        
        # Reshape data
        train_d, train_lbs = reshaping_data_for_model(train_d, train_lbs)
        val_d, val_lbs = reshaping_data_for_model(val_d, val_lbs)
        test_d, test_lbs = reshaping_data_for_model(prepared_test_data, prepared_test_labels)
        
        print(train_d.shape)

        # Create datasets
        #train_dataset = BreathingDataset(train_d, train_lbs, processor, window_size, step_sequence, augment=True)
        train_dataset = BreathingDataset(train_d, train_lbs, processor,window_size, step_sequence)
        val_dataset = BreathingDataset(val_d, val_lbs, processor, window_size, step_sequence)
        test_dataset = BreathingDataset(test_d, test_lbs, processor, window_size, step_sequence)

        # Create DataLoaders
        train_loader = DataLoader(train_dataset, batch_size=batch_size,num_workers=2, shuffle=True, collate_fn=train_dataset.collate_fn)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, num_workers=1, collate_fn=val_dataset.collate_fn)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, num_workers=1, collate_fn=test_dataset.collate_fn)
        print(config["output_size"])
        # Create and initialize model
        model = config["model"](config).to(device)
        
        #### training optimiser parameters fror apple
        #learning_rate = 0.005 
        #optimizer = optim.Adam(model.parameters(), lr=learning_rate)

        #### training optimiser parameters fror harma_2023 VRB model 
        learning_rate = 0.01 # From the paper
        optimizer = optim.SGD(model.parameters(), lr=learning_rate)       
        
        best_val_loss = float('inf')
        best_val_loss_flat = float('inf')
        early_stopping_counter = 0
        # To accumulate metrics across folds for each epoch
        train_acc = []
        val_acc = []
        test_acc = []
        for epoch in range(epochs):

            model.train()
            train_loss = 0.0
            progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")
            
            for batch_d, batch_lbs in progress_bar:
                optimizer.zero_grad()
                input_values = batch_d.to(device)
                batch_lbs = batch_lbs.to(device)
                outputs = model(input_values)
                loss = correlation_coefficient_loss(outputs, batch_lbs)
                loss.backward()
                optimizer.step()
                train_loss += loss.item()
                progress_bar.set_postfix({'train_loss': f'{train_loss/(progress_bar.n+1):.4f}'})


            train_loss /= len(train_loader)

            # Combined validation loop
            model.eval()
            val_loss = 0.0
            val_pred = []
            with torch.no_grad():
                for batch_d, batch_lbs in val_loader:
                    input_values = batch_d.to(device)
                    batch_lbs = batch_lbs.to(device)
                    
                    outputs = model(input_values)
                    loss = correlation_coefficient_loss(outputs, batch_lbs)
                    val_loss += loss.item()
                    val_pred.extend(outputs.cpu().numpy())

            val_loss /= len(val_loader)

            # Calculate validation metrics
            val_pred = np.array(val_pred).reshape(val_timesteps.shape)
            val_ground_truth = _get_ground_truth_labels([all_dict[i] for i in val_index], all_labels)
            val_pred_flat = unsplit_data_ogsize(val_pred, window_size, step_sequence, 25, val_ground_truth.shape[-1])
            val_prc_coef = _calculate_flattened_accuracy(val_pred_flat, val_ground_truth)
            
            # Accumulate metrics for this fold and epoch
            train_acc.append(1- train_loss)
            val_acc.append(1- val_loss)

            # Log metrics
            writer.add_scalar(f"Loss/train_fold_{fold + 1}", train_loss, epoch)
            writer.add_scalar(f"Loss/val_fold_{fold + 1}", val_loss, epoch)
            writer.add_scalar(f"Pearson/val_fold_{fold + 1}", val_prc_coef, epoch)

            print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Pearson: {val_prc_coef:.4f}")

            # Check if validation loss improved
            if val_loss < best_val_loss:
                print(f"Validation loss improved from {best_val_loss:.4f} to {val_loss:.4f}. Saving best model...")
                best_val_loss = val_loss
                best_val_loss_flat = val_prc_coef
                early_stopping_counter = 0

                # Save the best model
                torch.save(model.state_dict(), best_model_path)
            else:
                early_stopping_counter += 1
                print(f"Validation loss did not improve for {early_stopping_counter} epochs.")
                #model.load_state_dict(torch.load(best_model_path))


            # Early stopping
            if early_stopping_counter >= early_stopping_patience:
                print(f"Early stopping triggered at epoch {epoch + 1}. Loading best model.")
                # Load the best model's weights
                model.load_state_dict(torch.load(best_model_path))
                break

        test_pred = []
        test_loss = 0.0
        with torch.no_grad():
            for batch_d, batch_lbs in test_loader:
                input_values = batch_d.to(device)
                batch_lbs = batch_lbs.to(device)
                
                outputs = model(input_values)
                loss = correlation_coefficient_loss(outputs, batch_lbs)
                test_loss += loss.item()
                test_pred.extend(outputs.cpu().numpy())

        test_loss /= len(test_loader)
        test_pred = np.array(test_pred).reshape(prepared_test_labels_timesteps.shape)
        test_ground_truth = _get_ground_truth_labels(list(test_dict.values()), test_labels)
        test_pred_flat = unsplit_data_ogsize(test_pred, window_size, step_sequence, 25, test_ground_truth.shape[-1])
        test_prc_coef = _calculate_flattened_accuracy(test_pred_flat, test_ground_truth)

        print(f"Fold {fold + 1}:")
        print(f"  Validation Pearson Coefficient  acc: {1- best_val_loss}")
        print(f"  Validation Pearson Coefficient flat acc: {best_val_loss_flat}")
        print(f"  Test acc: {1- test_loss}")
        print(f"  Test Pearson Coefficient acc(flattened): {test_prc_coef}")

        fold_metrics.append({
            'Fold': fold + 1,
            'val_prc_acc': 1- best_val_loss,
            'val_prc_acc_flat': best_val_loss_flat,
            'test_acc': 1- test_loss,
            'test_prc_flat': test_prc_coef
        })


                # Log fold-specific metrics as tables
        fold_table = f"| Fold | Val Pearson Acc | Val Pearson Flat | Test Acc | Test Pearson Flat |\n" \
                     f"|------|-----------------|------------------|----------|-------------------|\n" \
                     f"| {fold + 1} | {1 - val_loss:.4f} | {val_prc_coef:.4f} | {1 - test_loss:.4f} | {test_prc_coef:.4f} |\n"
        writer.add_text(f"Fold_{fold + 1}_Metrics", fold_table)
        # Accumulate fold metrics across all folds
        train_acc_epoch.append(train_acc)
        val_acc_epoch.append(train_acc)


    
        # After all folds, compute and log the average metrics per epoch across all folds
    for epoch in range(epochs):
        avg_train_loss = np.mean([fold_losses[epoch] for fold_losses in train_acc_epoch if len(fold_losses) > epoch])
        avg_val_loss = np.mean([fold_losses[epoch] for fold_losses in val_acc_epoch if len(fold_losses) > epoch])

        # Log the averaged metrics for the epoch across all folds
        writer.add_scalar("Average_acc/train", avg_train_loss, epoch)
        writer.add_scalar("Average_acc/val", avg_val_loss, epoch)
            

    # Calculate average metrics
    avg_metrics = {key: np.mean([fold[key] for fold in fold_metrics if key != 'Fold']) for key in fold_metrics[0].keys() if key != 'Fold'}
        # Log the final average table
    avg_table = "| Fold | Val Pearson Acc | Val Pearson Flat | Test Acc | Test Pearson Flat |\n" \
                "|------|-----------------|------------------|----------|-------------------|\n" \
                f"| Average | {avg_metrics['val_prc_acc']:.4f} | {avg_metrics['val_prc_acc_flat']:.4f} | {avg_metrics['test_acc']:.4f} | {avg_metrics['test_prc_flat']:.4f} |\n"
    writer.add_text("Average_Metrics", avg_table)
    # Add average metrics to results
    avg_metrics['Fold'] = 'Average'
    fold_metrics.append(avg_metrics)

    # save averga date to CSV
    results_df = pd.DataFrame(fold_metrics)
    csv_path = os.path.join(run_dir, 'fold_results.csv')
    results_df.to_csv(csv_path, index=False)
    
    # Save fold indices CSV
    fold_indices_df.to_csv(os.path.join(run_dir, 'fold_indices.csv'), index=False)
    
    writer.close()


    print("\nTraining completed.")
    print("Average metrics across all folds:")
    for key, value in avg_metrics.items():
        print(f"  {key}: {value}")


if __name__ == "__main__":
    ## Path to data
    #path = "/home/glenn/Downloads/"
    path = "../DATA/"


    # Model parameters
    model_config = {
        "VRBModel": {
            "model" : VRBModel,
            "model_name": "facebook/hubert-large-ls960-ft",
            "hidden_units": 64,
            "n_gru": 3,
            "output_size": None  # Will be set dynamically
        },
        "Wav2Vec2ConvLSTMModel": {
            "model" : Wav2Vec2ConvLSTMModel,
            "model_name": "facebook/wav2vec2-base",
            "hidden_units": 128,
            "n_lstm": 2,
            "output_size": None  # Will be set dynamically
        }
    }
    

    
    # Train and data parameters
    epochs = 128
    batch_size = 64
    window_size = 30
    step_size = 6
    data_parts = 4 # aka folds
    early_stopping_patience = 10
    
    config = model_config["VRBModel"]
    

    ## same wav2vec2 base model and pipeline used in the paper
    processor = AutoProcessor.from_pretrained(config["model_name"])
    
    train(
        path_to_data=path+"ComParE2020_Breathing/wav/",
        path_to_labels=path+"ComParE2020_Breathing/lab/",
        window_size=window_size,
        batch_size=batch_size,
        config = config,
        step_size=step_size,
        data_parts= data_parts ,
        early_stopping_patience= early_stopping_patience,
        epochs= epochs,
        processor = processor
    )

Fold 1/4
(864, 480000)
750


Epoch 1/128: 100%|██████████| 14/14 [00:30<00:00,  2.16s/it, train_loss=1.0073]


Epoch 1/128 - Train Loss: 1.0073, Val Loss: 0.9901, Val Pearson: -0.0009
Validation loss improved from inf to 0.9901. Saving best model...


Epoch 2/128: 100%|██████████| 14/14 [00:29<00:00,  2.13s/it, train_loss=0.9927]


Restarted thesis (Python 3.11.4)

In [1]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import KFold
from datetime import datetime
#from torch.utils.tensorboard import SummaryWriter
import torchaudio
from typing import List, Tuple
from pt_utils import *
from pt_dataset import *
from pt_models import *
from pt_utils import *
from tensorboardX import SummaryWriter
from transformers import get_cosine_schedule_with_warmup


def create_run_directory():
    base_dir = "pt_runs_SOTA"
    timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
    run_dir = os.path.join(base_dir, timestamp)
    os.makedirs(run_dir, exist_ok=True)
    return run_dir

def _calculate_flattened_accuracy(average, ground_truth_labels):
    s_acc = 0
    for b in range(len(ground_truth_labels)):
        s, _ = scipy.stats.pearsonr(average[b], ground_truth_labels[b])
        s_acc += s
    return s_acc / len(ground_truth_labels)

def _choose_real_labs_only_with_filenames(labels, filenames):
    return labels[labels['filename'].isin(filenames)]

def _get_ground_truth_labels(ground_truth_names, labels):
    ground_truth_labels = []
    for batch_name in ground_truth_names:
        ground_truth_label = _choose_real_labs_only_with_filenames(labels, [batch_name])
        ground_truth_labels.append(ground_truth_label)
    return np.array(ground_truth_labels)[:, :, -1].astype(np.float32)

def train(path_to_data, path_to_labels, window_size=16, step_size=6, data_parts=4, epochs=100, batch_size=10, early_stopping_patience=20, config = None, processor = None):
    run_dir = create_run_directory()
    log_dir = os.path.join(run_dir, "logs")
    os.makedirs(log_dir, exist_ok=True)

    # Parameters
    length_sequence = window_size 
    step_sequence = step_size

    # Load and prepare data
    train_data, train_labels, train_dict, frame_rate = load_data(path_to_data, path_to_labels, 'train')
    devel_data, devel_labels, devel_dict, frame_rate = load_data(path_to_data, path_to_labels, 'devel')
    test_data, test_labels, test_dict, frame_rate = load_data(path_to_data, path_to_labels, 'test')

    # Combine train and devel data
    all_data = np.concatenate((train_data, devel_data), axis=0)
    all_labels = pd.concat([train_labels, devel_labels])
    all_dict = np.concatenate((list(train_dict.values()), list(devel_dict.values())), axis=0)

    # Prepare data
    prepared_data, prepared_labels, prepared_labels_timesteps = prepare_data(all_data, all_labels, all_dict, frame_rate, length_sequence * 16000, step_sequence * 16000)
    prepared_test_data, prepared_test_labels, prepared_test_labels_timesteps = prepare_data(test_data, test_labels, test_dict, frame_rate, length_sequence * 16000, step_sequence * 16000)

    # Create CSV file for storing fold indices
    fold_indices_df = pd.DataFrame(columns=['Fold', 'Train_Indices', 'Val_Indices'])

    # Cross-validation
    kf = KFold(n_splits=data_parts)
    fold_metrics = []
    # To accumulate metrics across folds for each epoch
    train_acc_epoch = []
    val_acc_epoch = []
    test_acc_epoch = []

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    
    config["output_size"] = prepared_labels.shape[-1]
    writer = SummaryWriter(log_dir=os.path.join(log_dir,config["model_name"]))

    for fold, (train_index, val_index) in enumerate(kf.split(prepared_data)):
        print(f"Fold {fold + 1}/{data_parts}")
        best_model_path = f"{run_dir}/best_model_fold{fold+1}"
        # Save fold indices
        fold_indices_df = fold_indices_df._append({
            'Fold': fold + 1,
            'Train_Indices': train_index.tolist(),
            'Val_Indices': val_index.tolist()
        }, ignore_index=True)

        # Split data
        train_d, val_d = prepared_data[train_index], prepared_data[val_index]
        train_lbs, val_lbs = prepared_labels[train_index], prepared_labels[val_index]
        train_timesteps, val_timesteps = prepared_labels_timesteps[train_index], prepared_labels_timesteps[val_index]
        
        # Reshape data
        train_d, train_lbs = reshaping_data_for_model(train_d, train_lbs)
        val_d, val_lbs = reshaping_data_for_model(val_d, val_lbs)
        test_d, test_lbs = reshaping_data_for_model(prepared_test_data, prepared_test_labels)
        
        print(train_d.shape)

        # Create datasets
        #train_dataset = BreathingDataset(train_d, train_lbs, processor, window_size, step_sequence, augment=True)
        train_dataset = BreathingDataset(train_d, train_lbs, processor,window_size, step_sequence)
        val_dataset = BreathingDataset(val_d, val_lbs, processor, window_size, step_sequence)
        test_dataset = BreathingDataset(test_d, test_lbs, processor, window_size, step_sequence)

        # Create DataLoaders
        train_loader = DataLoader(train_dataset, batch_size=batch_size,num_workers=2, shuffle=True, collate_fn=train_dataset.collate_fn)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, num_workers=1, collate_fn=val_dataset.collate_fn)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, num_workers=1, collate_fn=test_dataset.collate_fn)
        print(config["output_size"])
        # Create and initialize model
        model = config["model"](config).to(device)
        
        #### training optimiser parameters fror apple
        #learning_rate = 0.005 
        #optimizer = optim.Adam(model.parameters(), lr=learning_rate)

        #### training optimiser parameters fror harma_2023 VRB model 
        learning_rate = 0.01 # From the paper
        optimizer = optim.Adam(model.parameters())       
        
        best_val_loss = float('inf')
        best_val_loss_flat = float('inf')
        early_stopping_counter = 0
        # To accumulate metrics across folds for each epoch
        train_acc = []
        val_acc = []
        test_acc = []
        for epoch in range(epochs):

            model.train()
            train_loss = 0.0
            progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")
            
            for batch_d, batch_lbs in progress_bar:
                optimizer.zero_grad()
                input_values = batch_d.to(device)
                batch_lbs = batch_lbs.to(device)
                outputs = model(input_values)
                loss = correlation_coefficient_loss(outputs, batch_lbs)
                loss.backward()
                optimizer.step()
                train_loss += loss.item()
                progress_bar.set_postfix({'train_loss': f'{train_loss/(progress_bar.n+1):.4f}'})


            train_loss /= len(train_loader)

            # Combined validation loop
            model.eval()
            val_loss = 0.0
            val_pred = []
            with torch.no_grad():
                for batch_d, batch_lbs in val_loader:
                    input_values = batch_d.to(device)
                    batch_lbs = batch_lbs.to(device)
                    
                    outputs = model(input_values)
                    loss = correlation_coefficient_loss(outputs, batch_lbs)
                    val_loss += loss.item()
                    val_pred.extend(outputs.cpu().numpy())

            val_loss /= len(val_loader)

            # Calculate validation metrics
            val_pred = np.array(val_pred).reshape(val_timesteps.shape)
            val_ground_truth = _get_ground_truth_labels([all_dict[i] for i in val_index], all_labels)
            val_pred_flat = unsplit_data_ogsize(val_pred, window_size, step_sequence, 25, val_ground_truth.shape[-1])
            val_prc_coef = _calculate_flattened_accuracy(val_pred_flat, val_ground_truth)
            
            # Accumulate metrics for this fold and epoch
            train_acc.append(1- train_loss)
            val_acc.append(1- val_loss)

            # Log metrics
            writer.add_scalar(f"Loss/train_fold_{fold + 1}", train_loss, epoch)
            writer.add_scalar(f"Loss/val_fold_{fold + 1}", val_loss, epoch)
            writer.add_scalar(f"Pearson/val_fold_{fold + 1}", val_prc_coef, epoch)

            print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Pearson: {val_prc_coef:.4f}")

            # Check if validation loss improved
            if val_loss < best_val_loss:
                print(f"Validation loss improved from {best_val_loss:.4f} to {val_loss:.4f}. Saving best model...")
                best_val_loss = val_loss
                best_val_loss_flat = val_prc_coef
                early_stopping_counter = 0

                # Save the best model
                torch.save(model.state_dict(), best_model_path)
            else:
                early_stopping_counter += 1
                print(f"Validation loss did not improve for {early_stopping_counter} epochs.")
                #model.load_state_dict(torch.load(best_model_path))


            # Early stopping
            if early_stopping_counter >= early_stopping_patience:
                print(f"Early stopping triggered at epoch {epoch + 1}. Loading best model.")
                # Load the best model's weights
                model.load_state_dict(torch.load(best_model_path))
                break

        test_pred = []
        test_loss = 0.0
        with torch.no_grad():
            for batch_d, batch_lbs in test_loader:
                input_values = batch_d.to(device)
                batch_lbs = batch_lbs.to(device)
                
                outputs = model(input_values)
                loss = correlation_coefficient_loss(outputs, batch_lbs)
                test_loss += loss.item()
                test_pred.extend(outputs.cpu().numpy())

        test_loss /= len(test_loader)
        test_pred = np.array(test_pred).reshape(prepared_test_labels_timesteps.shape)
        test_ground_truth = _get_ground_truth_labels(list(test_dict.values()), test_labels)
        test_pred_flat = unsplit_data_ogsize(test_pred, window_size, step_sequence, 25, test_ground_truth.shape[-1])
        test_prc_coef = _calculate_flattened_accuracy(test_pred_flat, test_ground_truth)

        print(f"Fold {fold + 1}:")
        print(f"  Validation Pearson Coefficient  acc: {1- best_val_loss}")
        print(f"  Validation Pearson Coefficient flat acc: {best_val_loss_flat}")
        print(f"  Test acc: {1- test_loss}")
        print(f"  Test Pearson Coefficient acc(flattened): {test_prc_coef}")

        fold_metrics.append({
            'Fold': fold + 1,
            'val_prc_acc': 1- best_val_loss,
            'val_prc_acc_flat': best_val_loss_flat,
            'test_acc': 1- test_loss,
            'test_prc_flat': test_prc_coef
        })


                # Log fold-specific metrics as tables
        fold_table = f"| Fold | Val Pearson Acc | Val Pearson Flat | Test Acc | Test Pearson Flat |\n" \
                     f"|------|-----------------|------------------|----------|-------------------|\n" \
                     f"| {fold + 1} | {1 - val_loss:.4f} | {val_prc_coef:.4f} | {1 - test_loss:.4f} | {test_prc_coef:.4f} |\n"
        writer.add_text(f"Fold_{fold + 1}_Metrics", fold_table)
        # Accumulate fold metrics across all folds
        train_acc_epoch.append(train_acc)
        val_acc_epoch.append(train_acc)


    
        # After all folds, compute and log the average metrics per epoch across all folds
    for epoch in range(epochs):
        avg_train_loss = np.mean([fold_losses[epoch] for fold_losses in train_acc_epoch if len(fold_losses) > epoch])
        avg_val_loss = np.mean([fold_losses[epoch] for fold_losses in val_acc_epoch if len(fold_losses) > epoch])

        # Log the averaged metrics for the epoch across all folds
        writer.add_scalar("Average_acc/train", avg_train_loss, epoch)
        writer.add_scalar("Average_acc/val", avg_val_loss, epoch)
            

    # Calculate average metrics
    avg_metrics = {key: np.mean([fold[key] for fold in fold_metrics if key != 'Fold']) for key in fold_metrics[0].keys() if key != 'Fold'}
        # Log the final average table
    avg_table = "| Fold | Val Pearson Acc | Val Pearson Flat | Test Acc | Test Pearson Flat |\n" \
                "|------|-----------------|------------------|----------|-------------------|\n" \
                f"| Average | {avg_metrics['val_prc_acc']:.4f} | {avg_metrics['val_prc_acc_flat']:.4f} | {avg_metrics['test_acc']:.4f} | {avg_metrics['test_prc_flat']:.4f} |\n"
    writer.add_text("Average_Metrics", avg_table)
    # Add average metrics to results
    avg_metrics['Fold'] = 'Average'
    fold_metrics.append(avg_metrics)

    # save averga date to CSV
    results_df = pd.DataFrame(fold_metrics)
    csv_path = os.path.join(run_dir, 'fold_results.csv')
    results_df.to_csv(csv_path, index=False)
    
    # Save fold indices CSV
    fold_indices_df.to_csv(os.path.join(run_dir, 'fold_indices.csv'), index=False)
    
    writer.close()


    print("\nTraining completed.")
    print("Average metrics across all folds:")
    for key, value in avg_metrics.items():
        print(f"  {key}: {value}")


if __name__ == "__main__":
    ## Path to data
    #path = "/home/glenn/Downloads/"
    path = "../DATA/"


    # Model parameters
    model_config = {
        "VRBModel": {
            "model" : VRBModel,
            "model_name": "facebook/hubert-large-ls960-ft",
            "hidden_units": 64,
            "n_gru": 3,
            "output_size": None  # Will be set dynamically
        },
        "Wav2Vec2ConvLSTMModel": {
            "model" : Wav2Vec2ConvLSTMModel,
            "model_name": "facebook/wav2vec2-base",
            "hidden_units": 128,
            "n_lstm": 2,
            "output_size": None  # Will be set dynamically
        }
    }
    

    
    # Train and data parameters
    epochs = 128
    batch_size = 64
    window_size = 30
    step_size = 6
    data_parts = 4 # aka folds
    early_stopping_patience = 10
    
    config = model_config["VRBModel"]
    

    ## same wav2vec2 base model and pipeline used in the paper
    processor = AutoProcessor.from_pretrained(config["model_name"])
    
    train(
        path_to_data=path+"ComParE2020_Breathing/wav/",
        path_to_labels=path+"ComParE2020_Breathing/lab/",
        window_size=window_size,
        batch_size=batch_size,
        config = config,
        step_size=step_size,
        data_parts= data_parts ,
        early_stopping_patience= early_stopping_patience,
        epochs= epochs,
        processor = processor
    )

Fold 1/4
(864, 480000)
750


Epoch 1/128: 100%|██████████| 14/14 [00:29<00:00,  2.09s/it, train_loss=1.0013]


Epoch 1/128 - Train Loss: 1.0013, Val Loss: 0.9151, Val Pearson: 0.1194
Validation loss improved from inf to 0.9151. Saving best model...


Epoch 2/128: 100%|██████████| 14/14 [00:29<00:00,  2.13s/it, train_loss=0.7097]


Epoch 2/128 - Train Loss: 0.7097, Val Loss: 0.5928, Val Pearson: 0.4455
Validation loss improved from 0.9151 to 0.5928. Saving best model...


...

Epoch 26/128 - Train Loss: 0.0946, Val Loss: 0.4155, Val Pearson: 0.6314
Validation loss did not improve for 9 epochs.


Epoch 27/128: 100%|██████████| 15/15 [00:35<00:00,  2.38s/it, train_loss=0.0880]


Epoch 27/128 - Train Loss: 0.0880, Val Loss: 0.4218, Val Pearson: 0.6279
Validation loss did not improve for 10 epochs.
Early stopping triggered at epoch 27. Loading best model.


<ipython-input-1-38649e80a240>:208: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(best_model_path))
/home/gdwildt/.conda/envs/thesis/lib/pyt

Fold 4:
  Validation Pearson Coefficient  acc: 0.6237318634986877
  Validation Pearson Coefficient flat acc: 0.6625138473549642
  Test acc: 0.7788847287495931
  Test Pearson Coefficient acc(flattened): 0.7945983461230314

Training completed.
Average metrics across all folds:
  val_prc_acc: 0.6851448938250542
  val_prc_acc_flat: 0.7015284605559333
  test_acc: 0.7773389882511563
  test_prc_flat: 0.7908176527486469
  Fold: Average


In [ ]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import KFold
from datetime import datetime
#from torch.utils.tensorboard import SummaryWriter
import torchaudio
from typing import List, Tuple
from pt_utils import *
from pt_dataset import *
from pt_models import *
from pt_utils import *
from tensorboardX import SummaryWriter
from transformers import get_cosine_schedule_with_warmup


def create_run_directory():
    base_dir = "pt_runs_SOTA"
    timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
    run_dir = os.path.join(base_dir, timestamp)
    os.makedirs(run_dir, exist_ok=True)
    return run_dir

def _calculate_flattened_accuracy(average, ground_truth_labels):
    s_acc = 0
    for b in range(len(ground_truth_labels)):
        s, _ = scipy.stats.pearsonr(average[b], ground_truth_labels[b])
        s_acc += s
    return s_acc / len(ground_truth_labels)

def _choose_real_labs_only_with_filenames(labels, filenames):
    return labels[labels['filename'].isin(filenames)]

def _get_ground_truth_labels(ground_truth_names, labels):
    ground_truth_labels = []
    for batch_name in ground_truth_names:
        ground_truth_label = _choose_real_labs_only_with_filenames(labels, [batch_name])
        ground_truth_labels.append(ground_truth_label)
    return np.array(ground_truth_labels)[:, :, -1].astype(np.float32)

def train(path_to_data, path_to_labels, window_size=16, step_size=6, data_parts=4, epochs=100, batch_size=10, early_stopping_patience=20, config = None, processor = None):
    run_dir = create_run_directory()
    log_dir = os.path.join(run_dir, "logs")
    os.makedirs(log_dir, exist_ok=True)

    # Parameters
    length_sequence = window_size 
    step_sequence = step_size

    # Load and prepare data
    train_data, train_labels, train_dict, frame_rate = load_data(path_to_data, path_to_labels, 'train')
    devel_data, devel_labels, devel_dict, frame_rate = load_data(path_to_data, path_to_labels, 'devel')
    test_data, test_labels, test_dict, frame_rate = load_data(path_to_data, path_to_labels, 'test')

    # Combine train and devel data
    all_data = np.concatenate((train_data, devel_data), axis=0)
    all_labels = pd.concat([train_labels, devel_labels])
    all_dict = np.concatenate((list(train_dict.values()), list(devel_dict.values())), axis=0)

    # Prepare data
    prepared_data, prepared_labels, prepared_labels_timesteps = prepare_data(all_data, all_labels, all_dict, frame_rate, length_sequence * 16000, step_sequence * 16000)
    prepared_test_data, prepared_test_labels, prepared_test_labels_timesteps = prepare_data(test_data, test_labels, test_dict, frame_rate, length_sequence * 16000, step_sequence * 16000)

    # Create CSV file for storing fold indices
    fold_indices_df = pd.DataFrame(columns=['Fold', 'Train_Indices', 'Val_Indices'])

    # Cross-validation
    kf = KFold(n_splits=data_parts)
    fold_metrics = []
    # To accumulate metrics across folds for each epoch
    train_acc_epoch = []
    val_acc_epoch = []
    test_acc_epoch = []

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    
    config["output_size"] = prepared_labels.shape[-1]
    writer = SummaryWriter(log_dir=os.path.join(log_dir,config["model_name"]))

    for fold, (train_index, val_index) in enumerate(kf.split(prepared_data)):
        print(f"Fold {fold + 1}/{data_parts}")
        best_model_path = f"{run_dir}/best_model_fold{fold+1}"
        # Save fold indices
        fold_indices_df = fold_indices_df._append({
            'Fold': fold + 1,
            'Train_Indices': train_index.tolist(),
            'Val_Indices': val_index.tolist()
        }, ignore_index=True)

        # Split data
        train_d, val_d = prepared_data[train_index], prepared_data[val_index]
        train_lbs, val_lbs = prepared_labels[train_index], prepared_labels[val_index]
        train_timesteps, val_timesteps = prepared_labels_timesteps[train_index], prepared_labels_timesteps[val_index]
        
        # Reshape data
        train_d, train_lbs = reshaping_data_for_model(train_d, train_lbs)
        val_d, val_lbs = reshaping_data_for_model(val_d, val_lbs)
        test_d, test_lbs = reshaping_data_for_model(prepared_test_data, prepared_test_labels)
        
        print(train_d.shape)

        # Create datasets
        #train_dataset = BreathingDataset(train_d, train_lbs, processor, window_size, step_sequence, augment=True)
        train_dataset = BreathingDataset(train_d, train_lbs, processor,window_size, step_sequence)
        val_dataset = BreathingDataset(val_d, val_lbs, processor, window_size, step_sequence)
        test_dataset = BreathingDataset(test_d, test_lbs, processor, window_size, step_sequence)

        # Create DataLoaders
        train_loader = DataLoader(train_dataset, batch_size=batch_size,num_workers=2, shuffle=True, collate_fn=train_dataset.collate_fn)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, num_workers=1, collate_fn=val_dataset.collate_fn)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, num_workers=1, collate_fn=test_dataset.collate_fn)
        print(config["output_size"])
        # Create and initialize model
        model = config["model"](config).to(device)
        
        #### training optimiser parameters fror apple
        #learning_rate = 0.005 
        #optimizer = optim.Adam(model.parameters(), lr=learning_rate)

        #### training optimiser parameters fror harma_2023 VRB model 
        learning_rate = 0.01 # From the paper
        optimizer = optim.Adam(model.parameters())       
        
        best_val_loss = float('inf')
        best_val_loss_flat = float('inf')
        early_stopping_counter = 0
        # To accumulate metrics across folds for each epoch
        train_acc = []
        val_acc = []
        test_acc = []
        for epoch in range(epochs):

            model.train()
            train_loss = 0.0
            progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")
            
            for batch_d, batch_lbs in progress_bar:
                optimizer.zero_grad()
                input_values = batch_d.to(device)
                batch_lbs = batch_lbs.to(device)
                outputs = model(input_values)
                loss = correlation_coefficient_loss(outputs, batch_lbs)
                loss.backward()
                optimizer.step()
                train_loss += loss.item()
                progress_bar.set_postfix({'train_loss': f'{train_loss/(progress_bar.n+1):.4f}'})


            train_loss /= len(train_loader)

            # Combined validation loop
            model.eval()
            val_loss = 0.0
            val_pred = []
            with torch.no_grad():
                for batch_d, batch_lbs in val_loader:
                    input_values = batch_d.to(device)
                    batch_lbs = batch_lbs.to(device)
                    
                    outputs = model(input_values)
                    loss = correlation_coefficient_loss(outputs, batch_lbs)
                    val_loss += loss.item()
                    val_pred.extend(outputs.cpu().numpy())

            val_loss /= len(val_loader)

            # Calculate validation metrics
            val_pred = np.array(val_pred).reshape(val_timesteps.shape)
            val_ground_truth = _get_ground_truth_labels([all_dict[i] for i in val_index], all_labels)
            val_pred_flat = unsplit_data_ogsize(val_pred, window_size, step_sequence, 25, val_ground_truth.shape[-1])
            val_prc_coef = _calculate_flattened_accuracy(val_pred_flat, val_ground_truth)
            
            # Accumulate metrics for this fold and epoch
            train_acc.append(1- train_loss)
            val_acc.append(1- val_loss)

            # Log metrics
            writer.add_scalar(f"Loss/train_fold_{fold + 1}", train_loss, epoch)
            writer.add_scalar(f"Loss/val_fold_{fold + 1}", val_loss, epoch)
            writer.add_scalar(f"Pearson/val_fold_{fold + 1}", val_prc_coef, epoch)

            print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Pearson: {val_prc_coef:.4f}")

            # Check if validation loss improved
            if val_loss < best_val_loss:
                print(f"Validation loss improved from {best_val_loss:.4f} to {val_loss:.4f}. Saving best model...")
                best_val_loss = val_loss
                best_val_loss_flat = val_prc_coef
                early_stopping_counter = 0

                # Save the best model
                torch.save(model.state_dict(), best_model_path)
            else:
                early_stopping_counter += 1
                print(f"Validation loss did not improve for {early_stopping_counter} epochs.")
                #model.load_state_dict(torch.load(best_model_path))


            # Early stopping
            if early_stopping_counter >= early_stopping_patience:
                print(f"Early stopping triggered at epoch {epoch + 1}. Loading best model.")
                # Load the best model's weights
                model.load_state_dict(torch.load(best_model_path))
                break

        test_pred = []
        test_loss = 0.0
        with torch.no_grad():
            for batch_d, batch_lbs in test_loader:
                input_values = batch_d.to(device)
                batch_lbs = batch_lbs.to(device)
                
                outputs = model(input_values)
                loss = correlation_coefficient_loss(outputs, batch_lbs)
                test_loss += loss.item()
                test_pred.extend(outputs.cpu().numpy())

        test_loss /= len(test_loader)
        test_pred = np.array(test_pred).reshape(prepared_test_labels_timesteps.shape)
        test_ground_truth = _get_ground_truth_labels(list(test_dict.values()), test_labels)
        test_pred_flat = unsplit_data_ogsize(test_pred, window_size, step_sequence, 25, test_ground_truth.shape[-1])
        test_prc_coef = _calculate_flattened_accuracy(test_pred_flat, test_ground_truth)

        print(f"Fold {fold + 1}:")
        print(f"  Validation Pearson Coefficient  acc: {1- best_val_loss}")
        print(f"  Validation Pearson Coefficient flat acc: {best_val_loss_flat}")
        print(f"  Test acc: {1- test_loss}")
        print(f"  Test Pearson Coefficient acc(flattened): {test_prc_coef}")

        fold_metrics.append({
            'Fold': fold + 1,
            'val_prc_acc': 1- best_val_loss,
            'val_prc_acc_flat': best_val_loss_flat,
            'test_acc': 1- test_loss,
            'test_prc_flat': test_prc_coef
        })


                # Log fold-specific metrics as tables
        fold_table = f"| Fold | Val Pearson Acc | Val Pearson Flat | Test Acc | Test Pearson Flat |\n" \
                     f"|------|-----------------|------------------|----------|-------------------|\n" \
                     f"| {fold + 1} | {1 - val_loss:.4f} | {val_prc_coef:.4f} | {1 - test_loss:.4f} | {test_prc_coef:.4f} |\n"
        writer.add_text(f"Fold_{fold + 1}_Metrics", fold_table)
        # Accumulate fold metrics across all folds
        train_acc_epoch.append(train_acc)
        val_acc_epoch.append(train_acc)


    
        # After all folds, compute and log the average metrics per epoch across all folds
    for epoch in range(epochs):
        avg_train_loss = np.mean([fold_losses[epoch] for fold_losses in train_acc_epoch if len(fold_losses) > epoch])
        avg_val_loss = np.mean([fold_losses[epoch] for fold_losses in val_acc_epoch if len(fold_losses) > epoch])

        # Log the averaged metrics for the epoch across all folds
        writer.add_scalar("Average_acc/train", avg_train_loss, epoch)
        writer.add_scalar("Average_acc/val", avg_val_loss, epoch)
            

    # Calculate average metrics
    avg_metrics = {key: np.mean([fold[key] for fold in fold_metrics if key != 'Fold']) for key in fold_metrics[0].keys() if key != 'Fold'}
        # Log the final average table
    avg_table = "| Fold | Val Pearson Acc | Val Pearson Flat | Test Acc | Test Pearson Flat |\n" \
                "|------|-----------------|------------------|----------|-------------------|\n" \
                f"| Average | {avg_metrics['val_prc_acc']:.4f} | {avg_metrics['val_prc_acc_flat']:.4f} | {avg_metrics['test_acc']:.4f} | {avg_metrics['test_prc_flat']:.4f} |\n"
    writer.add_text("Average_Metrics", avg_table)
    # Add average metrics to results
    avg_metrics['Fold'] = 'Average'
    fold_metrics.append(avg_metrics)

    # save averga date to CSV
    results_df = pd.DataFrame(fold_metrics)
    csv_path = os.path.join(run_dir, 'fold_results.csv')
    results_df.to_csv(csv_path, index=False)
    
    # Save fold indices CSV
    fold_indices_df.to_csv(os.path.join(run_dir, 'fold_indices.csv'), index=False)
    
    writer.close()


    print("\nTraining completed.")
    print("Average metrics across all folds:")
    for key, value in avg_metrics.items():
        print(f"  {key}: {value}")


if __name__ == "__main__":
    ## Path to data
    #path = "/home/glenn/Downloads/"
    path = "../DATA/"


    # Model parameters
    model_config = {
        "VRBModel": {
            "model" : VRBModel,
            "model_name": "facebook/hubert-large-ls960-ft",
            "hidden_units": 64,
            "n_gru": 3,
            "output_size": None  # Will be set dynamically
        },
        "Wav2Vec2ConvLSTMModel": {
            "model" : Wav2Vec2ConvLSTMModel,
            "model_name": "facebook/wav2vec2-base",
            "hidden_units": 128,
            "n_lstm": 2,
            "output_size": None  # Will be set dynamically
        },
            "RespBertCNNModelV2": {
            'model' : RespBertCNNModelV2,
            "model_name": "microsoft/wavlm-large",
            "hidden_units": 128,
            "output_size": None  
        }
    }
    

    
    # Train and data parameters
    epochs = 60
    batch_size = 25
    window_size = 30
    step_size = 6
    data_parts = 4 # aka folds
    early_stopping_patience = 10
    
    config = model_config["RespBertCNNModelV2"]
    

    ## same wav2vec2 base model and pipeline used in the paper
    processor = AutoProcessor.from_pretrained(config["model_name"])
    
    train(
        path_to_data=path+"ComParE2020_Breathing/wav/",
        path_to_labels=path+"ComParE2020_Breathing/lab/",
        window_size=window_size,
        batch_size=batch_size,
        config = config,
        step_size=step_size,
        data_parts= data_parts ,
        early_stopping_patience= early_stopping_patience,
        epochs= epochs,
        processor = processor
    )

/home/gdwildt/.conda/envs/thesis/lib/python3.11/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:61: FutureWarning: Loading a tokenizer inside Wav2Vec2Processor from a config that does not include a `tokenizer_class` attribute is deprecated and will be removed in v5. Please add `'tokenizer_class': 'Wav2Vec2CTCTokenizer'` attribute to either your `config.json` or `tokenizer_config.json` file to suppress this warning: 
  warnings.warn(


OSError: Can't load tokenizer for 'microsoft/wavlm-large'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'microsoft/wavlm-large' is the correct path to a directory containing all relevant files for a Wav2Vec2CTCTokenizer tokenizer.

In [ ]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import KFold
from datetime import datetime
#from torch.utils.tensorboard import SummaryWriter
import torchaudio
from typing import List, Tuple
from pt_utils import *
from pt_dataset import *
from pt_models import *
from pt_utils import *
from tensorboardX import SummaryWriter
from transformers import get_cosine_schedule_with_warmup


def create_run_directory():
    base_dir = "pt_runs_SOTA"
    timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
    run_dir = os.path.join(base_dir, timestamp)
    os.makedirs(run_dir, exist_ok=True)
    return run_dir

def _calculate_flattened_accuracy(average, ground_truth_labels):
    s_acc = 0
    for b in range(len(ground_truth_labels)):
        s, _ = scipy.stats.pearsonr(average[b], ground_truth_labels[b])
        s_acc += s
    return s_acc / len(ground_truth_labels)

def _choose_real_labs_only_with_filenames(labels, filenames):
    return labels[labels['filename'].isin(filenames)]

def _get_ground_truth_labels(ground_truth_names, labels):
    ground_truth_labels = []
    for batch_name in ground_truth_names:
        ground_truth_label = _choose_real_labs_only_with_filenames(labels, [batch_name])
        ground_truth_labels.append(ground_truth_label)
    return np.array(ground_truth_labels)[:, :, -1].astype(np.float32)

def train(path_to_data, path_to_labels, window_size=16, step_size=6, data_parts=4, epochs=100, batch_size=10, early_stopping_patience=20, config = None, processor = None):
    run_dir = create_run_directory()
    log_dir = os.path.join(run_dir, "logs")
    os.makedirs(log_dir, exist_ok=True)

    # Parameters
    length_sequence = window_size 
    step_sequence = step_size

    # Load and prepare data
    train_data, train_labels, train_dict, frame_rate = load_data(path_to_data, path_to_labels, 'train')
    devel_data, devel_labels, devel_dict, frame_rate = load_data(path_to_data, path_to_labels, 'devel')
    test_data, test_labels, test_dict, frame_rate = load_data(path_to_data, path_to_labels, 'test')

    # Combine train and devel data
    all_data = np.concatenate((train_data, devel_data), axis=0)
    all_labels = pd.concat([train_labels, devel_labels])
    all_dict = np.concatenate((list(train_dict.values()), list(devel_dict.values())), axis=0)

    # Prepare data
    prepared_data, prepared_labels, prepared_labels_timesteps = prepare_data(all_data, all_labels, all_dict, frame_rate, length_sequence * 16000, step_sequence * 16000)
    prepared_test_data, prepared_test_labels, prepared_test_labels_timesteps = prepare_data(test_data, test_labels, test_dict, frame_rate, length_sequence * 16000, step_sequence * 16000)

    # Create CSV file for storing fold indices
    fold_indices_df = pd.DataFrame(columns=['Fold', 'Train_Indices', 'Val_Indices'])

    # Cross-validation
    kf = KFold(n_splits=data_parts)
    fold_metrics = []
    # To accumulate metrics across folds for each epoch
    train_acc_epoch = []
    val_acc_epoch = []
    test_acc_epoch = []

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    
    config["output_size"] = prepared_labels.shape[-1]
    writer = SummaryWriter(log_dir=os.path.join(log_dir,config["model_name"]))

    for fold, (train_index, val_index) in enumerate(kf.split(prepared_data)):
        print(f"Fold {fold + 1}/{data_parts}")
        best_model_path = f"{run_dir}/best_model_fold{fold+1}"
        # Save fold indices
        fold_indices_df = fold_indices_df._append({
            'Fold': fold + 1,
            'Train_Indices': train_index.tolist(),
            'Val_Indices': val_index.tolist()
        }, ignore_index=True)

        # Split data
        train_d, val_d = prepared_data[train_index], prepared_data[val_index]
        train_lbs, val_lbs = prepared_labels[train_index], prepared_labels[val_index]
        train_timesteps, val_timesteps = prepared_labels_timesteps[train_index], prepared_labels_timesteps[val_index]
        
        # Reshape data
        train_d, train_lbs = reshaping_data_for_model(train_d, train_lbs)
        val_d, val_lbs = reshaping_data_for_model(val_d, val_lbs)
        test_d, test_lbs = reshaping_data_for_model(prepared_test_data, prepared_test_labels)
        
        print(train_d.shape)

        # Create datasets
        #train_dataset = BreathingDataset(train_d, train_lbs, processor, window_size, step_sequence, augment=True)
        train_dataset = BreathingDataset(train_d, train_lbs, processor,window_size, step_sequence)
        val_dataset = BreathingDataset(val_d, val_lbs, processor, window_size, step_sequence)
        test_dataset = BreathingDataset(test_d, test_lbs, processor, window_size, step_sequence)

        # Create DataLoaders
        train_loader = DataLoader(train_dataset, batch_size=batch_size,num_workers=2, shuffle=True, collate_fn=train_dataset.collate_fn)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, num_workers=1, collate_fn=val_dataset.collate_fn)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, num_workers=1, collate_fn=test_dataset.collate_fn)
        print(config["output_size"])
        # Create and initialize model
        model = config["model"](config).to(device)
        
        #### training optimiser parameters fror apple
        #learning_rate = 0.005 
        #optimizer = optim.Adam(model.parameters(), lr=learning_rate)

        #### training optimiser parameters fror harma_2023 VRB model 
        learning_rate = 0.01 # From the paper
        optimizer = optim.Adam(model.parameters())       
        
        best_val_loss = float('inf')
        best_val_loss_flat = float('inf')
        early_stopping_counter = 0
        # To accumulate metrics across folds for each epoch
        train_acc = []
        val_acc = []
        test_acc = []
        for epoch in range(epochs):

            model.train()
            train_loss = 0.0
            progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")
            
            for batch_d, batch_lbs in progress_bar:
                optimizer.zero_grad()
                input_values = batch_d.to(device)
                batch_lbs = batch_lbs.to(device)
                outputs = model(input_values)
                loss = correlation_coefficient_loss(outputs, batch_lbs)
                loss.backward()
                optimizer.step()
                train_loss += loss.item()
                progress_bar.set_postfix({'train_loss': f'{train_loss/(progress_bar.n+1):.4f}'})


            train_loss /= len(train_loader)

            # Combined validation loop
            model.eval()
            val_loss = 0.0
            val_pred = []
            with torch.no_grad():
                for batch_d, batch_lbs in val_loader:
                    input_values = batch_d.to(device)
                    batch_lbs = batch_lbs.to(device)
                    
                    outputs = model(input_values)
                    loss = correlation_coefficient_loss(outputs, batch_lbs)
                    val_loss += loss.item()
                    val_pred.extend(outputs.cpu().numpy())

            val_loss /= len(val_loader)

            # Calculate validation metrics
            val_pred = np.array(val_pred).reshape(val_timesteps.shape)
            val_ground_truth = _get_ground_truth_labels([all_dict[i] for i in val_index], all_labels)
            val_pred_flat = unsplit_data_ogsize(val_pred, window_size, step_sequence, 25, val_ground_truth.shape[-1])
            val_prc_coef = _calculate_flattened_accuracy(val_pred_flat, val_ground_truth)
            
            # Accumulate metrics for this fold and epoch
            train_acc.append(1- train_loss)
            val_acc.append(1- val_loss)

            # Log metrics
            writer.add_scalar(f"Loss/train_fold_{fold + 1}", train_loss, epoch)
            writer.add_scalar(f"Loss/val_fold_{fold + 1}", val_loss, epoch)
            writer.add_scalar(f"Pearson/val_fold_{fold + 1}", val_prc_coef, epoch)

            print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Pearson: {val_prc_coef:.4f}")

            # Check if validation loss improved
            if val_loss < best_val_loss:
                print(f"Validation loss improved from {best_val_loss:.4f} to {val_loss:.4f}. Saving best model...")
                best_val_loss = val_loss
                best_val_loss_flat = val_prc_coef
                early_stopping_counter = 0

                # Save the best model
                torch.save(model.state_dict(), best_model_path)
            else:
                early_stopping_counter += 1
                print(f"Validation loss did not improve for {early_stopping_counter} epochs.")
                #model.load_state_dict(torch.load(best_model_path))


            # Early stopping
            if early_stopping_counter >= early_stopping_patience:
                print(f"Early stopping triggered at epoch {epoch + 1}. Loading best model.")
                # Load the best model's weights
                model.load_state_dict(torch.load(best_model_path))
                break

        test_pred = []
        test_loss = 0.0
        with torch.no_grad():
            for batch_d, batch_lbs in test_loader:
                input_values = batch_d.to(device)
                batch_lbs = batch_lbs.to(device)
                
                outputs = model(input_values)
                loss = correlation_coefficient_loss(outputs, batch_lbs)
                test_loss += loss.item()
                test_pred.extend(outputs.cpu().numpy())

        test_loss /= len(test_loader)
        test_pred = np.array(test_pred).reshape(prepared_test_labels_timesteps.shape)
        test_ground_truth = _get_ground_truth_labels(list(test_dict.values()), test_labels)
        test_pred_flat = unsplit_data_ogsize(test_pred, window_size, step_sequence, 25, test_ground_truth.shape[-1])
        test_prc_coef = _calculate_flattened_accuracy(test_pred_flat, test_ground_truth)

        print(f"Fold {fold + 1}:")
        print(f"  Validation Pearson Coefficient  acc: {1- best_val_loss}")
        print(f"  Validation Pearson Coefficient flat acc: {best_val_loss_flat}")
        print(f"  Test acc: {1- test_loss}")
        print(f"  Test Pearson Coefficient acc(flattened): {test_prc_coef}")

        fold_metrics.append({
            'Fold': fold + 1,
            'val_prc_acc': 1- best_val_loss,
            'val_prc_acc_flat': best_val_loss_flat,
            'test_acc': 1- test_loss,
            'test_prc_flat': test_prc_coef
        })


                # Log fold-specific metrics as tables
        fold_table = f"| Fold | Val Pearson Acc | Val Pearson Flat | Test Acc | Test Pearson Flat |\n" \
                     f"|------|-----------------|------------------|----------|-------------------|\n" \
                     f"| {fold + 1} | {1 - val_loss:.4f} | {val_prc_coef:.4f} | {1 - test_loss:.4f} | {test_prc_coef:.4f} |\n"
        writer.add_text(f"Fold_{fold + 1}_Metrics", fold_table)
        # Accumulate fold metrics across all folds
        train_acc_epoch.append(train_acc)
        val_acc_epoch.append(train_acc)


    
        # After all folds, compute and log the average metrics per epoch across all folds
    for epoch in range(epochs):
        avg_train_loss = np.mean([fold_losses[epoch] for fold_losses in train_acc_epoch if len(fold_losses) > epoch])
        avg_val_loss = np.mean([fold_losses[epoch] for fold_losses in val_acc_epoch if len(fold_losses) > epoch])

        # Log the averaged metrics for the epoch across all folds
        writer.add_scalar("Average_acc/train", avg_train_loss, epoch)
        writer.add_scalar("Average_acc/val", avg_val_loss, epoch)
            

    # Calculate average metrics
    avg_metrics = {key: np.mean([fold[key] for fold in fold_metrics if key != 'Fold']) for key in fold_metrics[0].keys() if key != 'Fold'}
        # Log the final average table
    avg_table = "| Fold | Val Pearson Acc | Val Pearson Flat | Test Acc | Test Pearson Flat |\n" \
                "|------|-----------------|------------------|----------|-------------------|\n" \
                f"| Average | {avg_metrics['val_prc_acc']:.4f} | {avg_metrics['val_prc_acc_flat']:.4f} | {avg_metrics['test_acc']:.4f} | {avg_metrics['test_prc_flat']:.4f} |\n"
    writer.add_text("Average_Metrics", avg_table)
    # Add average metrics to results
    avg_metrics['Fold'] = 'Average'
    fold_metrics.append(avg_metrics)

    # save averga date to CSV
    results_df = pd.DataFrame(fold_metrics)
    csv_path = os.path.join(run_dir, 'fold_results.csv')
    results_df.to_csv(csv_path, index=False)
    
    # Save fold indices CSV
    fold_indices_df.to_csv(os.path.join(run_dir, 'fold_indices.csv'), index=False)
    
    writer.close()


    print("\nTraining completed.")
    print("Average metrics across all folds:")
    for key, value in avg_metrics.items():
        print(f"  {key}: {value}")


if __name__ == "__main__":
    ## Path to data
    #path = "/home/glenn/Downloads/"
    path = "../DATA/"


    # Model parameters
    model_config = {
        "VRBModel": {
            "model" : VRBModel,
            "model_name": "facebook/hubert-large-ls960-ft",
            "hidden_units": 64,
            "n_gru": 3,
            "output_size": None  # Will be set dynamically
        },
        "Wav2Vec2ConvLSTMModel": {
            "model" : Wav2Vec2ConvLSTMModel,
            "model_name": "facebook/wav2vec2-base",
            "hidden_units": 128,
            "n_lstm": 2,
            "output_size": None  # Will be set dynamically
        },
            "RespBertCNNModelV2": {
            'model' : RespBertCNNModelV2,
            "model_name": "microsoft/wavlm-large",
            "hidden_units": 128,
            "output_size": None  
        }
    }
    

    
    # Train and data parameters
    epochs = 60
    batch_size = 25
    window_size = 30
    step_size = 6
    data_parts = 4 # aka folds
    early_stopping_patience = 10
    
    config = model_config["RespBertCNNModelV2"]
    

    ## same wav2vec2 base model and pipeline used in the paper
    #processor = AutoProcessor.from_pretrained(config["model_name"])
    processor = Wav2Vec2FeatureExtractor.from_pretrained(config["model_name"])

    train(
        path_to_data=path+"ComParE2020_Breathing/wav/",
        path_to_labels=path+"ComParE2020_Breathing/lab/",
        window_size=window_size,
        batch_size=batch_size,
        config = config,
        step_size=step_size,
        data_parts= data_parts ,
        early_stopping_patience= early_stopping_patience,
        epochs= epochs,
        processor = processor
    )

Fold 1/4
(864, 480000)
750


Epoch 1/60:   0%|          | 0/35 [00:00<?, ?it/s]/home/gdwildt/.conda/envs/thesis/lib/python3.11/site-packages/torch/nn/functional.py:5193: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
Epoch 1/60:   0%|          | 0/35 [00:02<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 3.36 GiB. GPU 0 has a total capacity of 93.12 GiB of which 2.34 GiB is free. Including non-PyTorch memory, this process has 90.77 GiB memory in use. Of the allocated memory 89.21 GiB is allocated by PyTorch, and 912.60 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import KFold
from datetime import datetime
#from torch.utils.tensorboard import SummaryWriter
import torchaudio
from typing import List, Tuple
from pt_utils import *
from pt_dataset import *
from pt_models import *
from pt_utils import *
from tensorboardX import SummaryWriter
from transformers import get_cosine_schedule_with_warmup


def create_run_directory():
    base_dir = "pt_runs_SOTA"
    timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
    run_dir = os.path.join(base_dir, timestamp)
    os.makedirs(run_dir, exist_ok=True)
    return run_dir

def _calculate_flattened_accuracy(average, ground_truth_labels):
    s_acc = 0
    for b in range(len(ground_truth_labels)):
        s, _ = scipy.stats.pearsonr(average[b], ground_truth_labels[b])
        s_acc += s
    return s_acc / len(ground_truth_labels)

def _choose_real_labs_only_with_filenames(labels, filenames):
    return labels[labels['filename'].isin(filenames)]

def _get_ground_truth_labels(ground_truth_names, labels):
    ground_truth_labels = []
    for batch_name in ground_truth_names:
        ground_truth_label = _choose_real_labs_only_with_filenames(labels, [batch_name])
        ground_truth_labels.append(ground_truth_label)
    return np.array(ground_truth_labels)[:, :, -1].astype(np.float32)

def train(path_to_data, path_to_labels, window_size=16, step_size=6, data_parts=4, epochs=100, batch_size=10, early_stopping_patience=20, config = None, processor = None):
    run_dir = create_run_directory()
    log_dir = os.path.join(run_dir, "logs")
    os.makedirs(log_dir, exist_ok=True)

    # Parameters
    length_sequence = window_size 
    step_sequence = step_size

    # Load and prepare data
    train_data, train_labels, train_dict, frame_rate = load_data(path_to_data, path_to_labels, 'train')
    devel_data, devel_labels, devel_dict, frame_rate = load_data(path_to_data, path_to_labels, 'devel')
    test_data, test_labels, test_dict, frame_rate = load_data(path_to_data, path_to_labels, 'test')

    # Combine train and devel data
    all_data = np.concatenate((train_data, devel_data), axis=0)
    all_labels = pd.concat([train_labels, devel_labels])
    all_dict = np.concatenate((list(train_dict.values()), list(devel_dict.values())), axis=0)

    # Prepare data
    prepared_data, prepared_labels, prepared_labels_timesteps = prepare_data(all_data, all_labels, all_dict, frame_rate, length_sequence * 16000, step_sequence * 16000)
    prepared_test_data, prepared_test_labels, prepared_test_labels_timesteps = prepare_data(test_data, test_labels, test_dict, frame_rate, length_sequence * 16000, step_sequence * 16000)

    # Create CSV file for storing fold indices
    fold_indices_df = pd.DataFrame(columns=['Fold', 'Train_Indices', 'Val_Indices'])

    # Cross-validation
    kf = KFold(n_splits=data_parts)
    fold_metrics = []
    # To accumulate metrics across folds for each epoch
    train_acc_epoch = []
    val_acc_epoch = []
    test_acc_epoch = []

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    
    config["output_size"] = prepared_labels.shape[-1]
    writer = SummaryWriter(log_dir=os.path.join(log_dir,config["model_name"]))

    for fold, (train_index, val_index) in enumerate(kf.split(prepared_data)):
        print(f"Fold {fold + 1}/{data_parts}")
        best_model_path = f"{run_dir}/best_model_fold{fold+1}"
        # Save fold indices
        fold_indices_df = fold_indices_df._append({
            'Fold': fold + 1,
            'Train_Indices': train_index.tolist(),
            'Val_Indices': val_index.tolist()
        }, ignore_index=True)

        # Split data
        train_d, val_d = prepared_data[train_index], prepared_data[val_index]
        train_lbs, val_lbs = prepared_labels[train_index], prepared_labels[val_index]
        train_timesteps, val_timesteps = prepared_labels_timesteps[train_index], prepared_labels_timesteps[val_index]
        
        # Reshape data
        train_d, train_lbs = reshaping_data_for_model(train_d, train_lbs)
        val_d, val_lbs = reshaping_data_for_model(val_d, val_lbs)
        test_d, test_lbs = reshaping_data_for_model(prepared_test_data, prepared_test_labels)
        
        print(train_d.shape)

        # Create datasets
        #train_dataset = BreathingDataset(train_d, train_lbs, processor, window_size, step_sequence, augment=True)
        train_dataset = BreathingDataset(train_d, train_lbs, processor,window_size, step_sequence)
        val_dataset = BreathingDataset(val_d, val_lbs, processor, window_size, step_sequence)
        test_dataset = BreathingDataset(test_d, test_lbs, processor, window_size, step_sequence)

        # Create DataLoaders
        train_loader = DataLoader(train_dataset, batch_size=batch_size,num_workers=2, shuffle=True, collate_fn=train_dataset.collate_fn)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, num_workers=1, collate_fn=val_dataset.collate_fn)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, num_workers=1, collate_fn=test_dataset.collate_fn)
        print(config["output_size"])
        # Create and initialize model
        model = config["model"](config).to(device)
        
        #### training optimiser parameters fror apple
        #learning_rate = 0.005 
        #optimizer = optim.Adam(model.parameters(), lr=learning_rate)

        #### training optimiser parameters fror harma_2023 VRB model 
        learning_rate = 0.01 # From the paper
        optimizer = optim.Adam(model.parameters())       
        
        best_val_loss = float('inf')
        best_val_loss_flat = float('inf')
        early_stopping_counter = 0
        # To accumulate metrics across folds for each epoch
        train_acc = []
        val_acc = []
        test_acc = []
        for epoch in range(epochs):

            model.train()
            train_loss = 0.0
            progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")
            
            for batch_d, batch_lbs in progress_bar:
                optimizer.zero_grad()
                input_values = batch_d.to(device)
                batch_lbs = batch_lbs.to(device)
                outputs = model(input_values)
                loss = correlation_coefficient_loss(outputs, batch_lbs)
                loss.backward()
                optimizer.step()
                train_loss += loss.item()
                progress_bar.set_postfix({'train_loss': f'{train_loss/(progress_bar.n+1):.4f}'})


            train_loss /= len(train_loader)

            # Combined validation loop
            model.eval()
            val_loss = 0.0
            val_pred = []
            with torch.no_grad():
                for batch_d, batch_lbs in val_loader:
                    input_values = batch_d.to(device)
                    batch_lbs = batch_lbs.to(device)
                    
                    outputs = model(input_values)
                    loss = correlation_coefficient_loss(outputs, batch_lbs)
                    val_loss += loss.item()
                    val_pred.extend(outputs.cpu().numpy())

            val_loss /= len(val_loader)

            # Calculate validation metrics
            val_pred = np.array(val_pred).reshape(val_timesteps.shape)
            val_ground_truth = _get_ground_truth_labels([all_dict[i] for i in val_index], all_labels)
            val_pred_flat = unsplit_data_ogsize(val_pred, window_size, step_sequence, 25, val_ground_truth.shape[-1])
            val_prc_coef = _calculate_flattened_accuracy(val_pred_flat, val_ground_truth)
            
            # Accumulate metrics for this fold and epoch
            train_acc.append(1- train_loss)
            val_acc.append(1- val_loss)

            # Log metrics
            writer.add_scalar(f"Loss/train_fold_{fold + 1}", train_loss, epoch)
            writer.add_scalar(f"Loss/val_fold_{fold + 1}", val_loss, epoch)
            writer.add_scalar(f"Pearson/val_fold_{fold + 1}", val_prc_coef, epoch)

            print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Pearson: {val_prc_coef:.4f}")

            # Check if validation loss improved
            if val_loss < best_val_loss:
                print(f"Validation loss improved from {best_val_loss:.4f} to {val_loss:.4f}. Saving best model...")
                best_val_loss = val_loss
                best_val_loss_flat = val_prc_coef
                early_stopping_counter = 0

                # Save the best model
                torch.save(model.state_dict(), best_model_path)
            else:
                early_stopping_counter += 1
                print(f"Validation loss did not improve for {early_stopping_counter} epochs.")
                #model.load_state_dict(torch.load(best_model_path))


            # Early stopping
            if early_stopping_counter >= early_stopping_patience:
                print(f"Early stopping triggered at epoch {epoch + 1}. Loading best model.")
                # Load the best model's weights
                model.load_state_dict(torch.load(best_model_path))
                break

        test_pred = []
        test_loss = 0.0
        with torch.no_grad():
            for batch_d, batch_lbs in test_loader:
                input_values = batch_d.to(device)
                batch_lbs = batch_lbs.to(device)
                
                outputs = model(input_values)
                loss = correlation_coefficient_loss(outputs, batch_lbs)
                test_loss += loss.item()
                test_pred.extend(outputs.cpu().numpy())

        test_loss /= len(test_loader)
        test_pred = np.array(test_pred).reshape(prepared_test_labels_timesteps.shape)
        test_ground_truth = _get_ground_truth_labels(list(test_dict.values()), test_labels)
        test_pred_flat = unsplit_data_ogsize(test_pred, window_size, step_sequence, 25, test_ground_truth.shape[-1])
        test_prc_coef = _calculate_flattened_accuracy(test_pred_flat, test_ground_truth)

        print(f"Fold {fold + 1}:")
        print(f"  Validation Pearson Coefficient  acc: {1- best_val_loss}")
        print(f"  Validation Pearson Coefficient flat acc: {best_val_loss_flat}")
        print(f"  Test acc: {1- test_loss}")
        print(f"  Test Pearson Coefficient acc(flattened): {test_prc_coef}")

        fold_metrics.append({
            'Fold': fold + 1,
            'val_prc_acc': 1- best_val_loss,
            'val_prc_acc_flat': best_val_loss_flat,
            'test_acc': 1- test_loss,
            'test_prc_flat': test_prc_coef
        })


                # Log fold-specific metrics as tables
        fold_table = f"| Fold | Val Pearson Acc | Val Pearson Flat | Test Acc | Test Pearson Flat |\n" \
                     f"|------|-----------------|------------------|----------|-------------------|\n" \
                     f"| {fold + 1} | {1 - val_loss:.4f} | {val_prc_coef:.4f} | {1 - test_loss:.4f} | {test_prc_coef:.4f} |\n"
        writer.add_text(f"Fold_{fold + 1}_Metrics", fold_table)
        # Accumulate fold metrics across all folds
        train_acc_epoch.append(train_acc)
        val_acc_epoch.append(train_acc)


    
        # After all folds, compute and log the average metrics per epoch across all folds
    for epoch in range(epochs):
        avg_train_loss = np.mean([fold_losses[epoch] for fold_losses in train_acc_epoch if len(fold_losses) > epoch])
        avg_val_loss = np.mean([fold_losses[epoch] for fold_losses in val_acc_epoch if len(fold_losses) > epoch])

        # Log the averaged metrics for the epoch across all folds
        writer.add_scalar("Average_acc/train", avg_train_loss, epoch)
        writer.add_scalar("Average_acc/val", avg_val_loss, epoch)
            

    # Calculate average metrics
    avg_metrics = {key: np.mean([fold[key] for fold in fold_metrics if key != 'Fold']) for key in fold_metrics[0].keys() if key != 'Fold'}
        # Log the final average table
    avg_table = "| Fold | Val Pearson Acc | Val Pearson Flat | Test Acc | Test Pearson Flat |\n" \
                "|------|-----------------|------------------|----------|-------------------|\n" \
                f"| Average | {avg_metrics['val_prc_acc']:.4f} | {avg_metrics['val_prc_acc_flat']:.4f} | {avg_metrics['test_acc']:.4f} | {avg_metrics['test_prc_flat']:.4f} |\n"
    writer.add_text("Average_Metrics", avg_table)
    # Add average metrics to results
    avg_metrics['Fold'] = 'Average'
    fold_metrics.append(avg_metrics)

    # save averga date to CSV
    results_df = pd.DataFrame(fold_metrics)
    csv_path = os.path.join(run_dir, 'fold_results.csv')
    results_df.to_csv(csv_path, index=False)
    
    # Save fold indices CSV
    fold_indices_df.to_csv(os.path.join(run_dir, 'fold_indices.csv'), index=False)
    
    writer.close()


    print("\nTraining completed.")
    print("Average metrics across all folds:")
    for key, value in avg_metrics.items():
        print(f"  {key}: {value}")


if __name__ == "__main__":
    ## Path to data
    #path = "/home/glenn/Downloads/"
    path = "../DATA/"


    # Model parameters
    model_config = {
        "VRBModel": {
            "model" : VRBModel,
            "model_name": "facebook/hubert-large-ls960-ft",
            "hidden_units": 64,
            "n_gru": 3,
            "output_size": None  # Will be set dynamically
        },
        "Wav2Vec2ConvLSTMModel": {
            "model" : Wav2Vec2ConvLSTMModel,
            "model_name": "facebook/wav2vec2-base",
            "hidden_units": 128,
            "n_lstm": 2,
            "output_size": None  # Will be set dynamically
        },
            "RespBertCNNModelV2": {
            'model' : RespBertCNNModelV2,
            "model_name": "microsoft/wavlm-large",
            "hidden_units": 128,
            "output_size": None  
        }
    }
    

    
    # Train and data parameters
    epochs = 60
    batch_size = 15
    window_size = 30
    step_size = 6
    data_parts = 4 # aka folds
    early_stopping_patience = 10
    
    config = model_config["RespBertCNNModelV2"]
    

    ## same wav2vec2 base model and pipeline used in the paper
    #processor = AutoProcessor.from_pretrained(config["model_name"])
    processor = Wav2Vec2FeatureExtractor.from_pretrained(config["model_name"])

    train(
        path_to_data=path+"ComParE2020_Breathing/wav/",
        path_to_labels=path+"ComParE2020_Breathing/lab/",
        window_size=window_size,
        batch_size=batch_size,
        config = config,
        step_size=step_size,
        data_parts= data_parts ,
        early_stopping_patience= early_stopping_patience,
        epochs= epochs,
        processor = processor
    )

Fold 1/4
(864, 480000)
750


Epoch 1/60:   0%|          | 0/58 [00:01<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 2.75 GiB. GPU 0 has a total capacity of 93.12 GiB of which 1.41 GiB is free. Including non-PyTorch memory, this process has 91.70 GiB memory in use. Of the allocated memory 90.99 GiB is allocated by PyTorch, and 40.32 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import KFold
from datetime import datetime
#from torch.utils.tensorboard import SummaryWriter
import torchaudio
from typing import List, Tuple
from pt_utils import *
from pt_dataset import *
from pt_models import *
from pt_utils import *
from tensorboardX import SummaryWriter
from transformers import get_cosine_schedule_with_warmup


def create_run_directory():
    base_dir = "pt_runs_SOTA"
    timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
    run_dir = os.path.join(base_dir, timestamp)
    os.makedirs(run_dir, exist_ok=True)
    return run_dir

def _calculate_flattened_accuracy(average, ground_truth_labels):
    s_acc = 0
    for b in range(len(ground_truth_labels)):
        s, _ = scipy.stats.pearsonr(average[b], ground_truth_labels[b])
        s_acc += s
    return s_acc / len(ground_truth_labels)

def _choose_real_labs_only_with_filenames(labels, filenames):
    return labels[labels['filename'].isin(filenames)]

def _get_ground_truth_labels(ground_truth_names, labels):
    ground_truth_labels = []
    for batch_name in ground_truth_names:
        ground_truth_label = _choose_real_labs_only_with_filenames(labels, [batch_name])
        ground_truth_labels.append(ground_truth_label)
    return np.array(ground_truth_labels)[:, :, -1].astype(np.float32)

def train(path_to_data, path_to_labels, window_size=16, step_size=6, data_parts=4, epochs=100, batch_size=10, early_stopping_patience=20, config = None, processor = None):
    run_dir = create_run_directory()
    log_dir = os.path.join(run_dir, "logs")
    os.makedirs(log_dir, exist_ok=True)

    # Parameters
    length_sequence = window_size 
    step_sequence = step_size

    # Load and prepare data
    train_data, train_labels, train_dict, frame_rate = load_data(path_to_data, path_to_labels, 'train')
    devel_data, devel_labels, devel_dict, frame_rate = load_data(path_to_data, path_to_labels, 'devel')
    test_data, test_labels, test_dict, frame_rate = load_data(path_to_data, path_to_labels, 'test')

    # Combine train and devel data
    all_data = np.concatenate((train_data, devel_data), axis=0)
    all_labels = pd.concat([train_labels, devel_labels])
    all_dict = np.concatenate((list(train_dict.values()), list(devel_dict.values())), axis=0)

    # Prepare data
    prepared_data, prepared_labels, prepared_labels_timesteps = prepare_data(all_data, all_labels, all_dict, frame_rate, length_sequence * 16000, step_sequence * 16000)
    prepared_test_data, prepared_test_labels, prepared_test_labels_timesteps = prepare_data(test_data, test_labels, test_dict, frame_rate, length_sequence * 16000, step_sequence * 16000)

    # Create CSV file for storing fold indices
    fold_indices_df = pd.DataFrame(columns=['Fold', 'Train_Indices', 'Val_Indices'])

    # Cross-validation
    kf = KFold(n_splits=data_parts)
    fold_metrics = []
    # To accumulate metrics across folds for each epoch
    train_acc_epoch = []
    val_acc_epoch = []
    test_acc_epoch = []

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    
    config["output_size"] = prepared_labels.shape[-1]
    writer = SummaryWriter(log_dir=os.path.join(log_dir,config["model_name"]))

    for fold, (train_index, val_index) in enumerate(kf.split(prepared_data)):
        print(f"Fold {fold + 1}/{data_parts}")
        best_model_path = f"{run_dir}/best_model_fold{fold+1}"
        # Save fold indices
        fold_indices_df = fold_indices_df._append({
            'Fold': fold + 1,
            'Train_Indices': train_index.tolist(),
            'Val_Indices': val_index.tolist()
        }, ignore_index=True)

        # Split data
        train_d, val_d = prepared_data[train_index], prepared_data[val_index]
        train_lbs, val_lbs = prepared_labels[train_index], prepared_labels[val_index]
        train_timesteps, val_timesteps = prepared_labels_timesteps[train_index], prepared_labels_timesteps[val_index]
        
        # Reshape data
        train_d, train_lbs = reshaping_data_for_model(train_d, train_lbs)
        val_d, val_lbs = reshaping_data_for_model(val_d, val_lbs)
        test_d, test_lbs = reshaping_data_for_model(prepared_test_data, prepared_test_labels)
        
        print(train_d.shape)

        # Create datasets
        #train_dataset = BreathingDataset(train_d, train_lbs, processor, window_size, step_sequence, augment=True)
        train_dataset = BreathingDataset(train_d, train_lbs, processor,window_size, step_sequence)
        val_dataset = BreathingDataset(val_d, val_lbs, processor, window_size, step_sequence)
        test_dataset = BreathingDataset(test_d, test_lbs, processor, window_size, step_sequence)

        # Create DataLoaders
        train_loader = DataLoader(train_dataset, batch_size=batch_size,num_workers=2, shuffle=True, collate_fn=train_dataset.collate_fn)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, num_workers=1, collate_fn=val_dataset.collate_fn)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, num_workers=1, collate_fn=test_dataset.collate_fn)
        print(config["output_size"])
        # Create and initialize model
        model = config["model"](config).to(device)
        
        #### training optimiser parameters fror apple
        #learning_rate = 0.005 
        #optimizer = optim.Adam(model.parameters(), lr=learning_rate)

        #### training optimiser parameters fror harma_2023 VRB model 
        learning_rate = 0.01 # From the paper
        optimizer = optim.Adam(model.parameters())       
        
        best_val_loss = float('inf')
        best_val_loss_flat = float('inf')
        early_stopping_counter = 0
        # To accumulate metrics across folds for each epoch
        train_acc = []
        val_acc = []
        test_acc = []
        for epoch in range(epochs):

            model.train()
            train_loss = 0.0
            progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")
            
            for batch_d, batch_lbs in progress_bar:
                optimizer.zero_grad()
                input_values = batch_d.to(device)
                batch_lbs = batch_lbs.to(device)
                outputs = model(input_values)
                loss = correlation_coefficient_loss(outputs, batch_lbs)
                loss.backward()
                optimizer.step()
                train_loss += loss.item()
                progress_bar.set_postfix({'train_loss': f'{train_loss/(progress_bar.n+1):.4f}'})


            train_loss /= len(train_loader)

            # Combined validation loop
            model.eval()
            val_loss = 0.0
            val_pred = []
            with torch.no_grad():
                for batch_d, batch_lbs in val_loader:
                    input_values = batch_d.to(device)
                    batch_lbs = batch_lbs.to(device)
                    
                    outputs = model(input_values)
                    loss = correlation_coefficient_loss(outputs, batch_lbs)
                    val_loss += loss.item()
                    val_pred.extend(outputs.cpu().numpy())

            val_loss /= len(val_loader)

            # Calculate validation metrics
            val_pred = np.array(val_pred).reshape(val_timesteps.shape)
            val_ground_truth = _get_ground_truth_labels([all_dict[i] for i in val_index], all_labels)
            val_pred_flat = unsplit_data_ogsize(val_pred, window_size, step_sequence, 25, val_ground_truth.shape[-1])
            val_prc_coef = _calculate_flattened_accuracy(val_pred_flat, val_ground_truth)
            
            # Accumulate metrics for this fold and epoch
            train_acc.append(1- train_loss)
            val_acc.append(1- val_loss)

            # Log metrics
            writer.add_scalar(f"Loss/train_fold_{fold + 1}", train_loss, epoch)
            writer.add_scalar(f"Loss/val_fold_{fold + 1}", val_loss, epoch)
            writer.add_scalar(f"Pearson/val_fold_{fold + 1}", val_prc_coef, epoch)

            print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Pearson: {val_prc_coef:.4f}")

            # Check if validation loss improved
            if val_loss < best_val_loss:
                print(f"Validation loss improved from {best_val_loss:.4f} to {val_loss:.4f}. Saving best model...")
                best_val_loss = val_loss
                best_val_loss_flat = val_prc_coef
                early_stopping_counter = 0

                # Save the best model
                torch.save(model.state_dict(), best_model_path)
            else:
                early_stopping_counter += 1
                print(f"Validation loss did not improve for {early_stopping_counter} epochs.")
                #model.load_state_dict(torch.load(best_model_path))


            # Early stopping
            if early_stopping_counter >= early_stopping_patience:
                print(f"Early stopping triggered at epoch {epoch + 1}. Loading best model.")
                # Load the best model's weights
                model.load_state_dict(torch.load(best_model_path))
                break

        test_pred = []
        test_loss = 0.0
        with torch.no_grad():
            for batch_d, batch_lbs in test_loader:
                input_values = batch_d.to(device)
                batch_lbs = batch_lbs.to(device)
                
                outputs = model(input_values)
                loss = correlation_coefficient_loss(outputs, batch_lbs)
                test_loss += loss.item()
                test_pred.extend(outputs.cpu().numpy())

        test_loss /= len(test_loader)
        test_pred = np.array(test_pred).reshape(prepared_test_labels_timesteps.shape)
        test_ground_truth = _get_ground_truth_labels(list(test_dict.values()), test_labels)
        test_pred_flat = unsplit_data_ogsize(test_pred, window_size, step_sequence, 25, test_ground_truth.shape[-1])
        test_prc_coef = _calculate_flattened_accuracy(test_pred_flat, test_ground_truth)

        print(f"Fold {fold + 1}:")
        print(f"  Validation Pearson Coefficient  acc: {1- best_val_loss}")
        print(f"  Validation Pearson Coefficient flat acc: {best_val_loss_flat}")
        print(f"  Test acc: {1- test_loss}")
        print(f"  Test Pearson Coefficient acc(flattened): {test_prc_coef}")

        fold_metrics.append({
            'Fold': fold + 1,
            'val_prc_acc': 1- best_val_loss,
            'val_prc_acc_flat': best_val_loss_flat,
            'test_acc': 1- test_loss,
            'test_prc_flat': test_prc_coef
        })


                # Log fold-specific metrics as tables
        fold_table = f"| Fold | Val Pearson Acc | Val Pearson Flat | Test Acc | Test Pearson Flat |\n" \
                     f"|------|-----------------|------------------|----------|-------------------|\n" \
                     f"| {fold + 1} | {1 - val_loss:.4f} | {val_prc_coef:.4f} | {1 - test_loss:.4f} | {test_prc_coef:.4f} |\n"
        writer.add_text(f"Fold_{fold + 1}_Metrics", fold_table)
        # Accumulate fold metrics across all folds
        train_acc_epoch.append(train_acc)
        val_acc_epoch.append(train_acc)


    
        # After all folds, compute and log the average metrics per epoch across all folds
    for epoch in range(epochs):
        avg_train_loss = np.mean([fold_losses[epoch] for fold_losses in train_acc_epoch if len(fold_losses) > epoch])
        avg_val_loss = np.mean([fold_losses[epoch] for fold_losses in val_acc_epoch if len(fold_losses) > epoch])

        # Log the averaged metrics for the epoch across all folds
        writer.add_scalar("Average_acc/train", avg_train_loss, epoch)
        writer.add_scalar("Average_acc/val", avg_val_loss, epoch)
            

    # Calculate average metrics
    avg_metrics = {key: np.mean([fold[key] for fold in fold_metrics if key != 'Fold']) for key in fold_metrics[0].keys() if key != 'Fold'}
        # Log the final average table
    avg_table = "| Fold | Val Pearson Acc | Val Pearson Flat | Test Acc | Test Pearson Flat |\n" \
                "|------|-----------------|------------------|----------|-------------------|\n" \
                f"| Average | {avg_metrics['val_prc_acc']:.4f} | {avg_metrics['val_prc_acc_flat']:.4f} | {avg_metrics['test_acc']:.4f} | {avg_metrics['test_prc_flat']:.4f} |\n"
    writer.add_text("Average_Metrics", avg_table)
    # Add average metrics to results
    avg_metrics['Fold'] = 'Average'
    fold_metrics.append(avg_metrics)

    # save averga date to CSV
    results_df = pd.DataFrame(fold_metrics)
    csv_path = os.path.join(run_dir, 'fold_results.csv')
    results_df.to_csv(csv_path, index=False)
    
    # Save fold indices CSV
    fold_indices_df.to_csv(os.path.join(run_dir, 'fold_indices.csv'), index=False)
    
    writer.close()


    print("\nTraining completed.")
    print("Average metrics across all folds:")
    for key, value in avg_metrics.items():
        print(f"  {key}: {value}")


if __name__ == "__main__":
    ## Path to data
    #path = "/home/glenn/Downloads/"
    path = "../DATA/"


    # Model parameters
    model_config = {
        "VRBModel": {
            "model" : VRBModel,
            "model_name": "facebook/hubert-large-ls960-ft",
            "hidden_units": 64,
            "n_gru": 3,
            "output_size": None  # Will be set dynamically
        },
        "Wav2Vec2ConvLSTMModel": {
            "model" : Wav2Vec2ConvLSTMModel,
            "model_name": "facebook/wav2vec2-base",
            "hidden_units": 128,
            "n_lstm": 2,
            "output_size": None  # Will be set dynamically
        },
            "RespBertCNNModelV2": {
            'model' : RespBertCNNModelV2,
            "model_name": "microsoft/wavlm-large",
            "hidden_units": 128,
            "output_size": None  
        }
    }
    

    
    # Train and data parameters
    epochs = 60
    batch_size = 15
    window_size = 30
    step_size = 6
    data_parts = 4 # aka folds
    early_stopping_patience = 10
    
    config = model_config["RespBertCNNModelV2"]
    

    ## same wav2vec2 base model and pipeline used in the paper
    #processor = AutoProcessor.from_pretrained(config["model_name"])
    processor = Wav2Vec2FeatureExtractor.from_pretrained(config["model_name"])

    train(
        path_to_data=path+"ComParE2020_Breathing/wav/",
        path_to_labels=path+"ComParE2020_Breathing/lab/",
        window_size=window_size,
        batch_size=batch_size,
        config = config,
        step_size=step_size,
        data_parts= data_parts ,
        early_stopping_patience= early_stopping_patience,
        epochs= epochs,
        processor = processor
    )

Fold 1/4
(864, 480000)
750


Epoch 1/60:   0%|          | 0/58 [00:00<?, ?it/s]/home/gdwildt/.conda/envs/thesis/lib/python3.11/site-packages/torch/nn/functional.py:5193: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
Epoch 1/60:   0%|          | 0/58 [00:02<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 2.02 GiB. GPU 0 has a total capacity of 93.12 GiB of which 314.75 MiB is free. Including non-PyTorch memory, this process has 92.80 GiB memory in use. Of the allocated memory 91.90 GiB is allocated by PyTorch, and 245.51 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import KFold
from datetime import datetime
#from torch.utils.tensorboard import SummaryWriter
import torchaudio
from typing import List, Tuple
from pt_utils import *
from pt_dataset import *
from pt_models import *
from pt_utils import *
from tensorboardX import SummaryWriter
from transformers import get_cosine_schedule_with_warmup


def create_run_directory():
    base_dir = "pt_runs_SOTA"
    timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
    run_dir = os.path.join(base_dir, timestamp)
    os.makedirs(run_dir, exist_ok=True)
    return run_dir

def _calculate_flattened_accuracy(average, ground_truth_labels):
    s_acc = 0
    for b in range(len(ground_truth_labels)):
        s, _ = scipy.stats.pearsonr(average[b], ground_truth_labels[b])
        s_acc += s
    return s_acc / len(ground_truth_labels)

def _choose_real_labs_only_with_filenames(labels, filenames):
    return labels[labels['filename'].isin(filenames)]

def _get_ground_truth_labels(ground_truth_names, labels):
    ground_truth_labels = []
    for batch_name in ground_truth_names:
        ground_truth_label = _choose_real_labs_only_with_filenames(labels, [batch_name])
        ground_truth_labels.append(ground_truth_label)
    return np.array(ground_truth_labels)[:, :, -1].astype(np.float32)

def train(path_to_data, path_to_labels, window_size=16, step_size=6, data_parts=4, epochs=100, batch_size=10, early_stopping_patience=20, config = None, processor = None):
    run_dir = create_run_directory()
    log_dir = os.path.join(run_dir, "logs")
    os.makedirs(log_dir, exist_ok=True)

    # Parameters
    length_sequence = window_size 
    step_sequence = step_size

    # Load and prepare data
    train_data, train_labels, train_dict, frame_rate = load_data(path_to_data, path_to_labels, 'train')
    devel_data, devel_labels, devel_dict, frame_rate = load_data(path_to_data, path_to_labels, 'devel')
    test_data, test_labels, test_dict, frame_rate = load_data(path_to_data, path_to_labels, 'test')

    # Combine train and devel data
    all_data = np.concatenate((train_data, devel_data), axis=0)
    all_labels = pd.concat([train_labels, devel_labels])
    all_dict = np.concatenate((list(train_dict.values()), list(devel_dict.values())), axis=0)

    # Prepare data
    prepared_data, prepared_labels, prepared_labels_timesteps = prepare_data(all_data, all_labels, all_dict, frame_rate, length_sequence * 16000, step_sequence * 16000)
    prepared_test_data, prepared_test_labels, prepared_test_labels_timesteps = prepare_data(test_data, test_labels, test_dict, frame_rate, length_sequence * 16000, step_sequence * 16000)

    # Create CSV file for storing fold indices
    fold_indices_df = pd.DataFrame(columns=['Fold', 'Train_Indices', 'Val_Indices'])

    # Cross-validation
    kf = KFold(n_splits=data_parts)
    fold_metrics = []
    # To accumulate metrics across folds for each epoch
    train_acc_epoch = []
    val_acc_epoch = []
    test_acc_epoch = []

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    
    config["output_size"] = prepared_labels.shape[-1]
    writer = SummaryWriter(log_dir=os.path.join(log_dir,config["model_name"]))

    for fold, (train_index, val_index) in enumerate(kf.split(prepared_data)):
        print(f"Fold {fold + 1}/{data_parts}")
        best_model_path = f"{run_dir}/best_model_fold{fold+1}"
        # Save fold indices
        fold_indices_df = fold_indices_df._append({
            'Fold': fold + 1,
            'Train_Indices': train_index.tolist(),
            'Val_Indices': val_index.tolist()
        }, ignore_index=True)

        # Split data
        train_d, val_d = prepared_data[train_index], prepared_data[val_index]
        train_lbs, val_lbs = prepared_labels[train_index], prepared_labels[val_index]
        train_timesteps, val_timesteps = prepared_labels_timesteps[train_index], prepared_labels_timesteps[val_index]
        
        # Reshape data
        train_d, train_lbs = reshaping_data_for_model(train_d, train_lbs)
        val_d, val_lbs = reshaping_data_for_model(val_d, val_lbs)
        test_d, test_lbs = reshaping_data_for_model(prepared_test_data, prepared_test_labels)
        
        print(train_d.shape)

        # Create datasets
        #train_dataset = BreathingDataset(train_d, train_lbs, processor, window_size, step_sequence, augment=True)
        train_dataset = BreathingDataset(train_d, train_lbs, processor,window_size, step_sequence)
        val_dataset = BreathingDataset(val_d, val_lbs, processor, window_size, step_sequence)
        test_dataset = BreathingDataset(test_d, test_lbs, processor, window_size, step_sequence)

        # Create DataLoaders
        train_loader = DataLoader(train_dataset, batch_size=batch_size,num_workers=2, shuffle=True, collate_fn=train_dataset.collate_fn)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, num_workers=1, collate_fn=val_dataset.collate_fn)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, num_workers=1, collate_fn=test_dataset.collate_fn)
        print(config["output_size"])
        # Create and initialize model
        model = config["model"](config).to(device)
        
        #### training optimiser parameters fror apple
        #learning_rate = 0.005 
        #optimizer = optim.Adam(model.parameters(), lr=learning_rate)

        #### training optimiser parameters fror harma_2023 VRB model 
        learning_rate = 0.01 # From the paper
        optimizer = optim.Adam(model.parameters())       
        
        best_val_loss = float('inf')
        best_val_loss_flat = float('inf')
        early_stopping_counter = 0
        # To accumulate metrics across folds for each epoch
        train_acc = []
        val_acc = []
        test_acc = []
        for epoch in range(epochs):

            model.train()
            train_loss = 0.0
            progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")
            
            for batch_d, batch_lbs in progress_bar:
                optimizer.zero_grad()
                input_values = batch_d.to(device)
                batch_lbs = batch_lbs.to(device)
                outputs = model(input_values)
                loss = correlation_coefficient_loss(outputs, batch_lbs)
                loss.backward()
                optimizer.step()
                train_loss += loss.item()
                progress_bar.set_postfix({'train_loss': f'{train_loss/(progress_bar.n+1):.4f}'})


            train_loss /= len(train_loader)

            # Combined validation loop
            model.eval()
            val_loss = 0.0
            val_pred = []
            with torch.no_grad():
                for batch_d, batch_lbs in val_loader:
                    input_values = batch_d.to(device)
                    batch_lbs = batch_lbs.to(device)
                    
                    outputs = model(input_values)
                    loss = correlation_coefficient_loss(outputs, batch_lbs)
                    val_loss += loss.item()
                    val_pred.extend(outputs.cpu().numpy())

            val_loss /= len(val_loader)

            # Calculate validation metrics
            val_pred = np.array(val_pred).reshape(val_timesteps.shape)
            val_ground_truth = _get_ground_truth_labels([all_dict[i] for i in val_index], all_labels)
            val_pred_flat = unsplit_data_ogsize(val_pred, window_size, step_sequence, 25, val_ground_truth.shape[-1])
            val_prc_coef = _calculate_flattened_accuracy(val_pred_flat, val_ground_truth)
            
            # Accumulate metrics for this fold and epoch
            train_acc.append(1- train_loss)
            val_acc.append(1- val_loss)

            # Log metrics
            writer.add_scalar(f"Loss/train_fold_{fold + 1}", train_loss, epoch)
            writer.add_scalar(f"Loss/val_fold_{fold + 1}", val_loss, epoch)
            writer.add_scalar(f"Pearson/val_fold_{fold + 1}", val_prc_coef, epoch)

            print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Pearson: {val_prc_coef:.4f}")

            # Check if validation loss improved
            if val_loss < best_val_loss:
                print(f"Validation loss improved from {best_val_loss:.4f} to {val_loss:.4f}. Saving best model...")
                best_val_loss = val_loss
                best_val_loss_flat = val_prc_coef
                early_stopping_counter = 0

                # Save the best model
                torch.save(model.state_dict(), best_model_path)
            else:
                early_stopping_counter += 1
                print(f"Validation loss did not improve for {early_stopping_counter} epochs.")
                #model.load_state_dict(torch.load(best_model_path))


            # Early stopping
            if early_stopping_counter >= early_stopping_patience:
                print(f"Early stopping triggered at epoch {epoch + 1}. Loading best model.")
                # Load the best model's weights
                model.load_state_dict(torch.load(best_model_path))
                break

        test_pred = []
        test_loss = 0.0
        with torch.no_grad():
            for batch_d, batch_lbs in test_loader:
                input_values = batch_d.to(device)
                batch_lbs = batch_lbs.to(device)
                
                outputs = model(input_values)
                loss = correlation_coefficient_loss(outputs, batch_lbs)
                test_loss += loss.item()
                test_pred.extend(outputs.cpu().numpy())

        test_loss /= len(test_loader)
        test_pred = np.array(test_pred).reshape(prepared_test_labels_timesteps.shape)
        test_ground_truth = _get_ground_truth_labels(list(test_dict.values()), test_labels)
        test_pred_flat = unsplit_data_ogsize(test_pred, window_size, step_sequence, 25, test_ground_truth.shape[-1])
        test_prc_coef = _calculate_flattened_accuracy(test_pred_flat, test_ground_truth)

        print(f"Fold {fold + 1}:")
        print(f"  Validation Pearson Coefficient  acc: {1- best_val_loss}")
        print(f"  Validation Pearson Coefficient flat acc: {best_val_loss_flat}")
        print(f"  Test acc: {1- test_loss}")
        print(f"  Test Pearson Coefficient acc(flattened): {test_prc_coef}")

        fold_metrics.append({
            'Fold': fold + 1,
            'val_prc_acc': 1- best_val_loss,
            'val_prc_acc_flat': best_val_loss_flat,
            'test_acc': 1- test_loss,
            'test_prc_flat': test_prc_coef
        })


                # Log fold-specific metrics as tables
        fold_table = f"| Fold | Val Pearson Acc | Val Pearson Flat | Test Acc | Test Pearson Flat |\n" \
                     f"|------|-----------------|------------------|----------|-------------------|\n" \
                     f"| {fold + 1} | {1 - val_loss:.4f} | {val_prc_coef:.4f} | {1 - test_loss:.4f} | {test_prc_coef:.4f} |\n"
        writer.add_text(f"Fold_{fold + 1}_Metrics", fold_table)
        # Accumulate fold metrics across all folds
        train_acc_epoch.append(train_acc)
        val_acc_epoch.append(train_acc)


    
        # After all folds, compute and log the average metrics per epoch across all folds
    for epoch in range(epochs):
        avg_train_loss = np.mean([fold_losses[epoch] for fold_losses in train_acc_epoch if len(fold_losses) > epoch])
        avg_val_loss = np.mean([fold_losses[epoch] for fold_losses in val_acc_epoch if len(fold_losses) > epoch])

        # Log the averaged metrics for the epoch across all folds
        writer.add_scalar("Average_acc/train", avg_train_loss, epoch)
        writer.add_scalar("Average_acc/val", avg_val_loss, epoch)
            

    # Calculate average metrics
    avg_metrics = {key: np.mean([fold[key] for fold in fold_metrics if key != 'Fold']) for key in fold_metrics[0].keys() if key != 'Fold'}
        # Log the final average table
    avg_table = "| Fold | Val Pearson Acc | Val Pearson Flat | Test Acc | Test Pearson Flat |\n" \
                "|------|-----------------|------------------|----------|-------------------|\n" \
                f"| Average | {avg_metrics['val_prc_acc']:.4f} | {avg_metrics['val_prc_acc_flat']:.4f} | {avg_metrics['test_acc']:.4f} | {avg_metrics['test_prc_flat']:.4f} |\n"
    writer.add_text("Average_Metrics", avg_table)
    # Add average metrics to results
    avg_metrics['Fold'] = 'Average'
    fold_metrics.append(avg_metrics)

    # save averga date to CSV
    results_df = pd.DataFrame(fold_metrics)
    csv_path = os.path.join(run_dir, 'fold_results.csv')
    results_df.to_csv(csv_path, index=False)
    
    # Save fold indices CSV
    fold_indices_df.to_csv(os.path.join(run_dir, 'fold_indices.csv'), index=False)
    
    writer.close()


    print("\nTraining completed.")
    print("Average metrics across all folds:")
    for key, value in avg_metrics.items():
        print(f"  {key}: {value}")


if __name__ == "__main__":
    ## Path to data
    #path = "/home/glenn/Downloads/"
    path = "../DATA/"


    # Model parameters
    model_config = {
        "VRBModel": {
            "model" : VRBModel,
            "model_name": "facebook/hubert-large-ls960-ft",
            "hidden_units": 64,
            "n_gru": 3,
            "output_size": None  # Will be set dynamically
        },
        "Wav2Vec2ConvLSTMModel": {
            "model" : Wav2Vec2ConvLSTMModel,
            "model_name": "facebook/wav2vec2-base",
            "hidden_units": 128,
            "n_lstm": 2,
            "output_size": None  # Will be set dynamically
        },
            "RespBertCNNModelV2": {
            'model' : RespBertCNNModelV2,
            "model_name": "microsoft/wavlm-large",
            "hidden_units": 128,
            "output_size": None  
        }
    }
    

    
    # Train and data parameters
    epochs = 60
    batch_size = 13
    window_size = 30
    step_size = 6
    data_parts = 4 # aka folds
    early_stopping_patience = 10
    
    config = model_config["RespBertCNNModelV2"]
    

    ## same wav2vec2 base model and pipeline used in the paper
    #processor = AutoProcessor.from_pretrained(config["model_name"])
    processor = Wav2Vec2FeatureExtractor.from_pretrained(config["model_name"])

    train(
        path_to_data=path+"ComParE2020_Breathing/wav/",
        path_to_labels=path+"ComParE2020_Breathing/lab/",
        window_size=window_size,
        batch_size=batch_size,
        config = config,
        step_size=step_size,
        data_parts= data_parts ,
        early_stopping_patience= early_stopping_patience,
        epochs= epochs,
        processor = processor
    )

Fold 1/4
(864, 480000)
750


Epoch 1/60:   0%|          | 0/67 [00:00<?, ?it/s]/home/gdwildt/.conda/envs/thesis/lib/python3.11/site-packages/torch/nn/functional.py:5193: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
Epoch 1/60: 100%|██████████| 67/67 [02:09<00:00,  1.93s/it, train_loss=0.8820]


Epoch 1/60 - Train Loss: 0.8820, Val Loss: 0.9271, Val Pearson: 0.1845
Validation loss improved from inf to 0.9271. Saving best model...


Epoch 2/60: 100%|██████████| 67/67 [02:09<00:00,  1.93s/it, train_loss=0.4037]


Epoch 2/60 - Train Loss: 0.4037, Val Loss: 0.4840, Val Pearson: 0.7195
Validation loss improved from 0.9271 to 0.4840. Saving best model...


...

Epoch 8/60: 100%|██████████| 70/70 [02:14<00:00,  1.91s/it, train_loss=0.0632]


Epoch 8/60 - Train Loss: 0.0632, Val Loss: 0.2542, Val Pearson: 0.7544
Validation loss did not improve for 3 epochs.


Epoch 9/60: 100%|██████████| 70/70 [02:14<00:00,  1.91s/it, train_loss=0.0512]


Epoch 9/60 - Train Loss: 0.0512, Val Loss: 0.2548, Val Pearson: 0.7558
Validation loss did not improve for 4 epochs.


Epoch 10/60: 100%|██████████| 70/70 [02:14<00:00,  1.92s/it, train_loss=0.0471]


In [ ]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import KFold
from datetime import datetime
#from torch.utils.tensorboard import SummaryWriter
import torchaudio
from typing import List, Tuple
from pt_utils import *
from pt_dataset import *
from pt_models import *
from pt_utils import *
from tensorboardX import SummaryWriter
from transformers import get_cosine_schedule_with_warmup


def create_run_directory():
    base_dir = "pt_runs_SOTA"
    timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
    run_dir = os.path.join(base_dir, timestamp)
    os.makedirs(run_dir, exist_ok=True)
    return run_dir

def _calculate_flattened_accuracy(average, ground_truth_labels):
    s_acc = 0
    for b in range(len(ground_truth_labels)):
        s, _ = scipy.stats.pearsonr(average[b], ground_truth_labels[b])
        s_acc += s
    return s_acc / len(ground_truth_labels)

def _choose_real_labs_only_with_filenames(labels, filenames):
    return labels[labels['filename'].isin(filenames)]

def _get_ground_truth_labels(ground_truth_names, labels):
    ground_truth_labels = []
    for batch_name in ground_truth_names:
        ground_truth_label = _choose_real_labs_only_with_filenames(labels, [batch_name])
        ground_truth_labels.append(ground_truth_label)
    return np.array(ground_truth_labels)[:, :, -1].astype(np.float32)

def train(path_to_data, path_to_labels, window_size=16, step_size=6, data_parts=4, epochs=100, batch_size=10, early_stopping_patience=20, config = None, processor = None):
    run_dir = create_run_directory()
    log_dir = os.path.join(run_dir, "logs")
    os.makedirs(log_dir, exist_ok=True)

    # Parameters
    length_sequence = window_size 
    step_sequence = step_size

    # Load and prepare data
    train_data, train_labels, train_dict, frame_rate = load_data(path_to_data, path_to_labels, 'train')
    devel_data, devel_labels, devel_dict, frame_rate = load_data(path_to_data, path_to_labels, 'devel')
    test_data, test_labels, test_dict, frame_rate = load_data(path_to_data, path_to_labels, 'test')

    # Combine train and devel data
    all_data = np.concatenate((train_data, devel_data), axis=0)
    all_labels = pd.concat([train_labels, devel_labels])
    all_dict = np.concatenate((list(train_dict.values()), list(devel_dict.values())), axis=0)

    # Prepare data
    prepared_data, prepared_labels, prepared_labels_timesteps = prepare_data(all_data, all_labels, all_dict, frame_rate, length_sequence * 16000, step_sequence * 16000)
    prepared_test_data, prepared_test_labels, prepared_test_labels_timesteps = prepare_data(test_data, test_labels, test_dict, frame_rate, length_sequence * 16000, step_sequence * 16000)

    # Create CSV file for storing fold indices
    fold_indices_df = pd.DataFrame(columns=['Fold', 'Train_Indices', 'Val_Indices'])

    # Cross-validation
    kf = KFold(n_splits=data_parts)
    fold_metrics = []
    # To accumulate metrics across folds for each epoch
    train_acc_epoch = []
    val_acc_epoch = []
    test_acc_epoch = []

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    
    config["output_size"] = prepared_labels.shape[-1]
    writer = SummaryWriter(log_dir=os.path.join(log_dir,config["model_name"]))

    for fold, (train_index, val_index) in enumerate(kf.split(prepared_data)):
        print(f"Fold {fold + 1}/{data_parts}")
        best_model_path = f"{run_dir}/best_model_fold{fold+1}"
        # Save fold indices
        fold_indices_df = fold_indices_df._append({
            'Fold': fold + 1,
            'Train_Indices': train_index.tolist(),
            'Val_Indices': val_index.tolist()
        }, ignore_index=True)

        # Split data
        train_d, val_d = prepared_data[train_index], prepared_data[val_index]
        train_lbs, val_lbs = prepared_labels[train_index], prepared_labels[val_index]
        train_timesteps, val_timesteps = prepared_labels_timesteps[train_index], prepared_labels_timesteps[val_index]
        
        # Reshape data
        train_d, train_lbs = reshaping_data_for_model(train_d, train_lbs)
        val_d, val_lbs = reshaping_data_for_model(val_d, val_lbs)
        test_d, test_lbs = reshaping_data_for_model(prepared_test_data, prepared_test_labels)
        
        print(train_d.shape)

        # Create datasets
        #train_dataset = BreathingDataset(train_d, train_lbs, processor, window_size, step_sequence, augment=True)
        train_dataset = BreathingDataset(train_d, train_lbs, processor,window_size, step_sequence)
        val_dataset = BreathingDataset(val_d, val_lbs, processor, window_size, step_sequence)
        test_dataset = BreathingDataset(test_d, test_lbs, processor, window_size, step_sequence)

        # Create DataLoaders
        train_loader = DataLoader(train_dataset, batch_size=batch_size,num_workers=2, shuffle=True, collate_fn=train_dataset.collate_fn)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, num_workers=1, collate_fn=val_dataset.collate_fn)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, num_workers=1, collate_fn=test_dataset.collate_fn)
        print(config["output_size"])
        # Create and initialize model
        model = config["model"](config).to(device)
        
        #### training optimiser parameters fror apple
        #learning_rate = 0.005 
        #optimizer = optim.Adam(model.parameters(), lr=learning_rate)

        #### training optimiser parameters fror harma_2023 VRB model 
        learning_rate = 0.01 # From the paper
        optimizer = optim.Adam(model.parameters())       
        
        best_val_loss = float('inf')
        best_val_loss_flat = float('inf')
        early_stopping_counter = 0
        # To accumulate metrics across folds for each epoch
        train_acc = []
        val_acc = []
        test_acc = []
        for epoch in range(epochs):

            model.train()
            train_loss = 0.0
            progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")
            
            for batch_d, batch_lbs in progress_bar:
                optimizer.zero_grad()
                input_values = batch_d.to(device)
                batch_lbs = batch_lbs.to(device)
                outputs = model(input_values)
                loss = correlation_coefficient_loss(outputs, batch_lbs)
                loss.backward()
                optimizer.step()
                train_loss += loss.item()
                progress_bar.set_postfix({'train_loss': f'{train_loss/(progress_bar.n+1):.4f}'})


            train_loss /= len(train_loader)

            # Combined validation loop
            model.eval()
            val_loss = 0.0
            val_pred = []
            with torch.no_grad():
                for batch_d, batch_lbs in val_loader:
                    input_values = batch_d.to(device)
                    batch_lbs = batch_lbs.to(device)
                    
                    outputs = model(input_values)
                    loss = correlation_coefficient_loss(outputs, batch_lbs)
                    val_loss += loss.item()
                    val_pred.extend(outputs.cpu().numpy())

            val_loss /= len(val_loader)

            # Calculate validation metrics
            val_pred = np.array(val_pred).reshape(val_timesteps.shape)
            val_ground_truth = _get_ground_truth_labels([all_dict[i] for i in val_index], all_labels)
            val_pred_flat = unsplit_data_ogsize(val_pred, window_size, step_sequence, 25, val_ground_truth.shape[-1])
            val_prc_coef = _calculate_flattened_accuracy(val_pred_flat, val_ground_truth)
            
            # Accumulate metrics for this fold and epoch
            train_acc.append(1- train_loss)
            val_acc.append(1- val_loss)

            # Log metrics
            writer.add_scalar(f"Loss/train_fold_{fold + 1}", train_loss, epoch)
            writer.add_scalar(f"Loss/val_fold_{fold + 1}", val_loss, epoch)
            writer.add_scalar(f"Pearson/val_fold_{fold + 1}", val_prc_coef, epoch)

            print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Pearson: {val_prc_coef:.4f}")

            # Check if validation loss improved
            if val_loss < best_val_loss:
                print(f"Validation loss improved from {best_val_loss:.4f} to {val_loss:.4f}. Saving best model...")
                best_val_loss = val_loss
                best_val_loss_flat = val_prc_coef
                early_stopping_counter = 0

                # Save the best model
                torch.save(model.state_dict(), best_model_path)
            else:
                early_stopping_counter += 1
                print(f"Validation loss did not improve for {early_stopping_counter} epochs.")
                #model.load_state_dict(torch.load(best_model_path))


            # Early stopping
            if early_stopping_counter >= early_stopping_patience:
                print(f"Early stopping triggered at epoch {epoch + 1}. Loading best model.")
                # Load the best model's weights
                model.load_state_dict(torch.load(best_model_path))
                break

        test_pred = []
        test_loss = 0.0
        with torch.no_grad():
            for batch_d, batch_lbs in test_loader:
                input_values = batch_d.to(device)
                batch_lbs = batch_lbs.to(device)
                
                outputs = model(input_values)
                loss = correlation_coefficient_loss(outputs, batch_lbs)
                test_loss += loss.item()
                test_pred.extend(outputs.cpu().numpy())

        test_loss /= len(test_loader)
        test_pred = np.array(test_pred).reshape(prepared_test_labels_timesteps.shape)
        test_ground_truth = _get_ground_truth_labels(list(test_dict.values()), test_labels)
        test_pred_flat = unsplit_data_ogsize(test_pred, window_size, step_sequence, 25, test_ground_truth.shape[-1])
        test_prc_coef = _calculate_flattened_accuracy(test_pred_flat, test_ground_truth)

        print(f"Fold {fold + 1}:")
        print(f"  Validation Pearson Coefficient  acc: {1- best_val_loss}")
        print(f"  Validation Pearson Coefficient flat acc: {best_val_loss_flat}")
        print(f"  Test acc: {1- test_loss}")
        print(f"  Test Pearson Coefficient acc(flattened): {test_prc_coef}")

        fold_metrics.append({
            'Fold': fold + 1,
            'val_prc_acc': 1- best_val_loss,
            'val_prc_acc_flat': best_val_loss_flat,
            'test_acc': 1- test_loss,
            'test_prc_flat': test_prc_coef
        })


                # Log fold-specific metrics as tables
        fold_table = f"| Fold | Val Pearson Acc | Val Pearson Flat | Test Acc | Test Pearson Flat |\n" \
                     f"|------|-----------------|------------------|----------|-------------------|\n" \
                     f"| {fold + 1} | {1 - val_loss:.4f} | {val_prc_coef:.4f} | {1 - test_loss:.4f} | {test_prc_coef:.4f} |\n"
        writer.add_text(f"Fold_{fold + 1}_Metrics", fold_table)
        # Accumulate fold metrics across all folds
        train_acc_epoch.append(train_acc)
        val_acc_epoch.append(train_acc)


    
        # After all folds, compute and log the average metrics per epoch across all folds
    for epoch in range(epochs):
        avg_train_loss = np.mean([fold_losses[epoch] for fold_losses in train_acc_epoch if len(fold_losses) > epoch])
        avg_val_loss = np.mean([fold_losses[epoch] for fold_losses in val_acc_epoch if len(fold_losses) > epoch])

        # Log the averaged metrics for the epoch across all folds
        writer.add_scalar("Average_acc/train", avg_train_loss, epoch)
        writer.add_scalar("Average_acc/val", avg_val_loss, epoch)
            

    # Calculate average metrics
    avg_metrics = {key: np.mean([fold[key] for fold in fold_metrics if key != 'Fold']) for key in fold_metrics[0].keys() if key != 'Fold'}
        # Log the final average table
    avg_table = "| Fold | Val Pearson Acc | Val Pearson Flat | Test Acc | Test Pearson Flat |\n" \
                "|------|-----------------|------------------|----------|-------------------|\n" \
                f"| Average | {avg_metrics['val_prc_acc']:.4f} | {avg_metrics['val_prc_acc_flat']:.4f} | {avg_metrics['test_acc']:.4f} | {avg_metrics['test_prc_flat']:.4f} |\n"
    writer.add_text("Average_Metrics", avg_table)
    # Add average metrics to results
    avg_metrics['Fold'] = 'Average'
    fold_metrics.append(avg_metrics)

    # save averga date to CSV
    results_df = pd.DataFrame(fold_metrics)
    csv_path = os.path.join(run_dir, 'fold_results.csv')
    results_df.to_csv(csv_path, index=False)
    
    # Save fold indices CSV
    fold_indices_df.to_csv(os.path.join(run_dir, 'fold_indices.csv'), index=False)
    
    writer.close()


    print("\nTraining completed.")
    print("Average metrics across all folds:")
    for key, value in avg_metrics.items():
        print(f"  {key}: {value}")


if __name__ == "__main__":
    ## Path to data
    #path = "/home/glenn/Downloads/"
    path = "../DATA/"


    # Model parameters
    model_config = {
        "VRBModel": {
            "model" : VRBModel,
            "model_name": "facebook/hubert-large-ll60k",
            "hidden_units": 64,
            "n_gru": 3,
            "output_size": None  # Will be set dynamically
        },
        "Wav2Vec2ConvLSTMModel": {
            "model" : Wav2Vec2ConvLSTMModel,
            "model_name": "facebook/wav2vec2-base",
            "hidden_units": 128,
            "n_lstm": 2,
            "output_size": None  # Will be set dynamically
        },
            "RespBertCNNModelV2": {
            'model' : RespBertCNNModelV2,
            "model_name": "microsoft/wavlm-large",
            "hidden_units": 128,
            "output_size": None  
        },
            "RespBertLSTMModelV2": {
            'model': RespBertLSTMModelV2,
            "model_name": "microsoft/wavlm-large",
            "hidden_units": 128,
            "n_lstm": 3,
            "output_size": None  
        }
    }
    

    
    # Train and data parameters
    epochs = 60
    batch_size = 13
    window_size = 30
    step_size = 6
    data_parts = 4 # aka folds
    early_stopping_patience = 10
    
    config = model_config["RespBertCNNModelV2"]
    

    ## same wav2vec2 base model and pipeline used in the paper
    #processor = AutoProcessor.from_pretrained(config["model_name"])
    processor = Wav2Vec2FeatureExtractor.from_pretrained(config["model_name"])

    train(
        path_to_data=path+"ComParE2020_Breathing/wav/",
        path_to_labels=path+"ComParE2020_Breathing/lab/",
        window_size=window_size,
        batch_size=batch_size,
        config = config,
        step_size=step_size,
        data_parts= data_parts ,
        early_stopping_patience= early_stopping_patience,
        epochs= epochs,
        processor = processor
    )

Fold 1/4
(864, 480000)
750


Epoch 1/60:   0%|          | 0/67 [00:00<?, ?it/s]/home/gdwildt/.conda/envs/thesis/lib/python3.11/site-packages/torch/nn/functional.py:5193: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
Epoch 1/60: 100%|██████████| 67/67 [02:09<00:00,  1.94s/it, train_loss=0.8485]


Epoch 1/60 - Train Loss: 0.8485, Val Loss: 0.5918, Val Pearson: 0.4822
Validation loss improved from inf to 0.5918. Saving best model...


Epoch 2/60: 100%|██████████| 67/67 [02:08<00:00,  1.91s/it, train_loss=0.4059]


Epoch 2/60 - Train Loss: 0.4059, Val Loss: 0.2168, Val Pearson: 0.7915
Validation loss improved from 0.5918 to 0.2168. Saving best model...


...

Epoch 18/60 - Train Loss: 0.0189, Val Loss: 0.3197, Val Pearson: 0.6954
Validation loss did not improve for 9 epochs.


Epoch 19/60: 100%|██████████| 70/70 [02:14<00:00,  1.92s/it, train_loss=0.0186]


Epoch 19/60 - Train Loss: 0.0186, Val Loss: 0.3251, Val Pearson: 0.6928
Validation loss did not improve for 10 epochs.
Early stopping triggered at epoch 19. Loading best model.


<ipython-input-1-5f409c942542>:208: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(best_model_path))
/home/gdwildt/.conda/envs/thesis/lib/pyt

Fold 4:
  Validation Pearson Coefficient  acc: 0.6979680061340332
  Validation Pearson Coefficient flat acc: 0.7109361224983757
  Test acc: 0.806722899278005
  Test Pearson Coefficient acc(flattened): 0.8146534796330512

Training completed.
Average metrics across all folds:
  val_prc_acc: 0.7420595638389174
  val_prc_acc_flat: 0.7515797918452681
  test_acc: 0.8080251759952969
  test_prc_flat: 0.8173247033530061
  Fold: Average


In [ ]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import KFold
from datetime import datetime
#from torch.utils.tensorboard import SummaryWriter
import torchaudio
from typing import List, Tuple
from pt_utils import *
from pt_dataset import *
from pt_models import *
from pt_utils import *
from tensorboardX import SummaryWriter
from transformers import get_cosine_schedule_with_warmup


def create_run_directory():
    base_dir = "pt_runs_SOTA"
    timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
    run_dir = os.path.join(base_dir, timestamp)
    os.makedirs(run_dir, exist_ok=True)
    return run_dir

def _calculate_flattened_accuracy(average, ground_truth_labels):
    s_acc = 0
    for b in range(len(ground_truth_labels)):
        s, _ = scipy.stats.pearsonr(average[b], ground_truth_labels[b])
        s_acc += s
    return s_acc / len(ground_truth_labels)

def _choose_real_labs_only_with_filenames(labels, filenames):
    return labels[labels['filename'].isin(filenames)]

def _get_ground_truth_labels(ground_truth_names, labels):
    ground_truth_labels = []
    for batch_name in ground_truth_names:
        ground_truth_label = _choose_real_labs_only_with_filenames(labels, [batch_name])
        ground_truth_labels.append(ground_truth_label)
    return np.array(ground_truth_labels)[:, :, -1].astype(np.float32)

def train(path_to_data, path_to_labels, window_size=16, step_size=6, data_parts=4, epochs=100, batch_size=10, early_stopping_patience=20, config = None, processor = None):
    run_dir = create_run_directory()
    log_dir = os.path.join(run_dir, "logs")
    os.makedirs(log_dir, exist_ok=True)

    # Parameters
    length_sequence = window_size 
    step_sequence = step_size

    # Load and prepare data
    train_data, train_labels, train_dict, frame_rate = load_data(path_to_data, path_to_labels, 'train')
    devel_data, devel_labels, devel_dict, frame_rate = load_data(path_to_data, path_to_labels, 'devel')
    test_data, test_labels, test_dict, frame_rate = load_data(path_to_data, path_to_labels, 'test')

    # Combine train and devel data
    all_data = np.concatenate((train_data, devel_data), axis=0)
    all_labels = pd.concat([train_labels, devel_labels])
    all_dict = np.concatenate((list(train_dict.values()), list(devel_dict.values())), axis=0)

    # Prepare data
    prepared_data, prepared_labels, prepared_labels_timesteps = prepare_data(all_data, all_labels, all_dict, frame_rate, length_sequence * 16000, step_sequence * 16000)
    prepared_test_data, prepared_test_labels, prepared_test_labels_timesteps = prepare_data(test_data, test_labels, test_dict, frame_rate, length_sequence * 16000, step_sequence * 16000)

    # Create CSV file for storing fold indices
    fold_indices_df = pd.DataFrame(columns=['Fold', 'Train_Indices', 'Val_Indices'])

    # Cross-validation
    kf = KFold(n_splits=data_parts)
    fold_metrics = []
    # To accumulate metrics across folds for each epoch
    train_acc_epoch = []
    val_acc_epoch = []
    test_acc_epoch = []

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    
    config["output_size"] = prepared_labels.shape[-1]
    writer = SummaryWriter(log_dir=os.path.join(log_dir,config["model_name"]))

    for fold, (train_index, val_index) in enumerate(kf.split(prepared_data)):
        print(f"Fold {fold + 1}/{data_parts}")
        best_model_path = f"{run_dir}/best_model_fold{fold+1}"
        # Save fold indices
        fold_indices_df = fold_indices_df._append({
            'Fold': fold + 1,
            'Train_Indices': train_index.tolist(),
            'Val_Indices': val_index.tolist()
        }, ignore_index=True)

        # Split data
        train_d, val_d = prepared_data[train_index], prepared_data[val_index]
        train_lbs, val_lbs = prepared_labels[train_index], prepared_labels[val_index]
        train_timesteps, val_timesteps = prepared_labels_timesteps[train_index], prepared_labels_timesteps[val_index]
        
        # Reshape data
        train_d, train_lbs = reshaping_data_for_model(train_d, train_lbs)
        val_d, val_lbs = reshaping_data_for_model(val_d, val_lbs)
        test_d, test_lbs = reshaping_data_for_model(prepared_test_data, prepared_test_labels)
        
        print(train_d.shape)

        # Create datasets
        #train_dataset = BreathingDataset(train_d, train_lbs, processor, window_size, step_sequence, augment=True)
        train_dataset = BreathingDataset(train_d, train_lbs, processor,window_size, step_sequence)
        val_dataset = BreathingDataset(val_d, val_lbs, processor, window_size, step_sequence)
        test_dataset = BreathingDataset(test_d, test_lbs, processor, window_size, step_sequence)

        # Create DataLoaders
        train_loader = DataLoader(train_dataset, batch_size=batch_size,num_workers=2, shuffle=True, collate_fn=train_dataset.collate_fn)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, num_workers=1, collate_fn=val_dataset.collate_fn)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, num_workers=1, collate_fn=test_dataset.collate_fn)
        print(config["output_size"])
        # Create and initialize model
        model = config["model"](config).to(device)
        
        #### training optimiser parameters fror apple
        #learning_rate = 0.005 
        #optimizer = optim.Adam(model.parameters(), lr=learning_rate)

        #### training optimiser parameters fror harma_2023 VRB model 
        learning_rate = 0.01 # From the paper
        optimizer = optim.Adam(model.parameters())       
        
        best_val_loss = float('inf')
        best_val_loss_flat = float('inf')
        early_stopping_counter = 0
        # To accumulate metrics across folds for each epoch
        train_acc = []
        val_acc = []
        test_acc = []
        for epoch in range(epochs):

            model.train()
            train_loss = 0.0
            progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")
            
            for batch_d, batch_lbs in progress_bar:
                optimizer.zero_grad()
                input_values = batch_d.to(device)
                batch_lbs = batch_lbs.to(device)
                outputs = model(input_values)
                loss = correlation_coefficient_loss(outputs, batch_lbs)
                loss.backward()
                optimizer.step()
                train_loss += loss.item()
                progress_bar.set_postfix({'train_loss': f'{train_loss/(progress_bar.n+1):.4f}'})


            train_loss /= len(train_loader)

            # Combined validation loop
            model.eval()
            val_loss = 0.0
            val_pred = []
            with torch.no_grad():
                for batch_d, batch_lbs in val_loader:
                    input_values = batch_d.to(device)
                    batch_lbs = batch_lbs.to(device)
                    
                    outputs = model(input_values)
                    loss = correlation_coefficient_loss(outputs, batch_lbs)
                    val_loss += loss.item()
                    val_pred.extend(outputs.cpu().numpy())

            val_loss /= len(val_loader)

            # Calculate validation metrics
            val_pred = np.array(val_pred).reshape(val_timesteps.shape)
            val_ground_truth = _get_ground_truth_labels([all_dict[i] for i in val_index], all_labels)
            val_pred_flat = unsplit_data_ogsize(val_pred, window_size, step_sequence, 25, val_ground_truth.shape[-1])
            val_prc_coef = _calculate_flattened_accuracy(val_pred_flat, val_ground_truth)
            
            # Accumulate metrics for this fold and epoch
            train_acc.append(1- train_loss)
            val_acc.append(1- val_loss)

            # Log metrics
            writer.add_scalar(f"Loss/train_fold_{fold + 1}", train_loss, epoch)
            writer.add_scalar(f"Loss/val_fold_{fold + 1}", val_loss, epoch)
            writer.add_scalar(f"Pearson/val_fold_{fold + 1}", val_prc_coef, epoch)

            print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Pearson: {val_prc_coef:.4f}")

            # Check if validation loss improved
            if val_loss < best_val_loss:
                print(f"Validation loss improved from {best_val_loss:.4f} to {val_loss:.4f}. Saving best model...")
                best_val_loss = val_loss
                best_val_loss_flat = val_prc_coef
                early_stopping_counter = 0

                # Save the best model
                torch.save(model.state_dict(), best_model_path)
            else:
                early_stopping_counter += 1
                print(f"Validation loss did not improve for {early_stopping_counter} epochs.")
                #model.load_state_dict(torch.load(best_model_path))


            # Early stopping
            if early_stopping_counter >= early_stopping_patience:
                print(f"Early stopping triggered at epoch {epoch + 1}. Loading best model.")
                # Load the best model's weights
                model.load_state_dict(torch.load(best_model_path))
                break

        test_pred = []
        test_loss = 0.0
        with torch.no_grad():
            for batch_d, batch_lbs in test_loader:
                input_values = batch_d.to(device)
                batch_lbs = batch_lbs.to(device)
                
                outputs = model(input_values)
                loss = correlation_coefficient_loss(outputs, batch_lbs)
                test_loss += loss.item()
                test_pred.extend(outputs.cpu().numpy())

        test_loss /= len(test_loader)
        test_pred = np.array(test_pred).reshape(prepared_test_labels_timesteps.shape)
        test_ground_truth = _get_ground_truth_labels(list(test_dict.values()), test_labels)
        test_pred_flat = unsplit_data_ogsize(test_pred, window_size, step_sequence, 25, test_ground_truth.shape[-1])
        test_prc_coef = _calculate_flattened_accuracy(test_pred_flat, test_ground_truth)

        print(f"Fold {fold + 1}:")
        print(f"  Validation Pearson Coefficient  acc: {1- best_val_loss}")
        print(f"  Validation Pearson Coefficient flat acc: {best_val_loss_flat}")
        print(f"  Test acc: {1- test_loss}")
        print(f"  Test Pearson Coefficient acc(flattened): {test_prc_coef}")

        fold_metrics.append({
            'Fold': fold + 1,
            'val_prc_acc': 1- best_val_loss,
            'val_prc_acc_flat': best_val_loss_flat,
            'test_acc': 1- test_loss,
            'test_prc_flat': test_prc_coef
        })


                # Log fold-specific metrics as tables
        fold_table = f"| Fold | Val Pearson Acc | Val Pearson Flat | Test Acc | Test Pearson Flat |\n" \
                     f"|------|-----------------|------------------|----------|-------------------|\n" \
                     f"| {fold + 1} | {1 - val_loss:.4f} | {val_prc_coef:.4f} | {1 - test_loss:.4f} | {test_prc_coef:.4f} |\n"
        writer.add_text(f"Fold_{fold + 1}_Metrics", fold_table)
        # Accumulate fold metrics across all folds
        train_acc_epoch.append(train_acc)
        val_acc_epoch.append(train_acc)


    
        # After all folds, compute and log the average metrics per epoch across all folds
    for epoch in range(epochs):
        avg_train_loss = np.mean([fold_losses[epoch] for fold_losses in train_acc_epoch if len(fold_losses) > epoch])
        avg_val_loss = np.mean([fold_losses[epoch] for fold_losses in val_acc_epoch if len(fold_losses) > epoch])

        # Log the averaged metrics for the epoch across all folds
        writer.add_scalar("Average_acc/train", avg_train_loss, epoch)
        writer.add_scalar("Average_acc/val", avg_val_loss, epoch)
            

    # Calculate average metrics
    avg_metrics = {key: np.mean([fold[key] for fold in fold_metrics if key != 'Fold']) for key in fold_metrics[0].keys() if key != 'Fold'}
        # Log the final average table
    avg_table = "| Fold | Val Pearson Acc | Val Pearson Flat | Test Acc | Test Pearson Flat |\n" \
                "|------|-----------------|------------------|----------|-------------------|\n" \
                f"| Average | {avg_metrics['val_prc_acc']:.4f} | {avg_metrics['val_prc_acc_flat']:.4f} | {avg_metrics['test_acc']:.4f} | {avg_metrics['test_prc_flat']:.4f} |\n"
    writer.add_text("Average_Metrics", avg_table)
    # Add average metrics to results
    avg_metrics['Fold'] = 'Average'
    fold_metrics.append(avg_metrics)

    # save averga date to CSV
    results_df = pd.DataFrame(fold_metrics)
    csv_path = os.path.join(run_dir, 'fold_results.csv')
    results_df.to_csv(csv_path, index=False)
    
    # Save fold indices CSV
    fold_indices_df.to_csv(os.path.join(run_dir, 'fold_indices.csv'), index=False)
    
    writer.close()


    print("\nTraining completed.")
    print("Average metrics across all folds:")
    for key, value in avg_metrics.items():
        print(f"  {key}: {value}")


if __name__ == "__main__":
    ## Path to data
    #path = "/home/glenn/Downloads/"
    path = "../DATA/"


    # Model parameters
    model_config = {
        "VRBModel": {
            "model" : VRBModel,
            "model_name": "facebook/hubert-large-ll60k",
            "hidden_units": 64,
            "n_gru": 3,
            "output_size": None  # Will be set dynamically
        },
        "Wav2Vec2ConvLSTMModel": {
            "model" : Wav2Vec2ConvLSTMModel,
            "model_name": "facebook/wav2vec2-base",
            "hidden_units": 128,
            "n_lstm": 2,
            "output_size": None  # Will be set dynamically
        },
            "RespBertCNNModelV2": {
            'model' : RespBertCNNModelV2,
            "model_name": "microsoft/wavlm-large",
            "hidden_units": 128,
            "output_size": None  
        },
            "RespBertLSTMModelV2": {
            'model': RespBertLSTMModelV2,
            "model_name": "microsoft/wavlm-large",
            "hidden_units": 128,
            "n_lstm": 3,
            "output_size": None  
        }
            ,
            "RespBertLSTMCNNTransformerModel": {
            'model': RespBertLSTMCNNTransformerModel,
            "model_name": "microsoft/wavlm-large",
            "hidden_units": 1024,
            "n_lstm": 2,
            "output_size": None  
        }
    }
    

    
    # Train and data parameters
    epochs = 60
    batch_size = 13
    window_size = 30
    step_size = 6
    data_parts = 4 # aka folds
    early_stopping_patience = 10
    
    config = model_config["RespBertCNNModelV2"]
    

    ## same wav2vec2 base model and pipeline used in the paper
    #processor = AutoProcessor.from_pretrained(config["model_name"])
    processor = Wav2Vec2FeatureExtractor.from_pretrained(config["model_name"])

    train(
        path_to_data=path+"ComParE2020_Breathing/wav/",
        path_to_labels=path+"ComParE2020_Breathing/lab/",
        window_size=window_size,
        batch_size=batch_size,
        config = config,
        step_size=step_size,
        data_parts= data_parts ,
        early_stopping_patience= early_stopping_patience,
        epochs= epochs,
        processor = processor
    )

Fold 1/4
(864, 480000)
750


Epoch 1/60:   0%|          | 0/67 [00:00<?, ?it/s]/home/gdwildt/.conda/envs/thesis/lib/python3.11/site-packages/torch/nn/functional.py:5193: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
Epoch 1/60:  33%|███▎      | 22/67 [00:44<01:27,  1.95s/it, train_loss=0.9990]

In [ ]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import KFold
from datetime import datetime
#from torch.utils.tensorboard import SummaryWriter
import torchaudio
from typing import List, Tuple
from pt_utils import *
from pt_dataset import *
from pt_models import *
from pt_utils import *
from tensorboardX import SummaryWriter
from transformers import get_cosine_schedule_with_warmup


def create_run_directory():
    base_dir = "pt_runs_SOTA"
    timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
    run_dir = os.path.join(base_dir, timestamp)
    os.makedirs(run_dir, exist_ok=True)
    return run_dir

def _calculate_flattened_accuracy(average, ground_truth_labels):
    s_acc = 0
    for b in range(len(ground_truth_labels)):
        s, _ = scipy.stats.pearsonr(average[b], ground_truth_labels[b])
        s_acc += s
    return s_acc / len(ground_truth_labels)

def _choose_real_labs_only_with_filenames(labels, filenames):
    return labels[labels['filename'].isin(filenames)]

def _get_ground_truth_labels(ground_truth_names, labels):
    ground_truth_labels = []
    for batch_name in ground_truth_names:
        ground_truth_label = _choose_real_labs_only_with_filenames(labels, [batch_name])
        ground_truth_labels.append(ground_truth_label)
    return np.array(ground_truth_labels)[:, :, -1].astype(np.float32)

def train(path_to_data, path_to_labels, window_size=16, step_size=6, data_parts=4, epochs=100, batch_size=10, early_stopping_patience=20, config = None, processor = None):
    run_dir = create_run_directory()
    log_dir = os.path.join(run_dir, "logs")
    os.makedirs(log_dir, exist_ok=True)

    # Parameters
    length_sequence = window_size 
    step_sequence = step_size

    # Load and prepare data
    train_data, train_labels, train_dict, frame_rate = load_data(path_to_data, path_to_labels, 'train')
    devel_data, devel_labels, devel_dict, frame_rate = load_data(path_to_data, path_to_labels, 'devel')
    test_data, test_labels, test_dict, frame_rate = load_data(path_to_data, path_to_labels, 'test')

    # Combine train and devel data
    all_data = np.concatenate((train_data, devel_data), axis=0)
    all_labels = pd.concat([train_labels, devel_labels])
    all_dict = np.concatenate((list(train_dict.values()), list(devel_dict.values())), axis=0)

    # Prepare data
    prepared_data, prepared_labels, prepared_labels_timesteps = prepare_data(all_data, all_labels, all_dict, frame_rate, length_sequence * 16000, step_sequence * 16000)
    prepared_test_data, prepared_test_labels, prepared_test_labels_timesteps = prepare_data(test_data, test_labels, test_dict, frame_rate, length_sequence * 16000, step_sequence * 16000)

    # Create CSV file for storing fold indices
    fold_indices_df = pd.DataFrame(columns=['Fold', 'Train_Indices', 'Val_Indices'])

    # Cross-validation
    kf = KFold(n_splits=data_parts)
    fold_metrics = []
    # To accumulate metrics across folds for each epoch
    train_acc_epoch = []
    val_acc_epoch = []
    test_acc_epoch = []

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    
    config["output_size"] = prepared_labels.shape[-1]
    writer = SummaryWriter(log_dir=os.path.join(log_dir,config["model_name"]))

    for fold, (train_index, val_index) in enumerate(kf.split(prepared_data)):
        print(f"Fold {fold + 1}/{data_parts}")
        best_model_path = f"{run_dir}/best_model_fold{fold+1}"
        # Save fold indices
        fold_indices_df = fold_indices_df._append({
            'Fold': fold + 1,
            'Train_Indices': train_index.tolist(),
            'Val_Indices': val_index.tolist()
        }, ignore_index=True)

        # Split data
        train_d, val_d = prepared_data[train_index], prepared_data[val_index]
        train_lbs, val_lbs = prepared_labels[train_index], prepared_labels[val_index]
        train_timesteps, val_timesteps = prepared_labels_timesteps[train_index], prepared_labels_timesteps[val_index]
        
        # Reshape data
        train_d, train_lbs = reshaping_data_for_model(train_d, train_lbs)
        val_d, val_lbs = reshaping_data_for_model(val_d, val_lbs)
        test_d, test_lbs = reshaping_data_for_model(prepared_test_data, prepared_test_labels)
        
        print(train_d.shape)

        # Create datasets
        #train_dataset = BreathingDataset(train_d, train_lbs, processor, window_size, step_sequence, augment=True)
        train_dataset = BreathingDataset(train_d, train_lbs, processor,window_size, step_sequence)
        val_dataset = BreathingDataset(val_d, val_lbs, processor, window_size, step_sequence)
        test_dataset = BreathingDataset(test_d, test_lbs, processor, window_size, step_sequence)

        # Create DataLoaders
        train_loader = DataLoader(train_dataset, batch_size=batch_size,num_workers=2, shuffle=True, collate_fn=train_dataset.collate_fn)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, num_workers=1, collate_fn=val_dataset.collate_fn)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, num_workers=1, collate_fn=test_dataset.collate_fn)
        print(config["output_size"])
        # Create and initialize model
        model = config["model"](config).to(device)
        
        #### training optimiser parameters fror apple
        #learning_rate = 0.005 
        #optimizer = optim.Adam(model.parameters(), lr=learning_rate)

        #### training optimiser parameters fror harma_2023 VRB model 
        learning_rate = 0.01 # From the paper
        optimizer = optim.Adam(model.parameters())       
        
        best_val_loss = float('inf')
        best_val_loss_flat = float('inf')
        early_stopping_counter = 0
        # To accumulate metrics across folds for each epoch
        train_acc = []
        val_acc = []
        test_acc = []
        for epoch in range(epochs):

            model.train()
            train_loss = 0.0
            progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")
            
            for batch_d, batch_lbs in progress_bar:
                optimizer.zero_grad()
                input_values = batch_d.to(device)
                batch_lbs = batch_lbs.to(device)
                outputs = model(input_values)
                loss = correlation_coefficient_loss(outputs, batch_lbs)
                loss.backward()
                optimizer.step()
                train_loss += loss.item()
                progress_bar.set_postfix({'train_loss': f'{train_loss/(progress_bar.n+1):.4f}'})


            train_loss /= len(train_loader)

            # Combined validation loop
            model.eval()
            val_loss = 0.0
            val_pred = []
            with torch.no_grad():
                for batch_d, batch_lbs in val_loader:
                    input_values = batch_d.to(device)
                    batch_lbs = batch_lbs.to(device)
                    
                    outputs = model(input_values)
                    loss = correlation_coefficient_loss(outputs, batch_lbs)
                    val_loss += loss.item()
                    val_pred.extend(outputs.cpu().numpy())

            val_loss /= len(val_loader)

            # Calculate validation metrics
            val_pred = np.array(val_pred).reshape(val_timesteps.shape)
            val_ground_truth = _get_ground_truth_labels([all_dict[i] for i in val_index], all_labels)
            val_pred_flat = unsplit_data_ogsize(val_pred, window_size, step_sequence, 25, val_ground_truth.shape[-1])
            val_prc_coef = _calculate_flattened_accuracy(val_pred_flat, val_ground_truth)
            
            # Accumulate metrics for this fold and epoch
            train_acc.append(1- train_loss)
            val_acc.append(1- val_loss)

            # Log metrics
            writer.add_scalar(f"Loss/train_fold_{fold + 1}", train_loss, epoch)
            writer.add_scalar(f"Loss/val_fold_{fold + 1}", val_loss, epoch)
            writer.add_scalar(f"Pearson/val_fold_{fold + 1}", val_prc_coef, epoch)

            print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Pearson: {val_prc_coef:.4f}")

            # Check if validation loss improved
            if val_loss < best_val_loss:
                print(f"Validation loss improved from {best_val_loss:.4f} to {val_loss:.4f}. Saving best model...")
                best_val_loss = val_loss
                best_val_loss_flat = val_prc_coef
                early_stopping_counter = 0

                # Save the best model
                torch.save(model.state_dict(), best_model_path)
            else:
                early_stopping_counter += 1
                print(f"Validation loss did not improve for {early_stopping_counter} epochs.")
                #model.load_state_dict(torch.load(best_model_path))


            # Early stopping
            if early_stopping_counter >= early_stopping_patience:
                print(f"Early stopping triggered at epoch {epoch + 1}. Loading best model.")
                # Load the best model's weights
                model.load_state_dict(torch.load(best_model_path))
                break

        test_pred = []
        test_loss = 0.0
        with torch.no_grad():
            for batch_d, batch_lbs in test_loader:
                input_values = batch_d.to(device)
                batch_lbs = batch_lbs.to(device)
                
                outputs = model(input_values)
                loss = correlation_coefficient_loss(outputs, batch_lbs)
                test_loss += loss.item()
                test_pred.extend(outputs.cpu().numpy())

        test_loss /= len(test_loader)
        test_pred = np.array(test_pred).reshape(prepared_test_labels_timesteps.shape)
        test_ground_truth = _get_ground_truth_labels(list(test_dict.values()), test_labels)
        test_pred_flat = unsplit_data_ogsize(test_pred, window_size, step_sequence, 25, test_ground_truth.shape[-1])
        test_prc_coef = _calculate_flattened_accuracy(test_pred_flat, test_ground_truth)

        print(f"Fold {fold + 1}:")
        print(f"  Validation Pearson Coefficient  acc: {1- best_val_loss}")
        print(f"  Validation Pearson Coefficient flat acc: {best_val_loss_flat}")
        print(f"  Test acc: {1- test_loss}")
        print(f"  Test Pearson Coefficient acc(flattened): {test_prc_coef}")

        fold_metrics.append({
            'Fold': fold + 1,
            'val_prc_acc': 1- best_val_loss,
            'val_prc_acc_flat': best_val_loss_flat,
            'test_acc': 1- test_loss,
            'test_prc_flat': test_prc_coef
        })


                # Log fold-specific metrics as tables
        fold_table = f"| Fold | Val Pearson Acc | Val Pearson Flat | Test Acc | Test Pearson Flat |\n" \
                     f"|------|-----------------|------------------|----------|-------------------|\n" \
                     f"| {fold + 1} | {1 - val_loss:.4f} | {val_prc_coef:.4f} | {1 - test_loss:.4f} | {test_prc_coef:.4f} |\n"
        writer.add_text(f"Fold_{fold + 1}_Metrics", fold_table)
        # Accumulate fold metrics across all folds
        train_acc_epoch.append(train_acc)
        val_acc_epoch.append(train_acc)


    
        # After all folds, compute and log the average metrics per epoch across all folds
    for epoch in range(epochs):
        avg_train_loss = np.mean([fold_losses[epoch] for fold_losses in train_acc_epoch if len(fold_losses) > epoch])
        avg_val_loss = np.mean([fold_losses[epoch] for fold_losses in val_acc_epoch if len(fold_losses) > epoch])

        # Log the averaged metrics for the epoch across all folds
        writer.add_scalar("Average_acc/train", avg_train_loss, epoch)
        writer.add_scalar("Average_acc/val", avg_val_loss, epoch)
            

    # Calculate average metrics
    avg_metrics = {key: np.mean([fold[key] for fold in fold_metrics if key != 'Fold']) for key in fold_metrics[0].keys() if key != 'Fold'}
        # Log the final average table
    avg_table = "| Fold | Val Pearson Acc | Val Pearson Flat | Test Acc | Test Pearson Flat |\n" \
                "|------|-----------------|------------------|----------|-------------------|\n" \
                f"| Average | {avg_metrics['val_prc_acc']:.4f} | {avg_metrics['val_prc_acc_flat']:.4f} | {avg_metrics['test_acc']:.4f} | {avg_metrics['test_prc_flat']:.4f} |\n"
    writer.add_text("Average_Metrics", avg_table)
    # Add average metrics to results
    avg_metrics['Fold'] = 'Average'
    fold_metrics.append(avg_metrics)

    # save averga date to CSV
    results_df = pd.DataFrame(fold_metrics)
    csv_path = os.path.join(run_dir, 'fold_results.csv')
    results_df.to_csv(csv_path, index=False)
    
    # Save fold indices CSV
    fold_indices_df.to_csv(os.path.join(run_dir, 'fold_indices.csv'), index=False)
    
    writer.close()


    print("\nTraining completed.")
    print("Average metrics across all folds:")
    for key, value in avg_metrics.items():
        print(f"  {key}: {value}")


if __name__ == "__main__":
    ## Path to data
    #path = "/home/glenn/Downloads/"
    path = "../DATA/"


    # Model parameters
    model_config = {
        "VRBModel": {
            "model" : VRBModel,
            "model_name": "facebook/hubert-large-ll60k",
            "hidden_units": 64,
            "n_gru": 3,
            "output_size": None  # Will be set dynamically
        },
        "Wav2Vec2ConvLSTMModel": {
            "model" : Wav2Vec2ConvLSTMModel,
            "model_name": "facebook/wav2vec2-base",
            "hidden_units": 128,
            "n_lstm": 2,
            "output_size": None  # Will be set dynamically
        },
            "RespBertCNNModelV2": {
            'model' : RespBertCNNModelV2,
            "model_name": "microsoft/wavlm-large",
            "hidden_units": 128,
            "output_size": None  
        },
            "RespBertLSTMModelV2": {
            'model': RespBertLSTMModelV2,
            "model_name": "microsoft/wavlm-large",
            "hidden_units": 128,
            "n_lstm": 3,
            "output_size": None  
        }
            ,
            "RespBertLSTMCNNTransformerModel": {
            'model': RespBertLSTMCNNTransformerModel,
            "model_name": "microsoft/wavlm-large",
            "hidden_units": 1024,
            "n_lstm": 2,
            "output_size": None  
        }
    }
    

    
    # Train and data parameters
    epochs = 60
    batch_size = 13
    window_size = 30
    step_size = 6
    data_parts = 4 # aka folds
    early_stopping_patience = 10
    
    config = model_config["RespBertLSTMCNNTransformerModel"]
    

    ## same wav2vec2 base model and pipeline used in the paper
    #processor = AutoProcessor.from_pretrained(config["model_name"])
    processor = Wav2Vec2FeatureExtractor.from_pretrained(config["model_name"])

    train(
        path_to_data=path+"ComParE2020_Breathing/wav/",
        path_to_labels=path+"ComParE2020_Breathing/lab/",
        window_size=window_size,
        batch_size=batch_size,
        config = config,
        step_size=step_size,
        data_parts= data_parts ,
        early_stopping_patience= early_stopping_patience,
        epochs= epochs,
        processor = processor
    )

Fold 1/4
(864, 480000)
750


/home/gdwildt/.conda/envs/thesis/lib/python3.11/site-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
Epoch 1/60:   0%|          | 0/67 [00:00<?, ?it/s]/home/gdwildt/.conda/envs/thesis/lib/python3.11/site-packages/torch/nn/functional.py:5193: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
Epoch 1/60:   0%|          | 0/67 [00:02<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.75 GiB. GPU 0 has a total capacity of 93.12 GiB of which 218.75 MiB is free. Including non-PyTorch memory, this process has 92.90 GiB memory in use. Of the allocated memory 91.37 GiB is allocated by PyTorch, and 882.05 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import KFold
from datetime import datetime
#from torch.utils.tensorboard import SummaryWriter
import torchaudio
from typing import List, Tuple
from pt_utils import *
from pt_dataset import *
from pt_models import *
from pt_utils import *
from tensorboardX import SummaryWriter
from transformers import get_cosine_schedule_with_warmup


def create_run_directory():
    base_dir = "pt_runs_SOTA"
    timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
    run_dir = os.path.join(base_dir, timestamp)
    os.makedirs(run_dir, exist_ok=True)
    return run_dir

def _calculate_flattened_accuracy(average, ground_truth_labels):
    s_acc = 0
    for b in range(len(ground_truth_labels)):
        s, _ = scipy.stats.pearsonr(average[b], ground_truth_labels[b])
        s_acc += s
    return s_acc / len(ground_truth_labels)

def _choose_real_labs_only_with_filenames(labels, filenames):
    return labels[labels['filename'].isin(filenames)]

def _get_ground_truth_labels(ground_truth_names, labels):
    ground_truth_labels = []
    for batch_name in ground_truth_names:
        ground_truth_label = _choose_real_labs_only_with_filenames(labels, [batch_name])
        ground_truth_labels.append(ground_truth_label)
    return np.array(ground_truth_labels)[:, :, -1].astype(np.float32)

def train(path_to_data, path_to_labels, window_size=16, step_size=6, data_parts=4, epochs=100, batch_size=10, early_stopping_patience=20, config = None, processor = None):
    run_dir = create_run_directory()
    log_dir = os.path.join(run_dir, "logs")
    os.makedirs(log_dir, exist_ok=True)

    # Parameters
    length_sequence = window_size 
    step_sequence = step_size

    # Load and prepare data
    train_data, train_labels, train_dict, frame_rate = load_data(path_to_data, path_to_labels, 'train')
    devel_data, devel_labels, devel_dict, frame_rate = load_data(path_to_data, path_to_labels, 'devel')
    test_data, test_labels, test_dict, frame_rate = load_data(path_to_data, path_to_labels, 'test')

    # Combine train and devel data
    all_data = np.concatenate((train_data, devel_data), axis=0)
    all_labels = pd.concat([train_labels, devel_labels])
    all_dict = np.concatenate((list(train_dict.values()), list(devel_dict.values())), axis=0)

    # Prepare data
    prepared_data, prepared_labels, prepared_labels_timesteps = prepare_data(all_data, all_labels, all_dict, frame_rate, length_sequence * 16000, step_sequence * 16000)
    prepared_test_data, prepared_test_labels, prepared_test_labels_timesteps = prepare_data(test_data, test_labels, test_dict, frame_rate, length_sequence * 16000, step_sequence * 16000)

    # Create CSV file for storing fold indices
    fold_indices_df = pd.DataFrame(columns=['Fold', 'Train_Indices', 'Val_Indices'])

    # Cross-validation
    kf = KFold(n_splits=data_parts)
    fold_metrics = []
    # To accumulate metrics across folds for each epoch
    train_acc_epoch = []
    val_acc_epoch = []
    test_acc_epoch = []

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    
    config["output_size"] = prepared_labels.shape[-1]
    writer = SummaryWriter(log_dir=os.path.join(log_dir,config["model_name"]))

    for fold, (train_index, val_index) in enumerate(kf.split(prepared_data)):
        print(f"Fold {fold + 1}/{data_parts}")
        best_model_path = f"{run_dir}/best_model_fold{fold+1}"
        # Save fold indices
        fold_indices_df = fold_indices_df._append({
            'Fold': fold + 1,
            'Train_Indices': train_index.tolist(),
            'Val_Indices': val_index.tolist()
        }, ignore_index=True)

        # Split data
        train_d, val_d = prepared_data[train_index], prepared_data[val_index]
        train_lbs, val_lbs = prepared_labels[train_index], prepared_labels[val_index]
        train_timesteps, val_timesteps = prepared_labels_timesteps[train_index], prepared_labels_timesteps[val_index]
        
        # Reshape data
        train_d, train_lbs = reshaping_data_for_model(train_d, train_lbs)
        val_d, val_lbs = reshaping_data_for_model(val_d, val_lbs)
        test_d, test_lbs = reshaping_data_for_model(prepared_test_data, prepared_test_labels)
        
        print(train_d.shape)

        # Create datasets
        #train_dataset = BreathingDataset(train_d, train_lbs, processor, window_size, step_sequence, augment=True)
        train_dataset = BreathingDataset(train_d, train_lbs, processor,window_size, step_sequence)
        val_dataset = BreathingDataset(val_d, val_lbs, processor, window_size, step_sequence)
        test_dataset = BreathingDataset(test_d, test_lbs, processor, window_size, step_sequence)

        # Create DataLoaders
        train_loader = DataLoader(train_dataset, batch_size=batch_size,num_workers=2, shuffle=True, collate_fn=train_dataset.collate_fn)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, num_workers=1, collate_fn=val_dataset.collate_fn)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, num_workers=1, collate_fn=test_dataset.collate_fn)
        print(config["output_size"])
        # Create and initialize model
        model = config["model"](config).to(device)
        
        #### training optimiser parameters fror apple
        #learning_rate = 0.005 
        #optimizer = optim.Adam(model.parameters(), lr=learning_rate)

        #### training optimiser parameters fror harma_2023 VRB model 
        learning_rate = 0.01 # From the paper
        optimizer = optim.Adam(model.parameters())       
        
        best_val_loss = float('inf')
        best_val_loss_flat = float('inf')
        early_stopping_counter = 0
        # To accumulate metrics across folds for each epoch
        train_acc = []
        val_acc = []
        test_acc = []
        for epoch in range(epochs):

            model.train()
            train_loss = 0.0
            progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")
            
            for batch_d, batch_lbs in progress_bar:
                optimizer.zero_grad()
                input_values = batch_d.to(device)
                batch_lbs = batch_lbs.to(device)
                outputs = model(input_values)
                loss = correlation_coefficient_loss(outputs, batch_lbs)
                loss.backward()
                optimizer.step()
                train_loss += loss.item()
                progress_bar.set_postfix({'train_loss': f'{train_loss/(progress_bar.n+1):.4f}'})


            train_loss /= len(train_loader)

            # Combined validation loop
            model.eval()
            val_loss = 0.0
            val_pred = []
            with torch.no_grad():
                for batch_d, batch_lbs in val_loader:
                    input_values = batch_d.to(device)
                    batch_lbs = batch_lbs.to(device)
                    
                    outputs = model(input_values)
                    loss = correlation_coefficient_loss(outputs, batch_lbs)
                    val_loss += loss.item()
                    val_pred.extend(outputs.cpu().numpy())

            val_loss /= len(val_loader)

            # Calculate validation metrics
            val_pred = np.array(val_pred).reshape(val_timesteps.shape)
            val_ground_truth = _get_ground_truth_labels([all_dict[i] for i in val_index], all_labels)
            val_pred_flat = unsplit_data_ogsize(val_pred, window_size, step_sequence, 25, val_ground_truth.shape[-1])
            val_prc_coef = _calculate_flattened_accuracy(val_pred_flat, val_ground_truth)
            
            # Accumulate metrics for this fold and epoch
            train_acc.append(1- train_loss)
            val_acc.append(1- val_loss)

            # Log metrics
            writer.add_scalar(f"Loss/train_fold_{fold + 1}", train_loss, epoch)
            writer.add_scalar(f"Loss/val_fold_{fold + 1}", val_loss, epoch)
            writer.add_scalar(f"Pearson/val_fold_{fold + 1}", val_prc_coef, epoch)

            print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Pearson: {val_prc_coef:.4f}")

            # Check if validation loss improved
            if val_loss < best_val_loss:
                print(f"Validation loss improved from {best_val_loss:.4f} to {val_loss:.4f}. Saving best model...")
                best_val_loss = val_loss
                best_val_loss_flat = val_prc_coef
                early_stopping_counter = 0

                # Save the best model
                torch.save(model.state_dict(), best_model_path)
            else:
                early_stopping_counter += 1
                print(f"Validation loss did not improve for {early_stopping_counter} epochs.")
                #model.load_state_dict(torch.load(best_model_path))


            # Early stopping
            if early_stopping_counter >= early_stopping_patience:
                print(f"Early stopping triggered at epoch {epoch + 1}. Loading best model.")
                # Load the best model's weights
                model.load_state_dict(torch.load(best_model_path))
                break

        test_pred = []
        test_loss = 0.0
        with torch.no_grad():
            for batch_d, batch_lbs in test_loader:
                input_values = batch_d.to(device)
                batch_lbs = batch_lbs.to(device)
                
                outputs = model(input_values)
                loss = correlation_coefficient_loss(outputs, batch_lbs)
                test_loss += loss.item()
                test_pred.extend(outputs.cpu().numpy())

        test_loss /= len(test_loader)
        test_pred = np.array(test_pred).reshape(prepared_test_labels_timesteps.shape)
        test_ground_truth = _get_ground_truth_labels(list(test_dict.values()), test_labels)
        test_pred_flat = unsplit_data_ogsize(test_pred, window_size, step_sequence, 25, test_ground_truth.shape[-1])
        test_prc_coef = _calculate_flattened_accuracy(test_pred_flat, test_ground_truth)

        print(f"Fold {fold + 1}:")
        print(f"  Validation Pearson Coefficient  acc: {1- best_val_loss}")
        print(f"  Validation Pearson Coefficient flat acc: {best_val_loss_flat}")
        print(f"  Test acc: {1- test_loss}")
        print(f"  Test Pearson Coefficient acc(flattened): {test_prc_coef}")

        fold_metrics.append({
            'Fold': fold + 1,
            'val_prc_acc': 1- best_val_loss,
            'val_prc_acc_flat': best_val_loss_flat,
            'test_acc': 1- test_loss,
            'test_prc_flat': test_prc_coef
        })


                # Log fold-specific metrics as tables
        fold_table = f"| Fold | Val Pearson Acc | Val Pearson Flat | Test Acc | Test Pearson Flat |\n" \
                     f"|------|-----------------|------------------|----------|-------------------|\n" \
                     f"| {fold + 1} | {1 - val_loss:.4f} | {val_prc_coef:.4f} | {1 - test_loss:.4f} | {test_prc_coef:.4f} |\n"
        writer.add_text(f"Fold_{fold + 1}_Metrics", fold_table)
        # Accumulate fold metrics across all folds
        train_acc_epoch.append(train_acc)
        val_acc_epoch.append(train_acc)


    
        # After all folds, compute and log the average metrics per epoch across all folds
    for epoch in range(epochs):
        avg_train_loss = np.mean([fold_losses[epoch] for fold_losses in train_acc_epoch if len(fold_losses) > epoch])
        avg_val_loss = np.mean([fold_losses[epoch] for fold_losses in val_acc_epoch if len(fold_losses) > epoch])

        # Log the averaged metrics for the epoch across all folds
        writer.add_scalar("Average_acc/train", avg_train_loss, epoch)
        writer.add_scalar("Average_acc/val", avg_val_loss, epoch)
            

    # Calculate average metrics
    avg_metrics = {key: np.mean([fold[key] for fold in fold_metrics if key != 'Fold']) for key in fold_metrics[0].keys() if key != 'Fold'}
        # Log the final average table
    avg_table = "| Fold | Val Pearson Acc | Val Pearson Flat | Test Acc | Test Pearson Flat |\n" \
                "|------|-----------------|------------------|----------|-------------------|\n" \
                f"| Average | {avg_metrics['val_prc_acc']:.4f} | {avg_metrics['val_prc_acc_flat']:.4f} | {avg_metrics['test_acc']:.4f} | {avg_metrics['test_prc_flat']:.4f} |\n"
    writer.add_text("Average_Metrics", avg_table)
    # Add average metrics to results
    avg_metrics['Fold'] = 'Average'
    fold_metrics.append(avg_metrics)

    # save averga date to CSV
    results_df = pd.DataFrame(fold_metrics)
    csv_path = os.path.join(run_dir, 'fold_results.csv')
    results_df.to_csv(csv_path, index=False)
    
    # Save fold indices CSV
    fold_indices_df.to_csv(os.path.join(run_dir, 'fold_indices.csv'), index=False)
    
    writer.close()


    print("\nTraining completed.")
    print("Average metrics across all folds:")
    for key, value in avg_metrics.items():
        print(f"  {key}: {value}")


if __name__ == "__main__":
    ## Path to data
    #path = "/home/glenn/Downloads/"
    path = "../DATA/"


    # Model parameters
    model_config = {
        "VRBModel": {
            "model" : VRBModel,
            "model_name": "facebook/hubert-large-ll60k",
            "hidden_units": 64,
            "n_gru": 3,
            "output_size": None  # Will be set dynamically
        },
        "Wav2Vec2ConvLSTMModel": {
            "model" : Wav2Vec2ConvLSTMModel,
            "model_name": "facebook/wav2vec2-base",
            "hidden_units": 128,
            "n_lstm": 2,
            "output_size": None  # Will be set dynamically
        },
            "RespBertCNNModelV2": {
            'model' : RespBertCNNModelV2,
            "model_name": "microsoft/wavlm-large",
            "hidden_units": 128,
            "output_size": None  
        },
            "RespBertLSTMModelV2": {
            'model': RespBertLSTMModelV2,
            "model_name": "microsoft/wavlm-large",
            "hidden_units": 128,
            "n_lstm": 3,
            "output_size": None  
        }
            ,
            "RespBertLSTMCNNTransformerModel": {
            'model': RespBertLSTMCNNTransformerModel,
            "model_name": "microsoft/wavlm-large",
            "hidden_units": 1024,
            "n_lstm": 2,
            "output_size": None  
        }
    }
    

    
    # Train and data parameters
    epochs = 60
    batch_size = 10
    window_size = 30
    step_size = 6
    data_parts = 4 # aka folds
    early_stopping_patience = 10
    
    config = model_config["RespBertLSTMCNNTransformerModel"]
    

    ## same wav2vec2 base model and pipeline used in the paper
    #processor = AutoProcessor.from_pretrained(config["model_name"])
    processor = Wav2Vec2FeatureExtractor.from_pretrained(config["model_name"])

    train(
        path_to_data=path+"ComParE2020_Breathing/wav/",
        path_to_labels=path+"ComParE2020_Breathing/lab/",
        window_size=window_size,
        batch_size=batch_size,
        config = config,
        step_size=step_size,
        data_parts= data_parts ,
        early_stopping_patience= early_stopping_patience,
        epochs= epochs,
        processor = processor
    )

Fold 1/4
(864, 480000)
750


/home/gdwildt/.conda/envs/thesis/lib/python3.11/site-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
Epoch 1/60:   0%|          | 0/87 [00:00<?, ?it/s]/home/gdwildt/.conda/envs/thesis/lib/python3.11/site-packages/torch/nn/functional.py:5193: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
Epoch 1/60:   0%|          | 0/87 [00:02<?, ?it/s]


AssertionError: was expecting embedding dimension of 3072, but got 1499

In [ ]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import KFold
from datetime import datetime
#from torch.utils.tensorboard import SummaryWriter
import torchaudio
from typing import List, Tuple
from pt_utils import *
from pt_dataset import *
from pt_models import *
from pt_utils import *
from tensorboardX import SummaryWriter
from transformers import get_cosine_schedule_with_warmup


def create_run_directory():
    base_dir = "pt_runs_SOTA"
    timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
    run_dir = os.path.join(base_dir, timestamp)
    os.makedirs(run_dir, exist_ok=True)
    return run_dir

def _calculate_flattened_accuracy(average, ground_truth_labels):
    s_acc = 0
    for b in range(len(ground_truth_labels)):
        s, _ = scipy.stats.pearsonr(average[b], ground_truth_labels[b])
        s_acc += s
    return s_acc / len(ground_truth_labels)

def _choose_real_labs_only_with_filenames(labels, filenames):
    return labels[labels['filename'].isin(filenames)]

def _get_ground_truth_labels(ground_truth_names, labels):
    ground_truth_labels = []
    for batch_name in ground_truth_names:
        ground_truth_label = _choose_real_labs_only_with_filenames(labels, [batch_name])
        ground_truth_labels.append(ground_truth_label)
    return np.array(ground_truth_labels)[:, :, -1].astype(np.float32)

def train(path_to_data, path_to_labels, window_size=16, step_size=6, data_parts=4, epochs=100, batch_size=10, early_stopping_patience=20, config = None, processor = None):
    run_dir = create_run_directory()
    log_dir = os.path.join(run_dir, "logs")
    os.makedirs(log_dir, exist_ok=True)

    # Parameters
    length_sequence = window_size 
    step_sequence = step_size

    # Load and prepare data
    train_data, train_labels, train_dict, frame_rate = load_data(path_to_data, path_to_labels, 'train')
    devel_data, devel_labels, devel_dict, frame_rate = load_data(path_to_data, path_to_labels, 'devel')
    test_data, test_labels, test_dict, frame_rate = load_data(path_to_data, path_to_labels, 'test')

    # Combine train and devel data
    all_data = np.concatenate((train_data, devel_data), axis=0)
    all_labels = pd.concat([train_labels, devel_labels])
    all_dict = np.concatenate((list(train_dict.values()), list(devel_dict.values())), axis=0)

    # Prepare data
    prepared_data, prepared_labels, prepared_labels_timesteps = prepare_data(all_data, all_labels, all_dict, frame_rate, length_sequence * 16000, step_sequence * 16000)
    prepared_test_data, prepared_test_labels, prepared_test_labels_timesteps = prepare_data(test_data, test_labels, test_dict, frame_rate, length_sequence * 16000, step_sequence * 16000)

    # Create CSV file for storing fold indices
    fold_indices_df = pd.DataFrame(columns=['Fold', 'Train_Indices', 'Val_Indices'])

    # Cross-validation
    kf = KFold(n_splits=data_parts)
    fold_metrics = []
    # To accumulate metrics across folds for each epoch
    train_acc_epoch = []
    val_acc_epoch = []
    test_acc_epoch = []

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    
    config["output_size"] = prepared_labels.shape[-1]
    writer = SummaryWriter(log_dir=os.path.join(log_dir,config["model_name"]))

    for fold, (train_index, val_index) in enumerate(kf.split(prepared_data)):
        print(f"Fold {fold + 1}/{data_parts}")
        best_model_path = f"{run_dir}/best_model_fold{fold+1}"
        # Save fold indices
        fold_indices_df = fold_indices_df._append({
            'Fold': fold + 1,
            'Train_Indices': train_index.tolist(),
            'Val_Indices': val_index.tolist()
        }, ignore_index=True)

        # Split data
        train_d, val_d = prepared_data[train_index], prepared_data[val_index]
        train_lbs, val_lbs = prepared_labels[train_index], prepared_labels[val_index]
        train_timesteps, val_timesteps = prepared_labels_timesteps[train_index], prepared_labels_timesteps[val_index]
        
        # Reshape data
        train_d, train_lbs = reshaping_data_for_model(train_d, train_lbs)
        val_d, val_lbs = reshaping_data_for_model(val_d, val_lbs)
        test_d, test_lbs = reshaping_data_for_model(prepared_test_data, prepared_test_labels)
        
        print(train_d.shape)

        # Create datasets
        #train_dataset = BreathingDataset(train_d, train_lbs, processor, window_size, step_sequence, augment=True)
        train_dataset = BreathingDataset(train_d, train_lbs, processor,window_size, step_sequence)
        val_dataset = BreathingDataset(val_d, val_lbs, processor, window_size, step_sequence)
        test_dataset = BreathingDataset(test_d, test_lbs, processor, window_size, step_sequence)

        # Create DataLoaders
        train_loader = DataLoader(train_dataset, batch_size=batch_size,num_workers=2, shuffle=True, collate_fn=train_dataset.collate_fn)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, num_workers=1, collate_fn=val_dataset.collate_fn)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, num_workers=1, collate_fn=test_dataset.collate_fn)
        print(config["output_size"])
        # Create and initialize model
        model = config["model"](config).to(device)
        
        #### training optimiser parameters fror apple
        #learning_rate = 0.005 
        #optimizer = optim.Adam(model.parameters(), lr=learning_rate)

        #### training optimiser parameters fror harma_2023 VRB model 
        learning_rate = 0.01 # From the paper
        optimizer = optim.Adam(model.parameters())       
        
        best_val_loss = float('inf')
        best_val_loss_flat = float('inf')
        early_stopping_counter = 0
        # To accumulate metrics across folds for each epoch
        train_acc = []
        val_acc = []
        test_acc = []
        for epoch in range(epochs):

            model.train()
            train_loss = 0.0
            progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")
            
            for batch_d, batch_lbs in progress_bar:
                optimizer.zero_grad()
                input_values = batch_d.to(device)
                batch_lbs = batch_lbs.to(device)
                outputs = model(input_values)
                loss = correlation_coefficient_loss(outputs, batch_lbs)
                loss.backward()
                optimizer.step()
                train_loss += loss.item()
                progress_bar.set_postfix({'train_loss': f'{train_loss/(progress_bar.n+1):.4f}'})


            train_loss /= len(train_loader)

            # Combined validation loop
            model.eval()
            val_loss = 0.0
            val_pred = []
            with torch.no_grad():
                for batch_d, batch_lbs in val_loader:
                    input_values = batch_d.to(device)
                    batch_lbs = batch_lbs.to(device)
                    
                    outputs = model(input_values)
                    loss = correlation_coefficient_loss(outputs, batch_lbs)
                    val_loss += loss.item()
                    val_pred.extend(outputs.cpu().numpy())

            val_loss /= len(val_loader)

            # Calculate validation metrics
            val_pred = np.array(val_pred).reshape(val_timesteps.shape)
            val_ground_truth = _get_ground_truth_labels([all_dict[i] for i in val_index], all_labels)
            val_pred_flat = unsplit_data_ogsize(val_pred, window_size, step_sequence, 25, val_ground_truth.shape[-1])
            val_prc_coef = _calculate_flattened_accuracy(val_pred_flat, val_ground_truth)
            
            # Accumulate metrics for this fold and epoch
            train_acc.append(1- train_loss)
            val_acc.append(1- val_loss)

            # Log metrics
            writer.add_scalar(f"Loss/train_fold_{fold + 1}", train_loss, epoch)
            writer.add_scalar(f"Loss/val_fold_{fold + 1}", val_loss, epoch)
            writer.add_scalar(f"Pearson/val_fold_{fold + 1}", val_prc_coef, epoch)

            print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Pearson: {val_prc_coef:.4f}")

            # Check if validation loss improved
            if val_loss < best_val_loss:
                print(f"Validation loss improved from {best_val_loss:.4f} to {val_loss:.4f}. Saving best model...")
                best_val_loss = val_loss
                best_val_loss_flat = val_prc_coef
                early_stopping_counter = 0

                # Save the best model
                torch.save(model.state_dict(), best_model_path)
            else:
                early_stopping_counter += 1
                print(f"Validation loss did not improve for {early_stopping_counter} epochs.")
                #model.load_state_dict(torch.load(best_model_path))


            # Early stopping
            if early_stopping_counter >= early_stopping_patience:
                print(f"Early stopping triggered at epoch {epoch + 1}. Loading best model.")
                # Load the best model's weights
                model.load_state_dict(torch.load(best_model_path))
                break

        test_pred = []
        test_loss = 0.0
        with torch.no_grad():
            for batch_d, batch_lbs in test_loader:
                input_values = batch_d.to(device)
                batch_lbs = batch_lbs.to(device)
                
                outputs = model(input_values)
                loss = correlation_coefficient_loss(outputs, batch_lbs)
                test_loss += loss.item()
                test_pred.extend(outputs.cpu().numpy())

        test_loss /= len(test_loader)
        test_pred = np.array(test_pred).reshape(prepared_test_labels_timesteps.shape)
        test_ground_truth = _get_ground_truth_labels(list(test_dict.values()), test_labels)
        test_pred_flat = unsplit_data_ogsize(test_pred, window_size, step_sequence, 25, test_ground_truth.shape[-1])
        test_prc_coef = _calculate_flattened_accuracy(test_pred_flat, test_ground_truth)

        print(f"Fold {fold + 1}:")
        print(f"  Validation Pearson Coefficient  acc: {1- best_val_loss}")
        print(f"  Validation Pearson Coefficient flat acc: {best_val_loss_flat}")
        print(f"  Test acc: {1- test_loss}")
        print(f"  Test Pearson Coefficient acc(flattened): {test_prc_coef}")

        fold_metrics.append({
            'Fold': fold + 1,
            'val_prc_acc': 1- best_val_loss,
            'val_prc_acc_flat': best_val_loss_flat,
            'test_acc': 1- test_loss,
            'test_prc_flat': test_prc_coef
        })


                # Log fold-specific metrics as tables
        fold_table = f"| Fold | Val Pearson Acc | Val Pearson Flat | Test Acc | Test Pearson Flat |\n" \
                     f"|------|-----------------|------------------|----------|-------------------|\n" \
                     f"| {fold + 1} | {1 - val_loss:.4f} | {val_prc_coef:.4f} | {1 - test_loss:.4f} | {test_prc_coef:.4f} |\n"
        writer.add_text(f"Fold_{fold + 1}_Metrics", fold_table)
        # Accumulate fold metrics across all folds
        train_acc_epoch.append(train_acc)
        val_acc_epoch.append(train_acc)


    
        # After all folds, compute and log the average metrics per epoch across all folds
    for epoch in range(epochs):
        avg_train_loss = np.mean([fold_losses[epoch] for fold_losses in train_acc_epoch if len(fold_losses) > epoch])
        avg_val_loss = np.mean([fold_losses[epoch] for fold_losses in val_acc_epoch if len(fold_losses) > epoch])

        # Log the averaged metrics for the epoch across all folds
        writer.add_scalar("Average_acc/train", avg_train_loss, epoch)
        writer.add_scalar("Average_acc/val", avg_val_loss, epoch)
            

    # Calculate average metrics
    avg_metrics = {key: np.mean([fold[key] for fold in fold_metrics if key != 'Fold']) for key in fold_metrics[0].keys() if key != 'Fold'}
        # Log the final average table
    avg_table = "| Fold | Val Pearson Acc | Val Pearson Flat | Test Acc | Test Pearson Flat |\n" \
                "|------|-----------------|------------------|----------|-------------------|\n" \
                f"| Average | {avg_metrics['val_prc_acc']:.4f} | {avg_metrics['val_prc_acc_flat']:.4f} | {avg_metrics['test_acc']:.4f} | {avg_metrics['test_prc_flat']:.4f} |\n"
    writer.add_text("Average_Metrics", avg_table)
    # Add average metrics to results
    avg_metrics['Fold'] = 'Average'
    fold_metrics.append(avg_metrics)

    # save averga date to CSV
    results_df = pd.DataFrame(fold_metrics)
    csv_path = os.path.join(run_dir, 'fold_results.csv')
    results_df.to_csv(csv_path, index=False)
    
    # Save fold indices CSV
    fold_indices_df.to_csv(os.path.join(run_dir, 'fold_indices.csv'), index=False)
    
    writer.close()


    print("\nTraining completed.")
    print("Average metrics across all folds:")
    for key, value in avg_metrics.items():
        print(f"  {key}: {value}")


if __name__ == "__main__":
    ## Path to data
    #path = "/home/glenn/Downloads/"
    path = "../DATA/"


    # Model parameters
    model_config = {
        "VRBModel": {
            "model" : VRBModel,
            "model_name": "facebook/hubert-large-ll60k",
            "hidden_units": 64,
            "n_gru": 3,
            "output_size": None  # Will be set dynamically
        },
        "Wav2Vec2ConvLSTMModel": {
            "model" : Wav2Vec2ConvLSTMModel,
            "model_name": "facebook/wav2vec2-base",
            "hidden_units": 128,
            "n_lstm": 2,
            "output_size": None  # Will be set dynamically
        },
            "RespBertCNNModelV2": {
            'model' : RespBertCNNModelV2,
            "model_name": "microsoft/wavlm-large",
            "hidden_units": 128,
            "output_size": None  
        },
            "RespBertLSTMModelV2": {
            'model': RespBertLSTMModelV2,
            "model_name": "microsoft/wavlm-large",
            "hidden_units": 128,
            "n_lstm": 3,
            "output_size": None  
        }
            ,
            "RespBertLSTMCNNTransformerModel": {
            'model': RespBertLSTMCNNTransformerModel,
            "model_name": "microsoft/wavlm-large",
            "hidden_units": 1024,
            "n_lstm": 2,
            "output_size": None  
        }
    }
    

    
    # Train and data parameters
    epochs = 60
    batch_size = 10
    window_size = 30
    step_size = 6
    data_parts = 4 # aka folds
    early_stopping_patience = 10
    
    config = model_config["RespBertLSTMCNNTransformerModel"]
    

    ## same wav2vec2 base model and pipeline used in the paper
    #processor = AutoProcessor.from_pretrained(config["model_name"])
    processor = Wav2Vec2FeatureExtractor.from_pretrained(config["model_name"])

    train(
        path_to_data=path+"ComParE2020_Breathing/wav/",
        path_to_labels=path+"ComParE2020_Breathing/lab/",
        window_size=window_size,
        batch_size=batch_size,
        config = config,
        step_size=step_size,
        data_parts= data_parts ,
        early_stopping_patience= early_stopping_patience,
        epochs= epochs,
        processor = processor
    )

Fold 1/4
(864, 480000)
750


/home/gdwildt/.conda/envs/thesis/lib/python3.11/site-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
Epoch 1/60:   0%|          | 0/87 [00:00<?, ?it/s]/home/gdwildt/.conda/envs/thesis/lib/python3.11/site-packages/torch/nn/functional.py:5193: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
Epoch 1/60:   0%|          | 0/87 [00:02<?, ?it/s]

torch.Size([10, 1499, 1024])
torch.Size([10, 1499, 3072])


AssertionError: was expecting embedding dimension of 3072, but got 1499

In [ ]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import KFold
from datetime import datetime
#from torch.utils.tensorboard import SummaryWriter
import torchaudio
from typing import List, Tuple
from pt_utils import *
from pt_dataset import *
from pt_models import *
from pt_utils import *
from tensorboardX import SummaryWriter
from transformers import get_cosine_schedule_with_warmup


def create_run_directory():
    base_dir = "pt_runs_SOTA"
    timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
    run_dir = os.path.join(base_dir, timestamp)
    os.makedirs(run_dir, exist_ok=True)
    return run_dir

def _calculate_flattened_accuracy(average, ground_truth_labels):
    s_acc = 0
    for b in range(len(ground_truth_labels)):
        s, _ = scipy.stats.pearsonr(average[b], ground_truth_labels[b])
        s_acc += s
    return s_acc / len(ground_truth_labels)

def _choose_real_labs_only_with_filenames(labels, filenames):
    return labels[labels['filename'].isin(filenames)]

def _get_ground_truth_labels(ground_truth_names, labels):
    ground_truth_labels = []
    for batch_name in ground_truth_names:
        ground_truth_label = _choose_real_labs_only_with_filenames(labels, [batch_name])
        ground_truth_labels.append(ground_truth_label)
    return np.array(ground_truth_labels)[:, :, -1].astype(np.float32)

def train(path_to_data, path_to_labels, window_size=16, step_size=6, data_parts=4, epochs=100, batch_size=10, early_stopping_patience=20, config = None, processor = None):
    run_dir = create_run_directory()
    log_dir = os.path.join(run_dir, "logs")
    os.makedirs(log_dir, exist_ok=True)

    # Parameters
    length_sequence = window_size 
    step_sequence = step_size

    # Load and prepare data
    train_data, train_labels, train_dict, frame_rate = load_data(path_to_data, path_to_labels, 'train')
    devel_data, devel_labels, devel_dict, frame_rate = load_data(path_to_data, path_to_labels, 'devel')
    test_data, test_labels, test_dict, frame_rate = load_data(path_to_data, path_to_labels, 'test')

    # Combine train and devel data
    all_data = np.concatenate((train_data, devel_data), axis=0)
    all_labels = pd.concat([train_labels, devel_labels])
    all_dict = np.concatenate((list(train_dict.values()), list(devel_dict.values())), axis=0)

    # Prepare data
    prepared_data, prepared_labels, prepared_labels_timesteps = prepare_data(all_data, all_labels, all_dict, frame_rate, length_sequence * 16000, step_sequence * 16000)
    prepared_test_data, prepared_test_labels, prepared_test_labels_timesteps = prepare_data(test_data, test_labels, test_dict, frame_rate, length_sequence * 16000, step_sequence * 16000)

    # Create CSV file for storing fold indices
    fold_indices_df = pd.DataFrame(columns=['Fold', 'Train_Indices', 'Val_Indices'])

    # Cross-validation
    kf = KFold(n_splits=data_parts)
    fold_metrics = []
    # To accumulate metrics across folds for each epoch
    train_acc_epoch = []
    val_acc_epoch = []
    test_acc_epoch = []

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    
    config["output_size"] = prepared_labels.shape[-1]
    writer = SummaryWriter(log_dir=os.path.join(log_dir,config["model_name"]))

    for fold, (train_index, val_index) in enumerate(kf.split(prepared_data)):
        print(f"Fold {fold + 1}/{data_parts}")
        best_model_path = f"{run_dir}/best_model_fold{fold+1}"
        # Save fold indices
        fold_indices_df = fold_indices_df._append({
            'Fold': fold + 1,
            'Train_Indices': train_index.tolist(),
            'Val_Indices': val_index.tolist()
        }, ignore_index=True)

        # Split data
        train_d, val_d = prepared_data[train_index], prepared_data[val_index]
        train_lbs, val_lbs = prepared_labels[train_index], prepared_labels[val_index]
        train_timesteps, val_timesteps = prepared_labels_timesteps[train_index], prepared_labels_timesteps[val_index]
        
        # Reshape data
        train_d, train_lbs = reshaping_data_for_model(train_d, train_lbs)
        val_d, val_lbs = reshaping_data_for_model(val_d, val_lbs)
        test_d, test_lbs = reshaping_data_for_model(prepared_test_data, prepared_test_labels)
        
        print(train_d.shape)

        # Create datasets
        #train_dataset = BreathingDataset(train_d, train_lbs, processor, window_size, step_sequence, augment=True)
        train_dataset = BreathingDataset(train_d, train_lbs, processor,window_size, step_sequence)
        val_dataset = BreathingDataset(val_d, val_lbs, processor, window_size, step_sequence)
        test_dataset = BreathingDataset(test_d, test_lbs, processor, window_size, step_sequence)

        # Create DataLoaders
        train_loader = DataLoader(train_dataset, batch_size=batch_size,num_workers=2, shuffle=True, collate_fn=train_dataset.collate_fn)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, num_workers=1, collate_fn=val_dataset.collate_fn)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, num_workers=1, collate_fn=test_dataset.collate_fn)
        print(config["output_size"])
        # Create and initialize model
        model = config["model"](config).to(device)
        
        #### training optimiser parameters fror apple
        #learning_rate = 0.005 
        #optimizer = optim.Adam(model.parameters(), lr=learning_rate)

        #### training optimiser parameters fror harma_2023 VRB model 
        learning_rate = 0.01 # From the paper
        optimizer = optim.Adam(model.parameters())       
        
        best_val_loss = float('inf')
        best_val_loss_flat = float('inf')
        early_stopping_counter = 0
        # To accumulate metrics across folds for each epoch
        train_acc = []
        val_acc = []
        test_acc = []
        for epoch in range(epochs):

            model.train()
            train_loss = 0.0
            progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")
            
            for batch_d, batch_lbs in progress_bar:
                optimizer.zero_grad()
                input_values = batch_d.to(device)
                batch_lbs = batch_lbs.to(device)
                outputs = model(input_values)
                loss = correlation_coefficient_loss(outputs, batch_lbs)
                loss.backward()
                optimizer.step()
                train_loss += loss.item()
                progress_bar.set_postfix({'train_loss': f'{train_loss/(progress_bar.n+1):.4f}'})


            train_loss /= len(train_loader)

            # Combined validation loop
            model.eval()
            val_loss = 0.0
            val_pred = []
            with torch.no_grad():
                for batch_d, batch_lbs in val_loader:
                    input_values = batch_d.to(device)
                    batch_lbs = batch_lbs.to(device)
                    
                    outputs = model(input_values)
                    loss = correlation_coefficient_loss(outputs, batch_lbs)
                    val_loss += loss.item()
                    val_pred.extend(outputs.cpu().numpy())

            val_loss /= len(val_loader)

            # Calculate validation metrics
            val_pred = np.array(val_pred).reshape(val_timesteps.shape)
            val_ground_truth = _get_ground_truth_labels([all_dict[i] for i in val_index], all_labels)
            val_pred_flat = unsplit_data_ogsize(val_pred, window_size, step_sequence, 25, val_ground_truth.shape[-1])
            val_prc_coef = _calculate_flattened_accuracy(val_pred_flat, val_ground_truth)
            
            # Accumulate metrics for this fold and epoch
            train_acc.append(1- train_loss)
            val_acc.append(1- val_loss)

            # Log metrics
            writer.add_scalar(f"Loss/train_fold_{fold + 1}", train_loss, epoch)
            writer.add_scalar(f"Loss/val_fold_{fold + 1}", val_loss, epoch)
            writer.add_scalar(f"Pearson/val_fold_{fold + 1}", val_prc_coef, epoch)

            print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Pearson: {val_prc_coef:.4f}")

            # Check if validation loss improved
            if val_loss < best_val_loss:
                print(f"Validation loss improved from {best_val_loss:.4f} to {val_loss:.4f}. Saving best model...")
                best_val_loss = val_loss
                best_val_loss_flat = val_prc_coef
                early_stopping_counter = 0

                # Save the best model
                torch.save(model.state_dict(), best_model_path)
            else:
                early_stopping_counter += 1
                print(f"Validation loss did not improve for {early_stopping_counter} epochs.")
                #model.load_state_dict(torch.load(best_model_path))


            # Early stopping
            if early_stopping_counter >= early_stopping_patience:
                print(f"Early stopping triggered at epoch {epoch + 1}. Loading best model.")
                # Load the best model's weights
                model.load_state_dict(torch.load(best_model_path))
                break

        test_pred = []
        test_loss = 0.0
        with torch.no_grad():
            for batch_d, batch_lbs in test_loader:
                input_values = batch_d.to(device)
                batch_lbs = batch_lbs.to(device)
                
                outputs = model(input_values)
                loss = correlation_coefficient_loss(outputs, batch_lbs)
                test_loss += loss.item()
                test_pred.extend(outputs.cpu().numpy())

        test_loss /= len(test_loader)
        test_pred = np.array(test_pred).reshape(prepared_test_labels_timesteps.shape)
        test_ground_truth = _get_ground_truth_labels(list(test_dict.values()), test_labels)
        test_pred_flat = unsplit_data_ogsize(test_pred, window_size, step_sequence, 25, test_ground_truth.shape[-1])
        test_prc_coef = _calculate_flattened_accuracy(test_pred_flat, test_ground_truth)

        print(f"Fold {fold + 1}:")
        print(f"  Validation Pearson Coefficient  acc: {1- best_val_loss}")
        print(f"  Validation Pearson Coefficient flat acc: {best_val_loss_flat}")
        print(f"  Test acc: {1- test_loss}")
        print(f"  Test Pearson Coefficient acc(flattened): {test_prc_coef}")

        fold_metrics.append({
            'Fold': fold + 1,
            'val_prc_acc': 1- best_val_loss,
            'val_prc_acc_flat': best_val_loss_flat,
            'test_acc': 1- test_loss,
            'test_prc_flat': test_prc_coef
        })


                # Log fold-specific metrics as tables
        fold_table = f"| Fold | Val Pearson Acc | Val Pearson Flat | Test Acc | Test Pearson Flat |\n" \
                     f"|------|-----------------|------------------|----------|-------------------|\n" \
                     f"| {fold + 1} | {1 - val_loss:.4f} | {val_prc_coef:.4f} | {1 - test_loss:.4f} | {test_prc_coef:.4f} |\n"
        writer.add_text(f"Fold_{fold + 1}_Metrics", fold_table)
        # Accumulate fold metrics across all folds
        train_acc_epoch.append(train_acc)
        val_acc_epoch.append(train_acc)


    
        # After all folds, compute and log the average metrics per epoch across all folds
    for epoch in range(epochs):
        avg_train_loss = np.mean([fold_losses[epoch] for fold_losses in train_acc_epoch if len(fold_losses) > epoch])
        avg_val_loss = np.mean([fold_losses[epoch] for fold_losses in val_acc_epoch if len(fold_losses) > epoch])

        # Log the averaged metrics for the epoch across all folds
        writer.add_scalar("Average_acc/train", avg_train_loss, epoch)
        writer.add_scalar("Average_acc/val", avg_val_loss, epoch)
            

    # Calculate average metrics
    avg_metrics = {key: np.mean([fold[key] for fold in fold_metrics if key != 'Fold']) for key in fold_metrics[0].keys() if key != 'Fold'}
        # Log the final average table
    avg_table = "| Fold | Val Pearson Acc | Val Pearson Flat | Test Acc | Test Pearson Flat |\n" \
                "|------|-----------------|------------------|----------|-------------------|\n" \
                f"| Average | {avg_metrics['val_prc_acc']:.4f} | {avg_metrics['val_prc_acc_flat']:.4f} | {avg_metrics['test_acc']:.4f} | {avg_metrics['test_prc_flat']:.4f} |\n"
    writer.add_text("Average_Metrics", avg_table)
    # Add average metrics to results
    avg_metrics['Fold'] = 'Average'
    fold_metrics.append(avg_metrics)

    # save averga date to CSV
    results_df = pd.DataFrame(fold_metrics)
    csv_path = os.path.join(run_dir, 'fold_results.csv')
    results_df.to_csv(csv_path, index=False)
    
    # Save fold indices CSV
    fold_indices_df.to_csv(os.path.join(run_dir, 'fold_indices.csv'), index=False)
    
    writer.close()


    print("\nTraining completed.")
    print("Average metrics across all folds:")
    for key, value in avg_metrics.items():
        print(f"  {key}: {value}")


if __name__ == "__main__":
    ## Path to data
    #path = "/home/glenn/Downloads/"
    path = "../DATA/"


    # Model parameters
    model_config = {
        "VRBModel": {
            "model" : VRBModel,
            "model_name": "facebook/hubert-large-ll60k",
            "hidden_units": 64,
            "n_gru": 3,
            "output_size": None  # Will be set dynamically
        },
        "Wav2Vec2ConvLSTMModel": {
            "model" : Wav2Vec2ConvLSTMModel,
            "model_name": "facebook/wav2vec2-base",
            "hidden_units": 128,
            "n_lstm": 2,
            "output_size": None  # Will be set dynamically
        },
            "RespBertCNNModelV2": {
            'model' : RespBertCNNModelV2,
            "model_name": "microsoft/wavlm-large",
            "hidden_units": 128,
            "output_size": None  
        },
            "RespBertLSTMModelV2": {
            'model': RespBertLSTMModelV2,
            "model_name": "microsoft/wavlm-large",
            "hidden_units": 128,
            "n_lstm": 3,
            "output_size": None  
        }
            ,
            "RespBertLSTMCNNTransformerModel": {
            'model': RespBertLSTMCNNTransformerModel,
            "model_name": "microsoft/wavlm-large",
            "hidden_units": 1024,
            "n_lstm": 2,
            "output_size": None  
        }
    }
    

    
    # Train and data parameters
    epochs = 60
    batch_size = 10
    window_size = 30
    step_size = 6
    data_parts = 4 # aka folds
    early_stopping_patience = 10
    
    config = model_config["RespBertLSTMCNNTransformerModel"]
    

    ## same wav2vec2 base model and pipeline used in the paper
    #processor = AutoProcessor.from_pretrained(config["model_name"])
    processor = Wav2Vec2FeatureExtractor.from_pretrained(config["model_name"])

    train(
        path_to_data=path+"ComParE2020_Breathing/wav/",
        path_to_labels=path+"ComParE2020_Breathing/lab/",
        window_size=window_size,
        batch_size=batch_size,
        config = config,
        step_size=step_size,
        data_parts= data_parts ,
        early_stopping_patience= early_stopping_patience,
        epochs= epochs,
        processor = processor
    )

Fold 1/4
(864, 480000)
750


/home/gdwildt/.conda/envs/thesis/lib/python3.11/site-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
Epoch 1/60:   0%|          | 0/87 [00:00<?, ?it/s]/home/gdwildt/.conda/envs/thesis/lib/python3.11/site-packages/torch/nn/functional.py:5193: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
Epoch 1/60:   0%|          | 0/87 [00:02<?, ?it/s]

torch.Size([10, 1499, 1024])
torch.Size([10, 1499, 3072])
torch.Size([1499, 10, 3072])


AssertionError: was expecting embedding dimension of 3072, but got 10

In [ ]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import KFold
from datetime import datetime
#from torch.utils.tensorboard import SummaryWriter
import torchaudio
from typing import List, Tuple
from pt_utils import *
from pt_dataset import *
from pt_models import *
from pt_utils import *
from tensorboardX import SummaryWriter
from transformers import get_cosine_schedule_with_warmup


def create_run_directory():
    base_dir = "pt_runs_SOTA"
    timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
    run_dir = os.path.join(base_dir, timestamp)
    os.makedirs(run_dir, exist_ok=True)
    return run_dir

def _calculate_flattened_accuracy(average, ground_truth_labels):
    s_acc = 0
    for b in range(len(ground_truth_labels)):
        s, _ = scipy.stats.pearsonr(average[b], ground_truth_labels[b])
        s_acc += s
    return s_acc / len(ground_truth_labels)

def _choose_real_labs_only_with_filenames(labels, filenames):
    return labels[labels['filename'].isin(filenames)]

def _get_ground_truth_labels(ground_truth_names, labels):
    ground_truth_labels = []
    for batch_name in ground_truth_names:
        ground_truth_label = _choose_real_labs_only_with_filenames(labels, [batch_name])
        ground_truth_labels.append(ground_truth_label)
    return np.array(ground_truth_labels)[:, :, -1].astype(np.float32)

def train(path_to_data, path_to_labels, window_size=16, step_size=6, data_parts=4, epochs=100, batch_size=10, early_stopping_patience=20, config = None, processor = None):
    run_dir = create_run_directory()
    log_dir = os.path.join(run_dir, "logs")
    os.makedirs(log_dir, exist_ok=True)

    # Parameters
    length_sequence = window_size 
    step_sequence = step_size

    # Load and prepare data
    train_data, train_labels, train_dict, frame_rate = load_data(path_to_data, path_to_labels, 'train')
    devel_data, devel_labels, devel_dict, frame_rate = load_data(path_to_data, path_to_labels, 'devel')
    test_data, test_labels, test_dict, frame_rate = load_data(path_to_data, path_to_labels, 'test')

    # Combine train and devel data
    all_data = np.concatenate((train_data, devel_data), axis=0)
    all_labels = pd.concat([train_labels, devel_labels])
    all_dict = np.concatenate((list(train_dict.values()), list(devel_dict.values())), axis=0)

    # Prepare data
    prepared_data, prepared_labels, prepared_labels_timesteps = prepare_data(all_data, all_labels, all_dict, frame_rate, length_sequence * 16000, step_sequence * 16000)
    prepared_test_data, prepared_test_labels, prepared_test_labels_timesteps = prepare_data(test_data, test_labels, test_dict, frame_rate, length_sequence * 16000, step_sequence * 16000)

    # Create CSV file for storing fold indices
    fold_indices_df = pd.DataFrame(columns=['Fold', 'Train_Indices', 'Val_Indices'])

    # Cross-validation
    kf = KFold(n_splits=data_parts)
    fold_metrics = []
    # To accumulate metrics across folds for each epoch
    train_acc_epoch = []
    val_acc_epoch = []
    test_acc_epoch = []

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    
    config["output_size"] = prepared_labels.shape[-1]
    writer = SummaryWriter(log_dir=os.path.join(log_dir,config["model_name"]))

    for fold, (train_index, val_index) in enumerate(kf.split(prepared_data)):
        print(f"Fold {fold + 1}/{data_parts}")
        best_model_path = f"{run_dir}/best_model_fold{fold+1}"
        # Save fold indices
        fold_indices_df = fold_indices_df._append({
            'Fold': fold + 1,
            'Train_Indices': train_index.tolist(),
            'Val_Indices': val_index.tolist()
        }, ignore_index=True)

        # Split data
        train_d, val_d = prepared_data[train_index], prepared_data[val_index]
        train_lbs, val_lbs = prepared_labels[train_index], prepared_labels[val_index]
        train_timesteps, val_timesteps = prepared_labels_timesteps[train_index], prepared_labels_timesteps[val_index]
        
        # Reshape data
        train_d, train_lbs = reshaping_data_for_model(train_d, train_lbs)
        val_d, val_lbs = reshaping_data_for_model(val_d, val_lbs)
        test_d, test_lbs = reshaping_data_for_model(prepared_test_data, prepared_test_labels)
        
        print(train_d.shape)

        # Create datasets
        #train_dataset = BreathingDataset(train_d, train_lbs, processor, window_size, step_sequence, augment=True)
        train_dataset = BreathingDataset(train_d, train_lbs, processor,window_size, step_sequence)
        val_dataset = BreathingDataset(val_d, val_lbs, processor, window_size, step_sequence)
        test_dataset = BreathingDataset(test_d, test_lbs, processor, window_size, step_sequence)

        # Create DataLoaders
        train_loader = DataLoader(train_dataset, batch_size=batch_size,num_workers=2, shuffle=True, collate_fn=train_dataset.collate_fn)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, num_workers=1, collate_fn=val_dataset.collate_fn)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, num_workers=1, collate_fn=test_dataset.collate_fn)
        print(config["output_size"])
        # Create and initialize model
        model = config["model"](config).to(device)
        
        #### training optimiser parameters fror apple
        #learning_rate = 0.005 
        #optimizer = optim.Adam(model.parameters(), lr=learning_rate)

        #### training optimiser parameters fror harma_2023 VRB model 
        learning_rate = 0.01 # From the paper
        optimizer = optim.Adam(model.parameters())       
        
        best_val_loss = float('inf')
        best_val_loss_flat = float('inf')
        early_stopping_counter = 0
        # To accumulate metrics across folds for each epoch
        train_acc = []
        val_acc = []
        test_acc = []
        for epoch in range(epochs):

            model.train()
            train_loss = 0.0
            progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")
            
            for batch_d, batch_lbs in progress_bar:
                optimizer.zero_grad()
                input_values = batch_d.to(device)
                batch_lbs = batch_lbs.to(device)
                outputs = model(input_values)
                loss = correlation_coefficient_loss(outputs, batch_lbs)
                loss.backward()
                optimizer.step()
                train_loss += loss.item()
                progress_bar.set_postfix({'train_loss': f'{train_loss/(progress_bar.n+1):.4f}'})


            train_loss /= len(train_loader)

            # Combined validation loop
            model.eval()
            val_loss = 0.0
            val_pred = []
            with torch.no_grad():
                for batch_d, batch_lbs in val_loader:
                    input_values = batch_d.to(device)
                    batch_lbs = batch_lbs.to(device)
                    
                    outputs = model(input_values)
                    loss = correlation_coefficient_loss(outputs, batch_lbs)
                    val_loss += loss.item()
                    val_pred.extend(outputs.cpu().numpy())

            val_loss /= len(val_loader)

            # Calculate validation metrics
            val_pred = np.array(val_pred).reshape(val_timesteps.shape)
            val_ground_truth = _get_ground_truth_labels([all_dict[i] for i in val_index], all_labels)
            val_pred_flat = unsplit_data_ogsize(val_pred, window_size, step_sequence, 25, val_ground_truth.shape[-1])
            val_prc_coef = _calculate_flattened_accuracy(val_pred_flat, val_ground_truth)
            
            # Accumulate metrics for this fold and epoch
            train_acc.append(1- train_loss)
            val_acc.append(1- val_loss)

            # Log metrics
            writer.add_scalar(f"Loss/train_fold_{fold + 1}", train_loss, epoch)
            writer.add_scalar(f"Loss/val_fold_{fold + 1}", val_loss, epoch)
            writer.add_scalar(f"Pearson/val_fold_{fold + 1}", val_prc_coef, epoch)

            print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Pearson: {val_prc_coef:.4f}")

            # Check if validation loss improved
            if val_loss < best_val_loss:
                print(f"Validation loss improved from {best_val_loss:.4f} to {val_loss:.4f}. Saving best model...")
                best_val_loss = val_loss
                best_val_loss_flat = val_prc_coef
                early_stopping_counter = 0

                # Save the best model
                torch.save(model.state_dict(), best_model_path)
            else:
                early_stopping_counter += 1
                print(f"Validation loss did not improve for {early_stopping_counter} epochs.")
                #model.load_state_dict(torch.load(best_model_path))


            # Early stopping
            if early_stopping_counter >= early_stopping_patience:
                print(f"Early stopping triggered at epoch {epoch + 1}. Loading best model.")
                # Load the best model's weights
                model.load_state_dict(torch.load(best_model_path))
                break

        test_pred = []
        test_loss = 0.0
        with torch.no_grad():
            for batch_d, batch_lbs in test_loader:
                input_values = batch_d.to(device)
                batch_lbs = batch_lbs.to(device)
                
                outputs = model(input_values)
                loss = correlation_coefficient_loss(outputs, batch_lbs)
                test_loss += loss.item()
                test_pred.extend(outputs.cpu().numpy())

        test_loss /= len(test_loader)
        test_pred = np.array(test_pred).reshape(prepared_test_labels_timesteps.shape)
        test_ground_truth = _get_ground_truth_labels(list(test_dict.values()), test_labels)
        test_pred_flat = unsplit_data_ogsize(test_pred, window_size, step_sequence, 25, test_ground_truth.shape[-1])
        test_prc_coef = _calculate_flattened_accuracy(test_pred_flat, test_ground_truth)

        print(f"Fold {fold + 1}:")
        print(f"  Validation Pearson Coefficient  acc: {1- best_val_loss}")
        print(f"  Validation Pearson Coefficient flat acc: {best_val_loss_flat}")
        print(f"  Test acc: {1- test_loss}")
        print(f"  Test Pearson Coefficient acc(flattened): {test_prc_coef}")

        fold_metrics.append({
            'Fold': fold + 1,
            'val_prc_acc': 1- best_val_loss,
            'val_prc_acc_flat': best_val_loss_flat,
            'test_acc': 1- test_loss,
            'test_prc_flat': test_prc_coef
        })


                # Log fold-specific metrics as tables
        fold_table = f"| Fold | Val Pearson Acc | Val Pearson Flat | Test Acc | Test Pearson Flat |\n" \
                     f"|------|-----------------|------------------|----------|-------------------|\n" \
                     f"| {fold + 1} | {1 - val_loss:.4f} | {val_prc_coef:.4f} | {1 - test_loss:.4f} | {test_prc_coef:.4f} |\n"
        writer.add_text(f"Fold_{fold + 1}_Metrics", fold_table)
        # Accumulate fold metrics across all folds
        train_acc_epoch.append(train_acc)
        val_acc_epoch.append(train_acc)


    
        # After all folds, compute and log the average metrics per epoch across all folds
    for epoch in range(epochs):
        avg_train_loss = np.mean([fold_losses[epoch] for fold_losses in train_acc_epoch if len(fold_losses) > epoch])
        avg_val_loss = np.mean([fold_losses[epoch] for fold_losses in val_acc_epoch if len(fold_losses) > epoch])

        # Log the averaged metrics for the epoch across all folds
        writer.add_scalar("Average_acc/train", avg_train_loss, epoch)
        writer.add_scalar("Average_acc/val", avg_val_loss, epoch)
            

    # Calculate average metrics
    avg_metrics = {key: np.mean([fold[key] for fold in fold_metrics if key != 'Fold']) for key in fold_metrics[0].keys() if key != 'Fold'}
        # Log the final average table
    avg_table = "| Fold | Val Pearson Acc | Val Pearson Flat | Test Acc | Test Pearson Flat |\n" \
                "|------|-----------------|------------------|----------|-------------------|\n" \
                f"| Average | {avg_metrics['val_prc_acc']:.4f} | {avg_metrics['val_prc_acc_flat']:.4f} | {avg_metrics['test_acc']:.4f} | {avg_metrics['test_prc_flat']:.4f} |\n"
    writer.add_text("Average_Metrics", avg_table)
    # Add average metrics to results
    avg_metrics['Fold'] = 'Average'
    fold_metrics.append(avg_metrics)

    # save averga date to CSV
    results_df = pd.DataFrame(fold_metrics)
    csv_path = os.path.join(run_dir, 'fold_results.csv')
    results_df.to_csv(csv_path, index=False)
    
    # Save fold indices CSV
    fold_indices_df.to_csv(os.path.join(run_dir, 'fold_indices.csv'), index=False)
    
    writer.close()


    print("\nTraining completed.")
    print("Average metrics across all folds:")
    for key, value in avg_metrics.items():
        print(f"  {key}: {value}")


if __name__ == "__main__":
    ## Path to data
    #path = "/home/glenn/Downloads/"
    path = "../DATA/"


    # Model parameters
    model_config = {
        "VRBModel": {
            "model" : VRBModel,
            "model_name": "facebook/hubert-large-ll60k",
            "hidden_units": 64,
            "n_gru": 3,
            "output_size": None  # Will be set dynamically
        },
        "Wav2Vec2ConvLSTMModel": {
            "model" : Wav2Vec2ConvLSTMModel,
            "model_name": "facebook/wav2vec2-base",
            "hidden_units": 128,
            "n_lstm": 2,
            "output_size": None  # Will be set dynamically
        },
            "RespBertCNNModelV2": {
            'model' : RespBertCNNModelV2,
            "model_name": "microsoft/wavlm-large",
            "hidden_units": 128,
            "output_size": None  
        },
            "RespBertLSTMModelV2": {
            'model': RespBertLSTMModelV2,
            "model_name": "microsoft/wavlm-large",
            "hidden_units": 128,
            "n_lstm": 3,
            "output_size": None  
        }
            ,
            "RespBertLSTMCNNTransformerModel": {
            'model': RespBertLSTMCNNTransformerModel,
            "model_name": "microsoft/wavlm-large",
            "hidden_units": 1024,
            "n_lstm": 2,
            "output_size": None  
        }
    }
    

    
    # Train and data parameters
    epochs = 60
    batch_size = 10
    window_size = 30
    step_size = 6
    data_parts = 4 # aka folds
    early_stopping_patience = 10
    
    config = model_config["RespBertLSTMCNNTransformerModel"]
    

    ## same wav2vec2 base model and pipeline used in the paper
    #processor = AutoProcessor.from_pretrained(config["model_name"])
    processor = Wav2Vec2FeatureExtractor.from_pretrained(config["model_name"])

    train(
        path_to_data=path+"ComParE2020_Breathing/wav/",
        path_to_labels=path+"ComParE2020_Breathing/lab/",
        window_size=window_size,
        batch_size=batch_size,
        config = config,
        step_size=step_size,
        data_parts= data_parts ,
        early_stopping_patience= early_stopping_patience,
        epochs= epochs,
        processor = processor
    )

Fold 1/4
(864, 480000)
750


/home/gdwildt/.conda/envs/thesis/lib/python3.11/site-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
Epoch 1/60:   0%|          | 0/87 [00:00<?, ?it/s]/home/gdwildt/.conda/envs/thesis/lib/python3.11/site-packages/torch/nn/functional.py:5193: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
Epoch 1/60:   0%|          | 0/87 [00:01<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.35 GiB. GPU 0 has a total capacity of 93.12 GiB of which 98.75 MiB is free. Including non-PyTorch memory, this process has 93.02 GiB memory in use. Of the allocated memory 91.01 GiB is allocated by PyTorch, and 1.34 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import KFold
from datetime import datetime
#from torch.utils.tensorboard import SummaryWriter
import torchaudio
from typing import List, Tuple
from pt_utils import *
from pt_dataset import *
from pt_models import *
from pt_utils import *
from tensorboardX import SummaryWriter
from transformers import get_cosine_schedule_with_warmup


def create_run_directory():
    base_dir = "pt_runs_SOTA"
    timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
    run_dir = os.path.join(base_dir, timestamp)
    os.makedirs(run_dir, exist_ok=True)
    return run_dir

def _calculate_flattened_accuracy(average, ground_truth_labels):
    s_acc = 0
    for b in range(len(ground_truth_labels)):
        s, _ = scipy.stats.pearsonr(average[b], ground_truth_labels[b])
        s_acc += s
    return s_acc / len(ground_truth_labels)

def _choose_real_labs_only_with_filenames(labels, filenames):
    return labels[labels['filename'].isin(filenames)]

def _get_ground_truth_labels(ground_truth_names, labels):
    ground_truth_labels = []
    for batch_name in ground_truth_names:
        ground_truth_label = _choose_real_labs_only_with_filenames(labels, [batch_name])
        ground_truth_labels.append(ground_truth_label)
    return np.array(ground_truth_labels)[:, :, -1].astype(np.float32)

def train(path_to_data, path_to_labels, window_size=16, step_size=6, data_parts=4, epochs=100, batch_size=10, early_stopping_patience=20, config = None, processor = None):
    run_dir = create_run_directory()
    log_dir = os.path.join(run_dir, "logs")
    os.makedirs(log_dir, exist_ok=True)

    # Parameters
    length_sequence = window_size 
    step_sequence = step_size

    # Load and prepare data
    train_data, train_labels, train_dict, frame_rate = load_data(path_to_data, path_to_labels, 'train')
    devel_data, devel_labels, devel_dict, frame_rate = load_data(path_to_data, path_to_labels, 'devel')
    test_data, test_labels, test_dict, frame_rate = load_data(path_to_data, path_to_labels, 'test')

    # Combine train and devel data
    all_data = np.concatenate((train_data, devel_data), axis=0)
    all_labels = pd.concat([train_labels, devel_labels])
    all_dict = np.concatenate((list(train_dict.values()), list(devel_dict.values())), axis=0)

    # Prepare data
    prepared_data, prepared_labels, prepared_labels_timesteps = prepare_data(all_data, all_labels, all_dict, frame_rate, length_sequence * 16000, step_sequence * 16000)
    prepared_test_data, prepared_test_labels, prepared_test_labels_timesteps = prepare_data(test_data, test_labels, test_dict, frame_rate, length_sequence * 16000, step_sequence * 16000)

    # Create CSV file for storing fold indices
    fold_indices_df = pd.DataFrame(columns=['Fold', 'Train_Indices', 'Val_Indices'])

    # Cross-validation
    kf = KFold(n_splits=data_parts)
    fold_metrics = []
    # To accumulate metrics across folds for each epoch
    train_acc_epoch = []
    val_acc_epoch = []
    test_acc_epoch = []

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    
    config["output_size"] = prepared_labels.shape[-1]
    writer = SummaryWriter(log_dir=os.path.join(log_dir,config["model_name"]))

    for fold, (train_index, val_index) in enumerate(kf.split(prepared_data)):
        print(f"Fold {fold + 1}/{data_parts}")
        best_model_path = f"{run_dir}/best_model_fold{fold+1}"
        # Save fold indices
        fold_indices_df = fold_indices_df._append({
            'Fold': fold + 1,
            'Train_Indices': train_index.tolist(),
            'Val_Indices': val_index.tolist()
        }, ignore_index=True)

        # Split data
        train_d, val_d = prepared_data[train_index], prepared_data[val_index]
        train_lbs, val_lbs = prepared_labels[train_index], prepared_labels[val_index]
        train_timesteps, val_timesteps = prepared_labels_timesteps[train_index], prepared_labels_timesteps[val_index]
        
        # Reshape data
        train_d, train_lbs = reshaping_data_for_model(train_d, train_lbs)
        val_d, val_lbs = reshaping_data_for_model(val_d, val_lbs)
        test_d, test_lbs = reshaping_data_for_model(prepared_test_data, prepared_test_labels)
        
        print(train_d.shape)

        # Create datasets
        #train_dataset = BreathingDataset(train_d, train_lbs, processor, window_size, step_sequence, augment=True)
        train_dataset = BreathingDataset(train_d, train_lbs, processor,window_size, step_sequence)
        val_dataset = BreathingDataset(val_d, val_lbs, processor, window_size, step_sequence)
        test_dataset = BreathingDataset(test_d, test_lbs, processor, window_size, step_sequence)

        # Create DataLoaders
        train_loader = DataLoader(train_dataset, batch_size=batch_size,num_workers=2, shuffle=True, collate_fn=train_dataset.collate_fn)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, num_workers=1, collate_fn=val_dataset.collate_fn)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, num_workers=1, collate_fn=test_dataset.collate_fn)
        print(config["output_size"])
        # Create and initialize model
        model = config["model"](config).to(device)
        
        #### training optimiser parameters fror apple
        #learning_rate = 0.005 
        #optimizer = optim.Adam(model.parameters(), lr=learning_rate)

        #### training optimiser parameters fror harma_2023 VRB model 
        learning_rate = 0.01 # From the paper
        optimizer = optim.Adam(model.parameters())       
        
        best_val_loss = float('inf')
        best_val_loss_flat = float('inf')
        early_stopping_counter = 0
        # To accumulate metrics across folds for each epoch
        train_acc = []
        val_acc = []
        test_acc = []
        for epoch in range(epochs):

            model.train()
            train_loss = 0.0
            progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")
            
            for batch_d, batch_lbs in progress_bar:
                optimizer.zero_grad()
                input_values = batch_d.to(device)
                batch_lbs = batch_lbs.to(device)
                outputs = model(input_values)
                loss = correlation_coefficient_loss(outputs, batch_lbs)
                loss.backward()
                optimizer.step()
                train_loss += loss.item()
                progress_bar.set_postfix({'train_loss': f'{train_loss/(progress_bar.n+1):.4f}'})


            train_loss /= len(train_loader)

            # Combined validation loop
            model.eval()
            val_loss = 0.0
            val_pred = []
            with torch.no_grad():
                for batch_d, batch_lbs in val_loader:
                    input_values = batch_d.to(device)
                    batch_lbs = batch_lbs.to(device)
                    
                    outputs = model(input_values)
                    loss = correlation_coefficient_loss(outputs, batch_lbs)
                    val_loss += loss.item()
                    val_pred.extend(outputs.cpu().numpy())

            val_loss /= len(val_loader)

            # Calculate validation metrics
            val_pred = np.array(val_pred).reshape(val_timesteps.shape)
            val_ground_truth = _get_ground_truth_labels([all_dict[i] for i in val_index], all_labels)
            val_pred_flat = unsplit_data_ogsize(val_pred, window_size, step_sequence, 25, val_ground_truth.shape[-1])
            val_prc_coef = _calculate_flattened_accuracy(val_pred_flat, val_ground_truth)
            
            # Accumulate metrics for this fold and epoch
            train_acc.append(1- train_loss)
            val_acc.append(1- val_loss)

            # Log metrics
            writer.add_scalar(f"Loss/train_fold_{fold + 1}", train_loss, epoch)
            writer.add_scalar(f"Loss/val_fold_{fold + 1}", val_loss, epoch)
            writer.add_scalar(f"Pearson/val_fold_{fold + 1}", val_prc_coef, epoch)

            print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Pearson: {val_prc_coef:.4f}")

            # Check if validation loss improved
            if val_loss < best_val_loss:
                print(f"Validation loss improved from {best_val_loss:.4f} to {val_loss:.4f}. Saving best model...")
                best_val_loss = val_loss
                best_val_loss_flat = val_prc_coef
                early_stopping_counter = 0

                # Save the best model
                torch.save(model.state_dict(), best_model_path)
            else:
                early_stopping_counter += 1
                print(f"Validation loss did not improve for {early_stopping_counter} epochs.")
                #model.load_state_dict(torch.load(best_model_path))


            # Early stopping
            if early_stopping_counter >= early_stopping_patience:
                print(f"Early stopping triggered at epoch {epoch + 1}. Loading best model.")
                # Load the best model's weights
                model.load_state_dict(torch.load(best_model_path))
                break

        test_pred = []
        test_loss = 0.0
        with torch.no_grad():
            for batch_d, batch_lbs in test_loader:
                input_values = batch_d.to(device)
                batch_lbs = batch_lbs.to(device)
                
                outputs = model(input_values)
                loss = correlation_coefficient_loss(outputs, batch_lbs)
                test_loss += loss.item()
                test_pred.extend(outputs.cpu().numpy())

        test_loss /= len(test_loader)
        test_pred = np.array(test_pred).reshape(prepared_test_labels_timesteps.shape)
        test_ground_truth = _get_ground_truth_labels(list(test_dict.values()), test_labels)
        test_pred_flat = unsplit_data_ogsize(test_pred, window_size, step_sequence, 25, test_ground_truth.shape[-1])
        test_prc_coef = _calculate_flattened_accuracy(test_pred_flat, test_ground_truth)

        print(f"Fold {fold + 1}:")
        print(f"  Validation Pearson Coefficient  acc: {1- best_val_loss}")
        print(f"  Validation Pearson Coefficient flat acc: {best_val_loss_flat}")
        print(f"  Test acc: {1- test_loss}")
        print(f"  Test Pearson Coefficient acc(flattened): {test_prc_coef}")

        fold_metrics.append({
            'Fold': fold + 1,
            'val_prc_acc': 1- best_val_loss,
            'val_prc_acc_flat': best_val_loss_flat,
            'test_acc': 1- test_loss,
            'test_prc_flat': test_prc_coef
        })


                # Log fold-specific metrics as tables
        fold_table = f"| Fold | Val Pearson Acc | Val Pearson Flat | Test Acc | Test Pearson Flat |\n" \
                     f"|------|-----------------|------------------|----------|-------------------|\n" \
                     f"| {fold + 1} | {1 - val_loss:.4f} | {val_prc_coef:.4f} | {1 - test_loss:.4f} | {test_prc_coef:.4f} |\n"
        writer.add_text(f"Fold_{fold + 1}_Metrics", fold_table)
        # Accumulate fold metrics across all folds
        train_acc_epoch.append(train_acc)
        val_acc_epoch.append(train_acc)


    
        # After all folds, compute and log the average metrics per epoch across all folds
    for epoch in range(epochs):
        avg_train_loss = np.mean([fold_losses[epoch] for fold_losses in train_acc_epoch if len(fold_losses) > epoch])
        avg_val_loss = np.mean([fold_losses[epoch] for fold_losses in val_acc_epoch if len(fold_losses) > epoch])

        # Log the averaged metrics for the epoch across all folds
        writer.add_scalar("Average_acc/train", avg_train_loss, epoch)
        writer.add_scalar("Average_acc/val", avg_val_loss, epoch)
            

    # Calculate average metrics
    avg_metrics = {key: np.mean([fold[key] for fold in fold_metrics if key != 'Fold']) for key in fold_metrics[0].keys() if key != 'Fold'}
        # Log the final average table
    avg_table = "| Fold | Val Pearson Acc | Val Pearson Flat | Test Acc | Test Pearson Flat |\n" \
                "|------|-----------------|------------------|----------|-------------------|\n" \
                f"| Average | {avg_metrics['val_prc_acc']:.4f} | {avg_metrics['val_prc_acc_flat']:.4f} | {avg_metrics['test_acc']:.4f} | {avg_metrics['test_prc_flat']:.4f} |\n"
    writer.add_text("Average_Metrics", avg_table)
    # Add average metrics to results
    avg_metrics['Fold'] = 'Average'
    fold_metrics.append(avg_metrics)

    # save averga date to CSV
    results_df = pd.DataFrame(fold_metrics)
    csv_path = os.path.join(run_dir, 'fold_results.csv')
    results_df.to_csv(csv_path, index=False)
    
    # Save fold indices CSV
    fold_indices_df.to_csv(os.path.join(run_dir, 'fold_indices.csv'), index=False)
    
    writer.close()


    print("\nTraining completed.")
    print("Average metrics across all folds:")
    for key, value in avg_metrics.items():
        print(f"  {key}: {value}")


if __name__ == "__main__":
    ## Path to data
    #path = "/home/glenn/Downloads/"
    path = "../DATA/"


    # Model parameters
    model_config = {
        "VRBModel": {
            "model" : VRBModel,
            "model_name": "facebook/hubert-large-ll60k",
            "hidden_units": 64,
            "n_gru": 3,
            "output_size": None  # Will be set dynamically
        },
        "Wav2Vec2ConvLSTMModel": {
            "model" : Wav2Vec2ConvLSTMModel,
            "model_name": "facebook/wav2vec2-base",
            "hidden_units": 128,
            "n_lstm": 2,
            "output_size": None  # Will be set dynamically
        },
            "RespBertCNNModelV2": {
            'model' : RespBertCNNModelV2,
            "model_name": "microsoft/wavlm-large",
            "hidden_units": 128,
            "output_size": None  
        },
            "RespBertLSTMModelV2": {
            'model': RespBertLSTMModelV2,
            "model_name": "microsoft/wavlm-large",
            "hidden_units": 128,
            "n_lstm": 3,
            "output_size": None  
        }
            ,
            "RespBertLSTMCNNTransformerModel": {
            'model': RespBertLSTMCNNTransformerModel,
            "model_name": "microsoft/wavlm-large",
            "hidden_units": 1024,
            "n_lstm": 2,
            "output_size": None  
        }
    }
    

    
    # Train and data parameters
    epochs = 60
    batch_size = 10
    window_size = 30
    step_size = 6
    data_parts = 4 # aka folds
    early_stopping_patience = 10
    
    config = model_config["RespBertLSTMCNNTransformerModel"]
    

    ## same wav2vec2 base model and pipeline used in the paper
    #processor = AutoProcessor.from_pretrained(config["model_name"])
    processor = Wav2Vec2FeatureExtractor.from_pretrained(config["model_name"])

    train(
        path_to_data=path+"ComParE2020_Breathing/wav/",
        path_to_labels=path+"ComParE2020_Breathing/lab/",
        window_size=window_size,
        batch_size=batch_size,
        config = config,
        step_size=step_size,
        data_parts= data_parts ,
        early_stopping_patience= early_stopping_patience,
        epochs= epochs,
        processor = processor
    )

Fold 1/4
(864, 480000)
750


/home/gdwildt/.conda/envs/thesis/lib/python3.11/site-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
Epoch 1/60:   0%|          | 0/87 [00:00<?, ?it/s]/home/gdwildt/.conda/envs/thesis/lib/python3.11/site-packages/torch/nn/functional.py:5193: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
Epoch 1/60:   0%|          | 0/87 [00:02<?, ?it/s]

torch.Size([10, 1499, 1024])
torch.Size([10, 1499, 3072])
torch.Size([10, 1499, 3072])


AssertionError: was expecting embedding dimension of 3072, but got 1499

In [ ]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import KFold
from datetime import datetime
#from torch.utils.tensorboard import SummaryWriter
import torchaudio
from typing import List, Tuple
from pt_utils import *
from pt_dataset import *
from pt_models import *
from pt_utils import *
from tensorboardX import SummaryWriter
from transformers import get_cosine_schedule_with_warmup


def create_run_directory():
    base_dir = "pt_runs_SOTA"
    timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
    run_dir = os.path.join(base_dir, timestamp)
    os.makedirs(run_dir, exist_ok=True)
    return run_dir

def _calculate_flattened_accuracy(average, ground_truth_labels):
    s_acc = 0
    for b in range(len(ground_truth_labels)):
        s, _ = scipy.stats.pearsonr(average[b], ground_truth_labels[b])
        s_acc += s
    return s_acc / len(ground_truth_labels)

def _choose_real_labs_only_with_filenames(labels, filenames):
    return labels[labels['filename'].isin(filenames)]

def _get_ground_truth_labels(ground_truth_names, labels):
    ground_truth_labels = []
    for batch_name in ground_truth_names:
        ground_truth_label = _choose_real_labs_only_with_filenames(labels, [batch_name])
        ground_truth_labels.append(ground_truth_label)
    return np.array(ground_truth_labels)[:, :, -1].astype(np.float32)

def train(path_to_data, path_to_labels, window_size=16, step_size=6, data_parts=4, epochs=100, batch_size=10, early_stopping_patience=20, config = None, processor = None):
    run_dir = create_run_directory()
    log_dir = os.path.join(run_dir, "logs")
    os.makedirs(log_dir, exist_ok=True)

    # Parameters
    length_sequence = window_size 
    step_sequence = step_size

    # Load and prepare data
    train_data, train_labels, train_dict, frame_rate = load_data(path_to_data, path_to_labels, 'train')
    devel_data, devel_labels, devel_dict, frame_rate = load_data(path_to_data, path_to_labels, 'devel')
    test_data, test_labels, test_dict, frame_rate = load_data(path_to_data, path_to_labels, 'test')

    # Combine train and devel data
    all_data = np.concatenate((train_data, devel_data), axis=0)
    all_labels = pd.concat([train_labels, devel_labels])
    all_dict = np.concatenate((list(train_dict.values()), list(devel_dict.values())), axis=0)

    # Prepare data
    prepared_data, prepared_labels, prepared_labels_timesteps = prepare_data(all_data, all_labels, all_dict, frame_rate, length_sequence * 16000, step_sequence * 16000)
    prepared_test_data, prepared_test_labels, prepared_test_labels_timesteps = prepare_data(test_data, test_labels, test_dict, frame_rate, length_sequence * 16000, step_sequence * 16000)

    # Create CSV file for storing fold indices
    fold_indices_df = pd.DataFrame(columns=['Fold', 'Train_Indices', 'Val_Indices'])

    # Cross-validation
    kf = KFold(n_splits=data_parts)
    fold_metrics = []
    # To accumulate metrics across folds for each epoch
    train_acc_epoch = []
    val_acc_epoch = []
    test_acc_epoch = []

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    
    config["output_size"] = prepared_labels.shape[-1]
    writer = SummaryWriter(log_dir=os.path.join(log_dir,config["model_name"]))

    for fold, (train_index, val_index) in enumerate(kf.split(prepared_data)):
        print(f"Fold {fold + 1}/{data_parts}")
        best_model_path = f"{run_dir}/best_model_fold{fold+1}"
        # Save fold indices
        fold_indices_df = fold_indices_df._append({
            'Fold': fold + 1,
            'Train_Indices': train_index.tolist(),
            'Val_Indices': val_index.tolist()
        }, ignore_index=True)

        # Split data
        train_d, val_d = prepared_data[train_index], prepared_data[val_index]
        train_lbs, val_lbs = prepared_labels[train_index], prepared_labels[val_index]
        train_timesteps, val_timesteps = prepared_labels_timesteps[train_index], prepared_labels_timesteps[val_index]
        
        # Reshape data
        train_d, train_lbs = reshaping_data_for_model(train_d, train_lbs)
        val_d, val_lbs = reshaping_data_for_model(val_d, val_lbs)
        test_d, test_lbs = reshaping_data_for_model(prepared_test_data, prepared_test_labels)
        
        print(train_d.shape)

        # Create datasets
        #train_dataset = BreathingDataset(train_d, train_lbs, processor, window_size, step_sequence, augment=True)
        train_dataset = BreathingDataset(train_d, train_lbs, processor,window_size, step_sequence)
        val_dataset = BreathingDataset(val_d, val_lbs, processor, window_size, step_sequence)
        test_dataset = BreathingDataset(test_d, test_lbs, processor, window_size, step_sequence)

        # Create DataLoaders
        train_loader = DataLoader(train_dataset, batch_size=batch_size,num_workers=2, shuffle=True, collate_fn=train_dataset.collate_fn)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, num_workers=1, collate_fn=val_dataset.collate_fn)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, num_workers=1, collate_fn=test_dataset.collate_fn)
        print(config["output_size"])
        # Create and initialize model
        model = config["model"](config).to(device)
        
        #### training optimiser parameters fror apple
        #learning_rate = 0.005 
        #optimizer = optim.Adam(model.parameters(), lr=learning_rate)

        #### training optimiser parameters fror harma_2023 VRB model 
        learning_rate = 0.01 # From the paper
        optimizer = optim.Adam(model.parameters())       
        
        best_val_loss = float('inf')
        best_val_loss_flat = float('inf')
        early_stopping_counter = 0
        # To accumulate metrics across folds for each epoch
        train_acc = []
        val_acc = []
        test_acc = []
        for epoch in range(epochs):

            model.train()
            train_loss = 0.0
            progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")
            
            for batch_d, batch_lbs in progress_bar:
                optimizer.zero_grad()
                input_values = batch_d.to(device)
                batch_lbs = batch_lbs.to(device)
                outputs = model(input_values)
                loss = correlation_coefficient_loss(outputs, batch_lbs)
                loss.backward()
                optimizer.step()
                train_loss += loss.item()
                progress_bar.set_postfix({'train_loss': f'{train_loss/(progress_bar.n+1):.4f}'})


            train_loss /= len(train_loader)

            # Combined validation loop
            model.eval()
            val_loss = 0.0
            val_pred = []
            with torch.no_grad():
                for batch_d, batch_lbs in val_loader:
                    input_values = batch_d.to(device)
                    batch_lbs = batch_lbs.to(device)
                    
                    outputs = model(input_values)
                    loss = correlation_coefficient_loss(outputs, batch_lbs)
                    val_loss += loss.item()
                    val_pred.extend(outputs.cpu().numpy())

            val_loss /= len(val_loader)

            # Calculate validation metrics
            val_pred = np.array(val_pred).reshape(val_timesteps.shape)
            val_ground_truth = _get_ground_truth_labels([all_dict[i] for i in val_index], all_labels)
            val_pred_flat = unsplit_data_ogsize(val_pred, window_size, step_sequence, 25, val_ground_truth.shape[-1])
            val_prc_coef = _calculate_flattened_accuracy(val_pred_flat, val_ground_truth)
            
            # Accumulate metrics for this fold and epoch
            train_acc.append(1- train_loss)
            val_acc.append(1- val_loss)

            # Log metrics
            writer.add_scalar(f"Loss/train_fold_{fold + 1}", train_loss, epoch)
            writer.add_scalar(f"Loss/val_fold_{fold + 1}", val_loss, epoch)
            writer.add_scalar(f"Pearson/val_fold_{fold + 1}", val_prc_coef, epoch)

            print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Pearson: {val_prc_coef:.4f}")

            # Check if validation loss improved
            if val_loss < best_val_loss:
                print(f"Validation loss improved from {best_val_loss:.4f} to {val_loss:.4f}. Saving best model...")
                best_val_loss = val_loss
                best_val_loss_flat = val_prc_coef
                early_stopping_counter = 0

                # Save the best model
                torch.save(model.state_dict(), best_model_path)
            else:
                early_stopping_counter += 1
                print(f"Validation loss did not improve for {early_stopping_counter} epochs.")
                #model.load_state_dict(torch.load(best_model_path))


            # Early stopping
            if early_stopping_counter >= early_stopping_patience:
                print(f"Early stopping triggered at epoch {epoch + 1}. Loading best model.")
                # Load the best model's weights
                model.load_state_dict(torch.load(best_model_path))
                break

        test_pred = []
        test_loss = 0.0
        with torch.no_grad():
            for batch_d, batch_lbs in test_loader:
                input_values = batch_d.to(device)
                batch_lbs = batch_lbs.to(device)
                
                outputs = model(input_values)
                loss = correlation_coefficient_loss(outputs, batch_lbs)
                test_loss += loss.item()
                test_pred.extend(outputs.cpu().numpy())

        test_loss /= len(test_loader)
        test_pred = np.array(test_pred).reshape(prepared_test_labels_timesteps.shape)
        test_ground_truth = _get_ground_truth_labels(list(test_dict.values()), test_labels)
        test_pred_flat = unsplit_data_ogsize(test_pred, window_size, step_sequence, 25, test_ground_truth.shape[-1])
        test_prc_coef = _calculate_flattened_accuracy(test_pred_flat, test_ground_truth)

        print(f"Fold {fold + 1}:")
        print(f"  Validation Pearson Coefficient  acc: {1- best_val_loss}")
        print(f"  Validation Pearson Coefficient flat acc: {best_val_loss_flat}")
        print(f"  Test acc: {1- test_loss}")
        print(f"  Test Pearson Coefficient acc(flattened): {test_prc_coef}")

        fold_metrics.append({
            'Fold': fold + 1,
            'val_prc_acc': 1- best_val_loss,
            'val_prc_acc_flat': best_val_loss_flat,
            'test_acc': 1- test_loss,
            'test_prc_flat': test_prc_coef
        })


                # Log fold-specific metrics as tables
        fold_table = f"| Fold | Val Pearson Acc | Val Pearson Flat | Test Acc | Test Pearson Flat |\n" \
                     f"|------|-----------------|------------------|----------|-------------------|\n" \
                     f"| {fold + 1} | {1 - val_loss:.4f} | {val_prc_coef:.4f} | {1 - test_loss:.4f} | {test_prc_coef:.4f} |\n"
        writer.add_text(f"Fold_{fold + 1}_Metrics", fold_table)
        # Accumulate fold metrics across all folds
        train_acc_epoch.append(train_acc)
        val_acc_epoch.append(train_acc)


    
        # After all folds, compute and log the average metrics per epoch across all folds
    for epoch in range(epochs):
        avg_train_loss = np.mean([fold_losses[epoch] for fold_losses in train_acc_epoch if len(fold_losses) > epoch])
        avg_val_loss = np.mean([fold_losses[epoch] for fold_losses in val_acc_epoch if len(fold_losses) > epoch])

        # Log the averaged metrics for the epoch across all folds
        writer.add_scalar("Average_acc/train", avg_train_loss, epoch)
        writer.add_scalar("Average_acc/val", avg_val_loss, epoch)
            

    # Calculate average metrics
    avg_metrics = {key: np.mean([fold[key] for fold in fold_metrics if key != 'Fold']) for key in fold_metrics[0].keys() if key != 'Fold'}
        # Log the final average table
    avg_table = "| Fold | Val Pearson Acc | Val Pearson Flat | Test Acc | Test Pearson Flat |\n" \
                "|------|-----------------|------------------|----------|-------------------|\n" \
                f"| Average | {avg_metrics['val_prc_acc']:.4f} | {avg_metrics['val_prc_acc_flat']:.4f} | {avg_metrics['test_acc']:.4f} | {avg_metrics['test_prc_flat']:.4f} |\n"
    writer.add_text("Average_Metrics", avg_table)
    # Add average metrics to results
    avg_metrics['Fold'] = 'Average'
    fold_metrics.append(avg_metrics)

    # save averga date to CSV
    results_df = pd.DataFrame(fold_metrics)
    csv_path = os.path.join(run_dir, 'fold_results.csv')
    results_df.to_csv(csv_path, index=False)
    
    # Save fold indices CSV
    fold_indices_df.to_csv(os.path.join(run_dir, 'fold_indices.csv'), index=False)
    
    writer.close()


    print("\nTraining completed.")
    print("Average metrics across all folds:")
    for key, value in avg_metrics.items():
        print(f"  {key}: {value}")


if __name__ == "__main__":
    ## Path to data
    #path = "/home/glenn/Downloads/"
    path = "../DATA/"


    # Model parameters
    model_config = {
        "VRBModel": {
            "model" : VRBModel,
            "model_name": "facebook/hubert-large-ll60k",
            "hidden_units": 64,
            "n_gru": 3,
            "output_size": None  # Will be set dynamically
        },
        "Wav2Vec2ConvLSTMModel": {
            "model" : Wav2Vec2ConvLSTMModel,
            "model_name": "facebook/wav2vec2-base",
            "hidden_units": 128,
            "n_lstm": 2,
            "output_size": None  # Will be set dynamically
        },
            "RespBertCNNModelV2": {
            'model' : RespBertCNNModelV2,
            "model_name": "microsoft/wavlm-large",
            "hidden_units": 128,
            "output_size": None  
        },
            "RespBertLSTMModelV2": {
            'model': RespBertLSTMModelV2,
            "model_name": "microsoft/wavlm-large",
            "hidden_units": 128,
            "n_lstm": 3,
            "output_size": None  
        }
            ,
            "RespBertLSTMCNNTransformerModel": {
            'model': RespBertLSTMCNNTransformerModel,
            "model_name": "microsoft/wavlm-large",
            "hidden_units": 1024,
            "n_lstm": 2,
            "output_size": None  
        }
    }
    

    
    # Train and data parameters
    epochs = 60
    batch_size = 10
    window_size = 30
    step_size = 6
    data_parts = 4 # aka folds
    early_stopping_patience = 10
    
    config = model_config["RespBertLSTMCNNTransformerModel"]
    

    ## same wav2vec2 base model and pipeline used in the paper
    #processor = AutoProcessor.from_pretrained(config["model_name"])
    processor = Wav2Vec2FeatureExtractor.from_pretrained(config["model_name"])

    train(
        path_to_data=path+"ComParE2020_Breathing/wav/",
        path_to_labels=path+"ComParE2020_Breathing/lab/",
        window_size=window_size,
        batch_size=batch_size,
        config = config,
        step_size=step_size,
        data_parts= data_parts ,
        early_stopping_patience= early_stopping_patience,
        epochs= epochs,
        processor = processor
    )

Fold 1/4
(864, 480000)
750


/home/gdwildt/.conda/envs/thesis/lib/python3.11/site-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
Epoch 1/60:   0%|          | 0/87 [00:00<?, ?it/s]/home/gdwildt/.conda/envs/thesis/lib/python3.11/site-packages/torch/nn/functional.py:5193: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
Epoch 1/60:   0%|          | 0/87 [00:02<?, ?it/s]

torch.Size([10, 1499, 1024])
torch.Size([10, 1499, 3072])
torch.Size([10, 1499, 3072])


TypeError: transpose() received an invalid combination of arguments - got (int, int, int), but expected one of:
 * (int dim0, int dim1)
 * (name dim0, name dim1)


In [ ]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import KFold
from datetime import datetime
#from torch.utils.tensorboard import SummaryWriter
import torchaudio
from typing import List, Tuple
from pt_utils import *
from pt_dataset import *
from pt_models import *
from pt_utils import *
from tensorboardX import SummaryWriter
from transformers import get_cosine_schedule_with_warmup


def create_run_directory():
    base_dir = "pt_runs_SOTA"
    timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
    run_dir = os.path.join(base_dir, timestamp)
    os.makedirs(run_dir, exist_ok=True)
    return run_dir

def _calculate_flattened_accuracy(average, ground_truth_labels):
    s_acc = 0
    for b in range(len(ground_truth_labels)):
        s, _ = scipy.stats.pearsonr(average[b], ground_truth_labels[b])
        s_acc += s
    return s_acc / len(ground_truth_labels)

def _choose_real_labs_only_with_filenames(labels, filenames):
    return labels[labels['filename'].isin(filenames)]

def _get_ground_truth_labels(ground_truth_names, labels):
    ground_truth_labels = []
    for batch_name in ground_truth_names:
        ground_truth_label = _choose_real_labs_only_with_filenames(labels, [batch_name])
        ground_truth_labels.append(ground_truth_label)
    return np.array(ground_truth_labels)[:, :, -1].astype(np.float32)

def train(path_to_data, path_to_labels, window_size=16, step_size=6, data_parts=4, epochs=100, batch_size=10, early_stopping_patience=20, config = None, processor = None):
    run_dir = create_run_directory()
    log_dir = os.path.join(run_dir, "logs")
    os.makedirs(log_dir, exist_ok=True)

    # Parameters
    length_sequence = window_size 
    step_sequence = step_size

    # Load and prepare data
    train_data, train_labels, train_dict, frame_rate = load_data(path_to_data, path_to_labels, 'train')
    devel_data, devel_labels, devel_dict, frame_rate = load_data(path_to_data, path_to_labels, 'devel')
    test_data, test_labels, test_dict, frame_rate = load_data(path_to_data, path_to_labels, 'test')

    # Combine train and devel data
    all_data = np.concatenate((train_data, devel_data), axis=0)
    all_labels = pd.concat([train_labels, devel_labels])
    all_dict = np.concatenate((list(train_dict.values()), list(devel_dict.values())), axis=0)

    # Prepare data
    prepared_data, prepared_labels, prepared_labels_timesteps = prepare_data(all_data, all_labels, all_dict, frame_rate, length_sequence * 16000, step_sequence * 16000)
    prepared_test_data, prepared_test_labels, prepared_test_labels_timesteps = prepare_data(test_data, test_labels, test_dict, frame_rate, length_sequence * 16000, step_sequence * 16000)

    # Create CSV file for storing fold indices
    fold_indices_df = pd.DataFrame(columns=['Fold', 'Train_Indices', 'Val_Indices'])

    # Cross-validation
    kf = KFold(n_splits=data_parts)
    fold_metrics = []
    # To accumulate metrics across folds for each epoch
    train_acc_epoch = []
    val_acc_epoch = []
    test_acc_epoch = []

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    
    config["output_size"] = prepared_labels.shape[-1]
    writer = SummaryWriter(log_dir=os.path.join(log_dir,config["model_name"]))

    for fold, (train_index, val_index) in enumerate(kf.split(prepared_data)):
        print(f"Fold {fold + 1}/{data_parts}")
        best_model_path = f"{run_dir}/best_model_fold{fold+1}"
        # Save fold indices
        fold_indices_df = fold_indices_df._append({
            'Fold': fold + 1,
            'Train_Indices': train_index.tolist(),
            'Val_Indices': val_index.tolist()
        }, ignore_index=True)

        # Split data
        train_d, val_d = prepared_data[train_index], prepared_data[val_index]
        train_lbs, val_lbs = prepared_labels[train_index], prepared_labels[val_index]
        train_timesteps, val_timesteps = prepared_labels_timesteps[train_index], prepared_labels_timesteps[val_index]
        
        # Reshape data
        train_d, train_lbs = reshaping_data_for_model(train_d, train_lbs)
        val_d, val_lbs = reshaping_data_for_model(val_d, val_lbs)
        test_d, test_lbs = reshaping_data_for_model(prepared_test_data, prepared_test_labels)
        
        print(train_d.shape)

        # Create datasets
        #train_dataset = BreathingDataset(train_d, train_lbs, processor, window_size, step_sequence, augment=True)
        train_dataset = BreathingDataset(train_d, train_lbs, processor,window_size, step_sequence)
        val_dataset = BreathingDataset(val_d, val_lbs, processor, window_size, step_sequence)
        test_dataset = BreathingDataset(test_d, test_lbs, processor, window_size, step_sequence)

        # Create DataLoaders
        train_loader = DataLoader(train_dataset, batch_size=batch_size,num_workers=2, shuffle=True, collate_fn=train_dataset.collate_fn)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, num_workers=1, collate_fn=val_dataset.collate_fn)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, num_workers=1, collate_fn=test_dataset.collate_fn)
        print(config["output_size"])
        # Create and initialize model
        model = config["model"](config).to(device)
        
        #### training optimiser parameters fror apple
        #learning_rate = 0.005 
        #optimizer = optim.Adam(model.parameters(), lr=learning_rate)

        #### training optimiser parameters fror harma_2023 VRB model 
        learning_rate = 0.01 # From the paper
        optimizer = optim.Adam(model.parameters())       
        
        best_val_loss = float('inf')
        best_val_loss_flat = float('inf')
        early_stopping_counter = 0
        # To accumulate metrics across folds for each epoch
        train_acc = []
        val_acc = []
        test_acc = []
        for epoch in range(epochs):

            model.train()
            train_loss = 0.0
            progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")
            
            for batch_d, batch_lbs in progress_bar:
                optimizer.zero_grad()
                input_values = batch_d.to(device)
                batch_lbs = batch_lbs.to(device)
                outputs = model(input_values)
                loss = correlation_coefficient_loss(outputs, batch_lbs)
                loss.backward()
                optimizer.step()
                train_loss += loss.item()
                progress_bar.set_postfix({'train_loss': f'{train_loss/(progress_bar.n+1):.4f}'})


            train_loss /= len(train_loader)

            # Combined validation loop
            model.eval()
            val_loss = 0.0
            val_pred = []
            with torch.no_grad():
                for batch_d, batch_lbs in val_loader:
                    input_values = batch_d.to(device)
                    batch_lbs = batch_lbs.to(device)
                    
                    outputs = model(input_values)
                    loss = correlation_coefficient_loss(outputs, batch_lbs)
                    val_loss += loss.item()
                    val_pred.extend(outputs.cpu().numpy())

            val_loss /= len(val_loader)

            # Calculate validation metrics
            val_pred = np.array(val_pred).reshape(val_timesteps.shape)
            val_ground_truth = _get_ground_truth_labels([all_dict[i] for i in val_index], all_labels)
            val_pred_flat = unsplit_data_ogsize(val_pred, window_size, step_sequence, 25, val_ground_truth.shape[-1])
            val_prc_coef = _calculate_flattened_accuracy(val_pred_flat, val_ground_truth)
            
            # Accumulate metrics for this fold and epoch
            train_acc.append(1- train_loss)
            val_acc.append(1- val_loss)

            # Log metrics
            writer.add_scalar(f"Loss/train_fold_{fold + 1}", train_loss, epoch)
            writer.add_scalar(f"Loss/val_fold_{fold + 1}", val_loss, epoch)
            writer.add_scalar(f"Pearson/val_fold_{fold + 1}", val_prc_coef, epoch)

            print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Pearson: {val_prc_coef:.4f}")

            # Check if validation loss improved
            if val_loss < best_val_loss:
                print(f"Validation loss improved from {best_val_loss:.4f} to {val_loss:.4f}. Saving best model...")
                best_val_loss = val_loss
                best_val_loss_flat = val_prc_coef
                early_stopping_counter = 0

                # Save the best model
                torch.save(model.state_dict(), best_model_path)
            else:
                early_stopping_counter += 1
                print(f"Validation loss did not improve for {early_stopping_counter} epochs.")
                #model.load_state_dict(torch.load(best_model_path))


            # Early stopping
            if early_stopping_counter >= early_stopping_patience:
                print(f"Early stopping triggered at epoch {epoch + 1}. Loading best model.")
                # Load the best model's weights
                model.load_state_dict(torch.load(best_model_path))
                break

        test_pred = []
        test_loss = 0.0
        with torch.no_grad():
            for batch_d, batch_lbs in test_loader:
                input_values = batch_d.to(device)
                batch_lbs = batch_lbs.to(device)
                
                outputs = model(input_values)
                loss = correlation_coefficient_loss(outputs, batch_lbs)
                test_loss += loss.item()
                test_pred.extend(outputs.cpu().numpy())

        test_loss /= len(test_loader)
        test_pred = np.array(test_pred).reshape(prepared_test_labels_timesteps.shape)
        test_ground_truth = _get_ground_truth_labels(list(test_dict.values()), test_labels)
        test_pred_flat = unsplit_data_ogsize(test_pred, window_size, step_sequence, 25, test_ground_truth.shape[-1])
        test_prc_coef = _calculate_flattened_accuracy(test_pred_flat, test_ground_truth)

        print(f"Fold {fold + 1}:")
        print(f"  Validation Pearson Coefficient  acc: {1- best_val_loss}")
        print(f"  Validation Pearson Coefficient flat acc: {best_val_loss_flat}")
        print(f"  Test acc: {1- test_loss}")
        print(f"  Test Pearson Coefficient acc(flattened): {test_prc_coef}")

        fold_metrics.append({
            'Fold': fold + 1,
            'val_prc_acc': 1- best_val_loss,
            'val_prc_acc_flat': best_val_loss_flat,
            'test_acc': 1- test_loss,
            'test_prc_flat': test_prc_coef
        })


                # Log fold-specific metrics as tables
        fold_table = f"| Fold | Val Pearson Acc | Val Pearson Flat | Test Acc | Test Pearson Flat |\n" \
                     f"|------|-----------------|------------------|----------|-------------------|\n" \
                     f"| {fold + 1} | {1 - val_loss:.4f} | {val_prc_coef:.4f} | {1 - test_loss:.4f} | {test_prc_coef:.4f} |\n"
        writer.add_text(f"Fold_{fold + 1}_Metrics", fold_table)
        # Accumulate fold metrics across all folds
        train_acc_epoch.append(train_acc)
        val_acc_epoch.append(train_acc)


    
        # After all folds, compute and log the average metrics per epoch across all folds
    for epoch in range(epochs):
        avg_train_loss = np.mean([fold_losses[epoch] for fold_losses in train_acc_epoch if len(fold_losses) > epoch])
        avg_val_loss = np.mean([fold_losses[epoch] for fold_losses in val_acc_epoch if len(fold_losses) > epoch])

        # Log the averaged metrics for the epoch across all folds
        writer.add_scalar("Average_acc/train", avg_train_loss, epoch)
        writer.add_scalar("Average_acc/val", avg_val_loss, epoch)
            

    # Calculate average metrics
    avg_metrics = {key: np.mean([fold[key] for fold in fold_metrics if key != 'Fold']) for key in fold_metrics[0].keys() if key != 'Fold'}
        # Log the final average table
    avg_table = "| Fold | Val Pearson Acc | Val Pearson Flat | Test Acc | Test Pearson Flat |\n" \
                "|------|-----------------|------------------|----------|-------------------|\n" \
                f"| Average | {avg_metrics['val_prc_acc']:.4f} | {avg_metrics['val_prc_acc_flat']:.4f} | {avg_metrics['test_acc']:.4f} | {avg_metrics['test_prc_flat']:.4f} |\n"
    writer.add_text("Average_Metrics", avg_table)
    # Add average metrics to results
    avg_metrics['Fold'] = 'Average'
    fold_metrics.append(avg_metrics)

    # save averga date to CSV
    results_df = pd.DataFrame(fold_metrics)
    csv_path = os.path.join(run_dir, 'fold_results.csv')
    results_df.to_csv(csv_path, index=False)
    
    # Save fold indices CSV
    fold_indices_df.to_csv(os.path.join(run_dir, 'fold_indices.csv'), index=False)
    
    writer.close()


    print("\nTraining completed.")
    print("Average metrics across all folds:")
    for key, value in avg_metrics.items():
        print(f"  {key}: {value}")


if __name__ == "__main__":
    ## Path to data
    #path = "/home/glenn/Downloads/"
    path = "../DATA/"


    # Model parameters
    model_config = {
        "VRBModel": {
            "model" : VRBModel,
            "model_name": "facebook/hubert-large-ll60k",
            "hidden_units": 64,
            "n_gru": 3,
            "output_size": None  # Will be set dynamically
        },
        "Wav2Vec2ConvLSTMModel": {
            "model" : Wav2Vec2ConvLSTMModel,
            "model_name": "facebook/wav2vec2-base",
            "hidden_units": 128,
            "n_lstm": 2,
            "output_size": None  # Will be set dynamically
        },
            "RespBertCNNModelV2": {
            'model' : RespBertCNNModelV2,
            "model_name": "microsoft/wavlm-large",
            "hidden_units": 128,
            "output_size": None  
        },
            "RespBertLSTMModelV2": {
            'model': RespBertLSTMModelV2,
            "model_name": "microsoft/wavlm-large",
            "hidden_units": 128,
            "n_lstm": 3,
            "output_size": None  
        }
            ,
            "RespBertLSTMCNNTransformerModel": {
            'model': RespBertLSTMCNNTransformerModel,
            "model_name": "microsoft/wavlm-large",
            "hidden_units": 1024,
            "n_lstm": 2,
            "output_size": None  
        }
    }
    

    
    # Train and data parameters
    epochs = 60
    batch_size = 10
    window_size = 30
    step_size = 6
    data_parts = 4 # aka folds
    early_stopping_patience = 10
    
    config = model_config["RespBertLSTMCNNTransformerModel"]
    

    ## same wav2vec2 base model and pipeline used in the paper
    #processor = AutoProcessor.from_pretrained(config["model_name"])
    processor = Wav2Vec2FeatureExtractor.from_pretrained(config["model_name"])

    train(
        path_to_data=path+"ComParE2020_Breathing/wav/",
        path_to_labels=path+"ComParE2020_Breathing/lab/",
        window_size=window_size,
        batch_size=batch_size,
        config = config,
        step_size=step_size,
        data_parts= data_parts ,
        early_stopping_patience= early_stopping_patience,
        epochs= epochs,
        processor = processor
    )

Fold 1/4
(864, 480000)
750


/home/gdwildt/.conda/envs/thesis/lib/python3.11/site-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
Epoch 1/60:   0%|          | 0/87 [00:00<?, ?it/s]/home/gdwildt/.conda/envs/thesis/lib/python3.11/site-packages/torch/nn/functional.py:5193: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
Epoch 1/60:   0%|          | 0/87 [00:02<?, ?it/s]

torch.Size([10, 1499, 1024])
torch.Size([10, 1499, 3072])
torch.Size([1499, 10, 3072])
torch.Size([1499, 10, 3072])
torch.Size([1499, 750])


RuntimeError: The size of tensor a (1499) must match the size of tensor b (10) at non-singleton dimension 0

In [ ]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import KFold
from datetime import datetime
#from torch.utils.tensorboard import SummaryWriter
import torchaudio
from typing import List, Tuple
from pt_utils import *
from pt_dataset import *
from pt_models import *
from pt_utils import *
from tensorboardX import SummaryWriter
from transformers import get_cosine_schedule_with_warmup


def create_run_directory():
    base_dir = "pt_runs_SOTA"
    timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
    run_dir = os.path.join(base_dir, timestamp)
    os.makedirs(run_dir, exist_ok=True)
    return run_dir

def _calculate_flattened_accuracy(average, ground_truth_labels):
    s_acc = 0
    for b in range(len(ground_truth_labels)):
        s, _ = scipy.stats.pearsonr(average[b], ground_truth_labels[b])
        s_acc += s
    return s_acc / len(ground_truth_labels)

def _choose_real_labs_only_with_filenames(labels, filenames):
    return labels[labels['filename'].isin(filenames)]

def _get_ground_truth_labels(ground_truth_names, labels):
    ground_truth_labels = []
    for batch_name in ground_truth_names:
        ground_truth_label = _choose_real_labs_only_with_filenames(labels, [batch_name])
        ground_truth_labels.append(ground_truth_label)
    return np.array(ground_truth_labels)[:, :, -1].astype(np.float32)

def train(path_to_data, path_to_labels, window_size=16, step_size=6, data_parts=4, epochs=100, batch_size=10, early_stopping_patience=20, config = None, processor = None):
    run_dir = create_run_directory()
    log_dir = os.path.join(run_dir, "logs")
    os.makedirs(log_dir, exist_ok=True)

    # Parameters
    length_sequence = window_size 
    step_sequence = step_size

    # Load and prepare data
    train_data, train_labels, train_dict, frame_rate = load_data(path_to_data, path_to_labels, 'train')
    devel_data, devel_labels, devel_dict, frame_rate = load_data(path_to_data, path_to_labels, 'devel')
    test_data, test_labels, test_dict, frame_rate = load_data(path_to_data, path_to_labels, 'test')

    # Combine train and devel data
    all_data = np.concatenate((train_data, devel_data), axis=0)
    all_labels = pd.concat([train_labels, devel_labels])
    all_dict = np.concatenate((list(train_dict.values()), list(devel_dict.values())), axis=0)

    # Prepare data
    prepared_data, prepared_labels, prepared_labels_timesteps = prepare_data(all_data, all_labels, all_dict, frame_rate, length_sequence * 16000, step_sequence * 16000)
    prepared_test_data, prepared_test_labels, prepared_test_labels_timesteps = prepare_data(test_data, test_labels, test_dict, frame_rate, length_sequence * 16000, step_sequence * 16000)

    # Create CSV file for storing fold indices
    fold_indices_df = pd.DataFrame(columns=['Fold', 'Train_Indices', 'Val_Indices'])

    # Cross-validation
    kf = KFold(n_splits=data_parts)
    fold_metrics = []
    # To accumulate metrics across folds for each epoch
    train_acc_epoch = []
    val_acc_epoch = []
    test_acc_epoch = []

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    
    config["output_size"] = prepared_labels.shape[-1]
    writer = SummaryWriter(log_dir=os.path.join(log_dir,config["model_name"]))

    for fold, (train_index, val_index) in enumerate(kf.split(prepared_data)):
        print(f"Fold {fold + 1}/{data_parts}")
        best_model_path = f"{run_dir}/best_model_fold{fold+1}"
        # Save fold indices
        fold_indices_df = fold_indices_df._append({
            'Fold': fold + 1,
            'Train_Indices': train_index.tolist(),
            'Val_Indices': val_index.tolist()
        }, ignore_index=True)

        # Split data
        train_d, val_d = prepared_data[train_index], prepared_data[val_index]
        train_lbs, val_lbs = prepared_labels[train_index], prepared_labels[val_index]
        train_timesteps, val_timesteps = prepared_labels_timesteps[train_index], prepared_labels_timesteps[val_index]
        
        # Reshape data
        train_d, train_lbs = reshaping_data_for_model(train_d, train_lbs)
        val_d, val_lbs = reshaping_data_for_model(val_d, val_lbs)
        test_d, test_lbs = reshaping_data_for_model(prepared_test_data, prepared_test_labels)
        
        print(train_d.shape)

        # Create datasets
        #train_dataset = BreathingDataset(train_d, train_lbs, processor, window_size, step_sequence, augment=True)
        train_dataset = BreathingDataset(train_d, train_lbs, processor,window_size, step_sequence)
        val_dataset = BreathingDataset(val_d, val_lbs, processor, window_size, step_sequence)
        test_dataset = BreathingDataset(test_d, test_lbs, processor, window_size, step_sequence)

        # Create DataLoaders
        train_loader = DataLoader(train_dataset, batch_size=batch_size,num_workers=2, shuffle=True, collate_fn=train_dataset.collate_fn)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, num_workers=1, collate_fn=val_dataset.collate_fn)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, num_workers=1, collate_fn=test_dataset.collate_fn)
        print(config["output_size"])
        # Create and initialize model
        model = config["model"](config).to(device)
        
        #### training optimiser parameters fror apple
        #learning_rate = 0.005 
        #optimizer = optim.Adam(model.parameters(), lr=learning_rate)

        #### training optimiser parameters fror harma_2023 VRB model 
        learning_rate = 0.01 # From the paper
        optimizer = optim.Adam(model.parameters())       
        
        best_val_loss = float('inf')
        best_val_loss_flat = float('inf')
        early_stopping_counter = 0
        # To accumulate metrics across folds for each epoch
        train_acc = []
        val_acc = []
        test_acc = []
        for epoch in range(epochs):

            model.train()
            train_loss = 0.0
            progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")
            
            for batch_d, batch_lbs in progress_bar:
                optimizer.zero_grad()
                input_values = batch_d.to(device)
                batch_lbs = batch_lbs.to(device)
                outputs = model(input_values)
                loss = correlation_coefficient_loss(outputs, batch_lbs)
                loss.backward()
                optimizer.step()
                train_loss += loss.item()
                progress_bar.set_postfix({'train_loss': f'{train_loss/(progress_bar.n+1):.4f}'})


            train_loss /= len(train_loader)

            # Combined validation loop
            model.eval()
            val_loss = 0.0
            val_pred = []
            with torch.no_grad():
                for batch_d, batch_lbs in val_loader:
                    input_values = batch_d.to(device)
                    batch_lbs = batch_lbs.to(device)
                    
                    outputs = model(input_values)
                    loss = correlation_coefficient_loss(outputs, batch_lbs)
                    val_loss += loss.item()
                    val_pred.extend(outputs.cpu().numpy())

            val_loss /= len(val_loader)

            # Calculate validation metrics
            val_pred = np.array(val_pred).reshape(val_timesteps.shape)
            val_ground_truth = _get_ground_truth_labels([all_dict[i] for i in val_index], all_labels)
            val_pred_flat = unsplit_data_ogsize(val_pred, window_size, step_sequence, 25, val_ground_truth.shape[-1])
            val_prc_coef = _calculate_flattened_accuracy(val_pred_flat, val_ground_truth)
            
            # Accumulate metrics for this fold and epoch
            train_acc.append(1- train_loss)
            val_acc.append(1- val_loss)

            # Log metrics
            writer.add_scalar(f"Loss/train_fold_{fold + 1}", train_loss, epoch)
            writer.add_scalar(f"Loss/val_fold_{fold + 1}", val_loss, epoch)
            writer.add_scalar(f"Pearson/val_fold_{fold + 1}", val_prc_coef, epoch)

            print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Pearson: {val_prc_coef:.4f}")

            # Check if validation loss improved
            if val_loss < best_val_loss:
                print(f"Validation loss improved from {best_val_loss:.4f} to {val_loss:.4f}. Saving best model...")
                best_val_loss = val_loss
                best_val_loss_flat = val_prc_coef
                early_stopping_counter = 0

                # Save the best model
                torch.save(model.state_dict(), best_model_path)
            else:
                early_stopping_counter += 1
                print(f"Validation loss did not improve for {early_stopping_counter} epochs.")
                #model.load_state_dict(torch.load(best_model_path))


            # Early stopping
            if early_stopping_counter >= early_stopping_patience:
                print(f"Early stopping triggered at epoch {epoch + 1}. Loading best model.")
                # Load the best model's weights
                model.load_state_dict(torch.load(best_model_path))
                break

        test_pred = []
        test_loss = 0.0
        with torch.no_grad():
            for batch_d, batch_lbs in test_loader:
                input_values = batch_d.to(device)
                batch_lbs = batch_lbs.to(device)
                
                outputs = model(input_values)
                loss = correlation_coefficient_loss(outputs, batch_lbs)
                test_loss += loss.item()
                test_pred.extend(outputs.cpu().numpy())

        test_loss /= len(test_loader)
        test_pred = np.array(test_pred).reshape(prepared_test_labels_timesteps.shape)
        test_ground_truth = _get_ground_truth_labels(list(test_dict.values()), test_labels)
        test_pred_flat = unsplit_data_ogsize(test_pred, window_size, step_sequence, 25, test_ground_truth.shape[-1])
        test_prc_coef = _calculate_flattened_accuracy(test_pred_flat, test_ground_truth)

        print(f"Fold {fold + 1}:")
        print(f"  Validation Pearson Coefficient  acc: {1- best_val_loss}")
        print(f"  Validation Pearson Coefficient flat acc: {best_val_loss_flat}")
        print(f"  Test acc: {1- test_loss}")
        print(f"  Test Pearson Coefficient acc(flattened): {test_prc_coef}")

        fold_metrics.append({
            'Fold': fold + 1,
            'val_prc_acc': 1- best_val_loss,
            'val_prc_acc_flat': best_val_loss_flat,
            'test_acc': 1- test_loss,
            'test_prc_flat': test_prc_coef
        })


                # Log fold-specific metrics as tables
        fold_table = f"| Fold | Val Pearson Acc | Val Pearson Flat | Test Acc | Test Pearson Flat |\n" \
                     f"|------|-----------------|------------------|----------|-------------------|\n" \
                     f"| {fold + 1} | {1 - val_loss:.4f} | {val_prc_coef:.4f} | {1 - test_loss:.4f} | {test_prc_coef:.4f} |\n"
        writer.add_text(f"Fold_{fold + 1}_Metrics", fold_table)
        # Accumulate fold metrics across all folds
        train_acc_epoch.append(train_acc)
        val_acc_epoch.append(train_acc)


    
        # After all folds, compute and log the average metrics per epoch across all folds
    for epoch in range(epochs):
        avg_train_loss = np.mean([fold_losses[epoch] for fold_losses in train_acc_epoch if len(fold_losses) > epoch])
        avg_val_loss = np.mean([fold_losses[epoch] for fold_losses in val_acc_epoch if len(fold_losses) > epoch])

        # Log the averaged metrics for the epoch across all folds
        writer.add_scalar("Average_acc/train", avg_train_loss, epoch)
        writer.add_scalar("Average_acc/val", avg_val_loss, epoch)
            

    # Calculate average metrics
    avg_metrics = {key: np.mean([fold[key] for fold in fold_metrics if key != 'Fold']) for key in fold_metrics[0].keys() if key != 'Fold'}
        # Log the final average table
    avg_table = "| Fold | Val Pearson Acc | Val Pearson Flat | Test Acc | Test Pearson Flat |\n" \
                "|------|-----------------|------------------|----------|-------------------|\n" \
                f"| Average | {avg_metrics['val_prc_acc']:.4f} | {avg_metrics['val_prc_acc_flat']:.4f} | {avg_metrics['test_acc']:.4f} | {avg_metrics['test_prc_flat']:.4f} |\n"
    writer.add_text("Average_Metrics", avg_table)
    # Add average metrics to results
    avg_metrics['Fold'] = 'Average'
    fold_metrics.append(avg_metrics)

    # save averga date to CSV
    results_df = pd.DataFrame(fold_metrics)
    csv_path = os.path.join(run_dir, 'fold_results.csv')
    results_df.to_csv(csv_path, index=False)
    
    # Save fold indices CSV
    fold_indices_df.to_csv(os.path.join(run_dir, 'fold_indices.csv'), index=False)
    
    writer.close()


    print("\nTraining completed.")
    print("Average metrics across all folds:")
    for key, value in avg_metrics.items():
        print(f"  {key}: {value}")


if __name__ == "__main__":
    ## Path to data
    #path = "/home/glenn/Downloads/"
    path = "../DATA/"


    # Model parameters
    model_config = {
        "VRBModel": {
            "model" : VRBModel,
            "model_name": "facebook/hubert-large-ll60k",
            "hidden_units": 64,
            "n_gru": 3,
            "output_size": None  # Will be set dynamically
        },
        "Wav2Vec2ConvLSTMModel": {
            "model" : Wav2Vec2ConvLSTMModel,
            "model_name": "facebook/wav2vec2-base",
            "hidden_units": 128,
            "n_lstm": 2,
            "output_size": None  # Will be set dynamically
        },
            "RespBertCNNModelV2": {
            'model' : RespBertCNNModelV2,
            "model_name": "microsoft/wavlm-large",
            "hidden_units": 128,
            "output_size": None  
        },
            "RespBertLSTMModelV2": {
            'model': RespBertLSTMModelV2,
            "model_name": "microsoft/wavlm-large",
            "hidden_units": 128,
            "n_lstm": 3,
            "output_size": None  
        }
            ,
            "RespBertLSTMCNNTransformerModel": {
            'model': RespBertLSTMCNNTransformerModel,
            "model_name": "microsoft/wavlm-large",
            "hidden_units": 1024,
            "n_lstm": 2,
            "output_size": None  
        }
    }
    

    
    # Train and data parameters
    epochs = 60
    batch_size = 10
    window_size = 30
    step_size = 6
    data_parts = 4 # aka folds
    early_stopping_patience = 10
    
    config = model_config["RespBertLSTMCNNTransformerModel"]
    

    ## same wav2vec2 base model and pipeline used in the paper
    #processor = AutoProcessor.from_pretrained(config["model_name"])
    processor = Wav2Vec2FeatureExtractor.from_pretrained(config["model_name"])

    train(
        path_to_data=path+"ComParE2020_Breathing/wav/",
        path_to_labels=path+"ComParE2020_Breathing/lab/",
        window_size=window_size,
        batch_size=batch_size,
        config = config,
        step_size=step_size,
        data_parts= data_parts ,
        early_stopping_patience= early_stopping_patience,
        epochs= epochs,
        processor = processor
    )

Fold 1/4
(864, 480000)
750


/home/gdwildt/.conda/envs/thesis/lib/python3.11/site-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
Epoch 1/60:   0%|          | 0/87 [00:00<?, ?it/s]/home/gdwildt/.conda/envs/thesis/lib/python3.11/site-packages/torch/nn/functional.py:5193: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


torch.Size([10, 1499, 1024])
torch.Size([10, 1499, 3072])
torch.Size([1499, 10, 3072])
torch.Size([10, 1499, 3072])
torch.Size([10, 750])


Epoch 1/60:   1%|          | 1/87 [00:03<05:06,  3.56s/it, train_loss=1.0101]

torch.Size([10, 1499, 1024])
torch.Size([10, 1499, 3072])
torch.Size([1499, 10, 3072])
torch.Size([10, 1499, 3072])
torch.Size([10, 750])


...

Epoch 1/60:  44%|████▎     | 38/87 [01:04<01:17,  1.57s/it, train_loss=1.0030]

torch.Size([10, 1499, 1024])
torch.Size([10, 1499, 3072])
torch.Size([1499, 10, 3072])
torch.Size([10, 1499, 3072])
torch.Size([10, 750])


Epoch 1/60:  45%|████▍     | 39/87 [01:06<01:15,  1.57s/it, train_loss=1.0041]

torch.Size([10, 1499, 1024])
torch.Size([10, 1499, 3072])
torch.Size([1499, 10, 3072])
torch.Size([10, 1499, 3072])
torch.Size([10, 750])


Epoch 1/60:  46%|████▌     | 40/87 [01:07<01:17,  1.64s/it, train_loss=1.0038]

In [ ]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import KFold
from datetime import datetime
#from torch.utils.tensorboard import SummaryWriter
import torchaudio
from typing import List, Tuple
from pt_utils import *
from pt_dataset import *
from pt_models import *
from pt_utils import *
from tensorboardX import SummaryWriter
from transformers import get_cosine_schedule_with_warmup


def create_run_directory():
    base_dir = "pt_runs_SOTA"
    timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
    run_dir = os.path.join(base_dir, timestamp)
    os.makedirs(run_dir, exist_ok=True)
    return run_dir

def _calculate_flattened_accuracy(average, ground_truth_labels):
    s_acc = 0
    for b in range(len(ground_truth_labels)):
        s, _ = scipy.stats.pearsonr(average[b], ground_truth_labels[b])
        s_acc += s
    return s_acc / len(ground_truth_labels)

def _choose_real_labs_only_with_filenames(labels, filenames):
    return labels[labels['filename'].isin(filenames)]

def _get_ground_truth_labels(ground_truth_names, labels):
    ground_truth_labels = []
    for batch_name in ground_truth_names:
        ground_truth_label = _choose_real_labs_only_with_filenames(labels, [batch_name])
        ground_truth_labels.append(ground_truth_label)
    return np.array(ground_truth_labels)[:, :, -1].astype(np.float32)

def train(path_to_data, path_to_labels, window_size=16, step_size=6, data_parts=4, epochs=100, batch_size=10, early_stopping_patience=20, config = None, processor = None):
    run_dir = create_run_directory()
    log_dir = os.path.join(run_dir, "logs")
    os.makedirs(log_dir, exist_ok=True)

    # Parameters
    length_sequence = window_size 
    step_sequence = step_size

    # Load and prepare data
    train_data, train_labels, train_dict, frame_rate = load_data(path_to_data, path_to_labels, 'train')
    devel_data, devel_labels, devel_dict, frame_rate = load_data(path_to_data, path_to_labels, 'devel')
    test_data, test_labels, test_dict, frame_rate = load_data(path_to_data, path_to_labels, 'test')

    # Combine train and devel data
    all_data = np.concatenate((train_data, devel_data), axis=0)
    all_labels = pd.concat([train_labels, devel_labels])
    all_dict = np.concatenate((list(train_dict.values()), list(devel_dict.values())), axis=0)

    # Prepare data
    prepared_data, prepared_labels, prepared_labels_timesteps = prepare_data(all_data, all_labels, all_dict, frame_rate, length_sequence * 16000, step_sequence * 16000)
    prepared_test_data, prepared_test_labels, prepared_test_labels_timesteps = prepare_data(test_data, test_labels, test_dict, frame_rate, length_sequence * 16000, step_sequence * 16000)

    # Create CSV file for storing fold indices
    fold_indices_df = pd.DataFrame(columns=['Fold', 'Train_Indices', 'Val_Indices'])

    # Cross-validation
    kf = KFold(n_splits=data_parts)
    fold_metrics = []
    # To accumulate metrics across folds for each epoch
    train_acc_epoch = []
    val_acc_epoch = []
    test_acc_epoch = []

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    
    config["output_size"] = prepared_labels.shape[-1]
    writer = SummaryWriter(log_dir=os.path.join(log_dir,config["model_name"]))

    for fold, (train_index, val_index) in enumerate(kf.split(prepared_data)):
        print(f"Fold {fold + 1}/{data_parts}")
        best_model_path = f"{run_dir}/best_model_fold{fold+1}"
        # Save fold indices
        fold_indices_df = fold_indices_df._append({
            'Fold': fold + 1,
            'Train_Indices': train_index.tolist(),
            'Val_Indices': val_index.tolist()
        }, ignore_index=True)

        # Split data
        train_d, val_d = prepared_data[train_index], prepared_data[val_index]
        train_lbs, val_lbs = prepared_labels[train_index], prepared_labels[val_index]
        train_timesteps, val_timesteps = prepared_labels_timesteps[train_index], prepared_labels_timesteps[val_index]
        
        # Reshape data
        train_d, train_lbs = reshaping_data_for_model(train_d, train_lbs)
        val_d, val_lbs = reshaping_data_for_model(val_d, val_lbs)
        test_d, test_lbs = reshaping_data_for_model(prepared_test_data, prepared_test_labels)
        
        print(train_d.shape)

        # Create datasets
        #train_dataset = BreathingDataset(train_d, train_lbs, processor, window_size, step_sequence, augment=True)
        train_dataset = BreathingDataset(train_d, train_lbs, processor,window_size, step_sequence)
        val_dataset = BreathingDataset(val_d, val_lbs, processor, window_size, step_sequence)
        test_dataset = BreathingDataset(test_d, test_lbs, processor, window_size, step_sequence)

        # Create DataLoaders
        train_loader = DataLoader(train_dataset, batch_size=batch_size,num_workers=2, shuffle=True, collate_fn=train_dataset.collate_fn)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, num_workers=1, collate_fn=val_dataset.collate_fn)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, num_workers=1, collate_fn=test_dataset.collate_fn)
        print(config["output_size"])
        # Create and initialize model
        model = config["model"](config).to(device)
        
        #### training optimiser parameters fror apple
        #learning_rate = 0.005 
        #optimizer = optim.Adam(model.parameters(), lr=learning_rate)

        #### training optimiser parameters fror harma_2023 VRB model 
        learning_rate = 0.01 # From the paper
        optimizer = optim.Adam(model.parameters())       
        
        best_val_loss = float('inf')
        best_val_loss_flat = float('inf')
        early_stopping_counter = 0
        # To accumulate metrics across folds for each epoch
        train_acc = []
        val_acc = []
        test_acc = []
        for epoch in range(epochs):

            model.train()
            train_loss = 0.0
            progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")
            
            for batch_d, batch_lbs in progress_bar:
                optimizer.zero_grad()
                input_values = batch_d.to(device)
                batch_lbs = batch_lbs.to(device)
                outputs = model(input_values)
                loss = correlation_coefficient_loss(outputs, batch_lbs)
                loss.backward()
                optimizer.step()
                train_loss += loss.item()
                progress_bar.set_postfix({'train_loss': f'{train_loss/(progress_bar.n+1):.4f}'})


            train_loss /= len(train_loader)

            # Combined validation loop
            model.eval()
            val_loss = 0.0
            val_pred = []
            with torch.no_grad():
                for batch_d, batch_lbs in val_loader:
                    input_values = batch_d.to(device)
                    batch_lbs = batch_lbs.to(device)
                    
                    outputs = model(input_values)
                    loss = correlation_coefficient_loss(outputs, batch_lbs)
                    val_loss += loss.item()
                    val_pred.extend(outputs.cpu().numpy())

            val_loss /= len(val_loader)

            # Calculate validation metrics
            val_pred = np.array(val_pred).reshape(val_timesteps.shape)
            val_ground_truth = _get_ground_truth_labels([all_dict[i] for i in val_index], all_labels)
            val_pred_flat = unsplit_data_ogsize(val_pred, window_size, step_sequence, 25, val_ground_truth.shape[-1])
            val_prc_coef = _calculate_flattened_accuracy(val_pred_flat, val_ground_truth)
            
            # Accumulate metrics for this fold and epoch
            train_acc.append(1- train_loss)
            val_acc.append(1- val_loss)

            # Log metrics
            writer.add_scalar(f"Loss/train_fold_{fold + 1}", train_loss, epoch)
            writer.add_scalar(f"Loss/val_fold_{fold + 1}", val_loss, epoch)
            writer.add_scalar(f"Pearson/val_fold_{fold + 1}", val_prc_coef, epoch)

            print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Pearson: {val_prc_coef:.4f}")

            # Check if validation loss improved
            if val_loss < best_val_loss:
                print(f"Validation loss improved from {best_val_loss:.4f} to {val_loss:.4f}. Saving best model...")
                best_val_loss = val_loss
                best_val_loss_flat = val_prc_coef
                early_stopping_counter = 0

                # Save the best model
                torch.save(model.state_dict(), best_model_path)
            else:
                early_stopping_counter += 1
                print(f"Validation loss did not improve for {early_stopping_counter} epochs.")
                #model.load_state_dict(torch.load(best_model_path))


            # Early stopping
            if early_stopping_counter >= early_stopping_patience:
                print(f"Early stopping triggered at epoch {epoch + 1}. Loading best model.")
                # Load the best model's weights
                model.load_state_dict(torch.load(best_model_path))
                break

        test_pred = []
        test_loss = 0.0
        with torch.no_grad():
            for batch_d, batch_lbs in test_loader:
                input_values = batch_d.to(device)
                batch_lbs = batch_lbs.to(device)
                
                outputs = model(input_values)
                loss = correlation_coefficient_loss(outputs, batch_lbs)
                test_loss += loss.item()
                test_pred.extend(outputs.cpu().numpy())

        test_loss /= len(test_loader)
        test_pred = np.array(test_pred).reshape(prepared_test_labels_timesteps.shape)
        test_ground_truth = _get_ground_truth_labels(list(test_dict.values()), test_labels)
        test_pred_flat = unsplit_data_ogsize(test_pred, window_size, step_sequence, 25, test_ground_truth.shape[-1])
        test_prc_coef = _calculate_flattened_accuracy(test_pred_flat, test_ground_truth)

        print(f"Fold {fold + 1}:")
        print(f"  Validation Pearson Coefficient  acc: {1- best_val_loss}")
        print(f"  Validation Pearson Coefficient flat acc: {best_val_loss_flat}")
        print(f"  Test acc: {1- test_loss}")
        print(f"  Test Pearson Coefficient acc(flattened): {test_prc_coef}")

        fold_metrics.append({
            'Fold': fold + 1,
            'val_prc_acc': 1- best_val_loss,
            'val_prc_acc_flat': best_val_loss_flat,
            'test_acc': 1- test_loss,
            'test_prc_flat': test_prc_coef
        })


                # Log fold-specific metrics as tables
        fold_table = f"| Fold | Val Pearson Acc | Val Pearson Flat | Test Acc | Test Pearson Flat |\n" \
                     f"|------|-----------------|------------------|----------|-------------------|\n" \
                     f"| {fold + 1} | {1 - val_loss:.4f} | {val_prc_coef:.4f} | {1 - test_loss:.4f} | {test_prc_coef:.4f} |\n"
        writer.add_text(f"Fold_{fold + 1}_Metrics", fold_table)
        # Accumulate fold metrics across all folds
        train_acc_epoch.append(train_acc)
        val_acc_epoch.append(train_acc)


    
        # After all folds, compute and log the average metrics per epoch across all folds
    for epoch in range(epochs):
        avg_train_loss = np.mean([fold_losses[epoch] for fold_losses in train_acc_epoch if len(fold_losses) > epoch])
        avg_val_loss = np.mean([fold_losses[epoch] for fold_losses in val_acc_epoch if len(fold_losses) > epoch])

        # Log the averaged metrics for the epoch across all folds
        writer.add_scalar("Average_acc/train", avg_train_loss, epoch)
        writer.add_scalar("Average_acc/val", avg_val_loss, epoch)
            

    # Calculate average metrics
    avg_metrics = {key: np.mean([fold[key] for fold in fold_metrics if key != 'Fold']) for key in fold_metrics[0].keys() if key != 'Fold'}
        # Log the final average table
    avg_table = "| Fold | Val Pearson Acc | Val Pearson Flat | Test Acc | Test Pearson Flat |\n" \
                "|------|-----------------|------------------|----------|-------------------|\n" \
                f"| Average | {avg_metrics['val_prc_acc']:.4f} | {avg_metrics['val_prc_acc_flat']:.4f} | {avg_metrics['test_acc']:.4f} | {avg_metrics['test_prc_flat']:.4f} |\n"
    writer.add_text("Average_Metrics", avg_table)
    # Add average metrics to results
    avg_metrics['Fold'] = 'Average'
    fold_metrics.append(avg_metrics)

    # save averga date to CSV
    results_df = pd.DataFrame(fold_metrics)
    csv_path = os.path.join(run_dir, 'fold_results.csv')
    results_df.to_csv(csv_path, index=False)
    
    # Save fold indices CSV
    fold_indices_df.to_csv(os.path.join(run_dir, 'fold_indices.csv'), index=False)
    
    writer.close()


    print("\nTraining completed.")
    print("Average metrics across all folds:")
    for key, value in avg_metrics.items():
        print(f"  {key}: {value}")


if __name__ == "__main__":
    ## Path to data
    #path = "/home/glenn/Downloads/"
    path = "../DATA/"


    # Model parameters
    model_config = {
        "VRBModel": {
            "model" : VRBModel,
            "model_name": "facebook/hubert-large-ll60k",
            "hidden_units": 64,
            "n_gru": 3,
            "output_size": None  # Will be set dynamically
        },
        "Wav2Vec2ConvLSTMModel": {
            "model" : Wav2Vec2ConvLSTMModel,
            "model_name": "facebook/wav2vec2-base",
            "hidden_units": 128,
            "n_lstm": 2,
            "output_size": None  # Will be set dynamically
        },
            "RespBertCNNModelV2": {
            'model' : RespBertCNNModelV2,
            "model_name": "microsoft/wavlm-large",
            "hidden_units": 128,
            "output_size": None  
        },
            "RespBertLSTMModelV2": {
            'model': RespBertLSTMModelV2,
            "model_name": "microsoft/wavlm-large",
            "hidden_units": 128,
            "n_lstm": 3,
            "output_size": None  
        }
            ,
            "RespBertLSTMCNNTransformerModel": {
            'model': RespBertLSTMCNNTransformerModel,
            "model_name": "microsoft/wavlm-large",
            "hidden_units": 1024,
            "n_lstm": 2,
            "output_size": None  
        }
    }
    

    
    # Train and data parameters
    epochs = 60
    batch_size = 15
    window_size = 30
    step_size = 6
    data_parts = 4 # aka folds
    early_stopping_patience = 10
    
    config = model_config["RespBertLSTMCNNTransformerModel"]
    

    ## same wav2vec2 base model and pipeline used in the paper
    #processor = AutoProcessor.from_pretrained(config["model_name"])
    processor = Wav2Vec2FeatureExtractor.from_pretrained(config["model_name"])

    train(
        path_to_data=path+"ComParE2020_Breathing/wav/",
        path_to_labels=path+"ComParE2020_Breathing/lab/",
        window_size=window_size,
        batch_size=batch_size,
        config = config,
        step_size=step_size,
        data_parts= data_parts ,
        early_stopping_patience= early_stopping_patience,
        epochs= epochs,
        processor = processor
    )

Fold 1/4
(864, 480000)
750


/home/gdwildt/.conda/envs/thesis/lib/python3.11/site-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
Epoch 1/60:   0%|          | 0/58 [00:00<?, ?it/s]/home/gdwildt/.conda/envs/thesis/lib/python3.11/site-packages/torch/nn/functional.py:5193: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
Epoch 1/60:   0%|          | 0/58 [00:02<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 2.02 GiB. GPU 0 has a total capacity of 93.12 GiB of which 590.75 MiB is free. Including non-PyTorch memory, this process has 92.54 GiB memory in use. Of the allocated memory 90.94 GiB is allocated by PyTorch, and 954.10 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)